In [7]:
!pip install --upgrade pip
!pip install --upgrade tensorflow keras

import os
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import MaxNLocator
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from scipy.fft import fft2, ifft2, fftshift
import tensorflow as tf
from tensorflow.keras.datasets import mnist, fashion_mnist
from PIL import Image
import itertools

# -----------------------------
# Utility functions
# -----------------------------
def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)

def norm01(x):
    mn = np.min(x)
    mx = np.max(x)
    if mx - mn == 0:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def pad_image(img, P=3, mode='cyclic'):
    # img: 2D array
    if mode == 'cyclic':
        return np.pad(img, ((P,P),(P,P)), mode='wrap')
    elif mode == 'symmetric':
        return np.pad(img, ((P,P),(P,P)), mode='symmetric')
    else:
        return np.pad(img, ((P,P),(P,P)), mode='constant')

# AA amplification formula (manuscript): sin((2m+1) * theta)
def aa_amplitude(m, theta):
    return math.sin((2*m + 1) * theta)

# QPE amplitude spike synthetic model: a few exponential peaks across frequency bins
def synthetic_qpe_spectrum(bins=128, peak_locations=None, heights=None, widths=None):
    x = np.linspace(0, 1, bins)
    y = np.zeros_like(x)
    if peak_locations is None:
        peak_locations = [0.12, 0.44, 0.78]
    if heights is None:
        heights = [1.0, 2.2, 1.6]
    if widths is None:
        widths = [0.02, 0.01, 0.015]
    for loc,h,w in zip(peak_locations, heights, widths):
        y += h * np.exp(-0.5 * ((x - loc)/w)**2)
    # add a baseline noise floor
    y += 0.05 * np.random.rand(len(x))
    return x, y

# Frequency-domain "Hadamard-like" elementwise product demonstration
def freq_hadamard_demo(img, kern):
    Fimg = fftshift(fft2(img))
    Fkern = fftshift(fft2(kern, s=img.shape))
    # elementwise multiply in frequency domain (Hadamard-like)
    Frecon = Fimg * Fkern
    recon = np.real(ifft2(fftshift(Frecon)))
    return Fimg, Fkern, Frecon, recon

# Pretty label helper for subplots
def label_ax(ax, label, xpos=-0.05, ypos=1.08, weight='bold'):
    ax.text(xpos, ypos, label, transform=ax.transAxes,
            fontsize=12, fontweight=weight, va='top')

# -----------------------------
# Load datasets (MNIST & FashionMNIST)
# -----------------------------
print("Loading MNIST and FashionMNIST...")
(x1_train, y1_train), (x1_test, y1_test) = mnist.load_data()
(x2_train, y2_train), (x2_test, y2_test) = fashion_mnist.load_data()

# Convert to float [0,1]
x1_train = x1_train.astype(np.float32) / 255.0
x1_test  = x1_test.astype(np.float32) / 255.0
x2_train = x2_train.astype(np.float32) / 255.0
x2_test  = x2_test.astype(np.float32) / 255.0

# select small representative subsets for plotting to speed computation
n_samples_display = 5
mnist_samples = x1_train[:n_samples_display]
fashion_samples = x2_train[:n_samples_display]

# -----------------------------
# Generate values based on manuscript equations and settings
# -----------------------------
# 1) Spectral resolution bound vs P (Eq similar to manuscript: Delta_xi_min >= const * ||[...]||^-1 * e^{-zeta P^2})
zeta = 0.25
kernel_norm = 2.5  # approximate from manuscript notes
hbar = 1.0
P_vals = np.arange(0, 9)  # padding sizes 0..8
Delta_xi_min = (hbar / 2.0) * (1.0 / kernel_norm) * np.exp(-zeta * (P_vals**2))

# 2) Fidelity vs padding P (epsilon(P) = exp(-kappa P^2) + zeta/(product D^alpha))
kappa = 0.5
zeta_fid = 1e-2
Dprod = 28**2  # very rough proxy spatial dims product
alpha = 1.0
epsilon_P = np.exp(-kappa * P_vals**2) + zeta_fid / (Dprod**alpha)
fidelity = 1.0 - epsilon_P

# 3) QPE synthetic spectrum (simulate QPE amplitude spikes)
bins = 256
qpe_x, qpe_y = synthetic_qpe_spectrum(bins=bins, peak_locations=[0.15, 0.5, 0.82],
                                     heights=[3.2, 72.37, 18.0], widths=[0.012, 0.004, 0.01])

# 4) AA amplification: choose theta from manuscript (theta = arcsin(1/sqrt(8)) for example)
theta = math.asin(1.0 / math.sqrt(8.0))
m_iters = np.arange(0, 7)  # 0..6 iterations
aa_vals = [aa_amplitude(m, theta) for m in m_iters]

# 5) Intensity transforms (manuscript suggests QPE amplifies intensity)
# We'll compute mean intensity across sample images for original, padded, qpe-augmented (synthetic)
def compute_mean_max(images):
    ms = [np.mean(img) for img in images]
    Mx = [np.max(img) for img in images]
    return np.mean(ms), np.mean(Mx)

mnist_mean, mnist_max = compute_mean_max(mnist_samples)
fashion_mean, fashion_max = compute_mean_max(fashion_samples)
# synthetic QPE intensity scale factors (from file, e.g., 72.4 for MNIST QPE max)
qpe_gain_mnist_mean = 3.71  # from manuscript
qpe_gain_mnist_max = 72.37
qpe_gain_fashion_mean = 3.42
qpe_gain_fashion_max = 131.2

# 6) Feature-space t-SNE: produce a small set of vectors (we'll use flattened images as 'features')
X_tsne = np.vstack([x1_train[:500].reshape(500, -1), x2_train[:500].reshape(500, -1)])
labels_tsne = np.hstack([y1_train[:500], y2_train[:500] + 10])  # keep classes separate for coloring
tsne = TSNE(n_components=2, perplexity=40, init='random', random_state=0, learning_rate=200)
X_embedded = tsne.fit_transform(X_tsne)

# 7) Synthetic "training" curves and uncertainty for panel of learning dynamics
epochs = np.arange(1, 51)
train_acc = 0.5 + 0.48 * (1 - np.exp(-0.12 * (epochs - 1)))  # asymptotic to ~0.98
val_acc = train_acc - 0.02 - 0.06 * np.exp(-0.08 * (epochs - 1))
quantum_entropy = 4.9 - 0.02 * epochs + 0.1 * np.sin(0.25 * epochs)  # synthetic tracking metric
ensemble_loss_mean = 1.2 * np.exp(-0.06 * epochs) + 0.05
ensemble_loss_std = 0.12 * np.exp(-0.03 * epochs) + 0.02

# 8) Confusion matrix for synthetic predictions on small test sample (we simulate a classifier)
np.random.seed(0)
y_true = y2_test[:200]  # fashion test labels
# simulate predicted labels with some confusion
y_pred = y_true.copy()
# add errors: flip 12% randomly
flip_idx = np.random.choice(len(y_true), size=int(0.12 * len(y_true)), replace=False)
for idx in flip_idx:
    y_pred[idx] = (y_pred[idx] + np.random.randint(1,10)) % 10
cm = confusion_matrix(y_true, y_pred)

# 9) Frequency domain Hadamard demo for an example MNIST image and a synthetic kernel
sample_img = x1_train[3]
kernel = np.zeros((7,7))
kernel[3,3] = 1.0
# create a smoothed kernel
for i in range(7):
    for j in range(7):
        kernel[i,j] = math.exp(-((i-3)**2 + (j-3)**2) / (2*1.5**2))
kernel = kernel / np.sum(kernel)
Fimg, Fkern, Frecon, recon_img = freq_hadamard_demo(sample_img, kernel)

# -----------------------------
# Print numeric values used (so authors can paste them into manuscript)
# -----------------------------
print("\n--- Numeric values used for plotting (printed arrays saved to qcnn_plot_values.npz) ---")
print("P_vals:", P_vals)
print("Delta_xi_min:", np.round(Delta_xi_min, 6))
print("fidelity (1 - epsilon):", np.round(fidelity, 6))
print("qpe spectrum sample (first 10 y values):", np.round(qpe_y[:10], 6))
print("aa_vals for m=0..6:", np.round(aa_vals, 6))
print("mnist_mean, mnist_max:", mnist_mean, mnist_max)
print("fashion_mean, fashion_max:", fashion_mean, fashion_max)
print("synthetic training final train_acc[-1]:", train_acc[-1], "val_acc[-1]:", val_acc[-1])
print("ensemble loss mean/ std (epoch 50):", ensemble_loss_mean[-1], ensemble_loss_std[-1])
print("confusion matrix shape:", cm.shape)

# Save arrays used
np.savez('qcnn_plot_values.npz',
         P_vals=P_vals,
         Delta_xi_min=Delta_xi_min,
         fidelity=fidelity,
         qpe_x=qpe_x, qpe_y=qpe_y,
         aa_m_iters=m_iters, aa_vals=aa_vals,
         mnist_mean=mnist_mean, mnist_max=mnist_max,
         fashion_mean=fashion_mean, fashion_max=fashion_max,
         X_embedded=X_embedded, labels_tsne=labels_tsne,
         epochs=epochs, train_acc=train_acc, val_acc=val_acc, quantum_entropy=quantum_entropy,
         ensemble_loss_mean=ensemble_loss_mean, ensemble_loss_std=ensemble_loss_std,
         confusion_matrix=cm,
         Fimg=np.abs(Fimg), Fkern=np.abs(Fkern), Frecon=np.abs(Frecon), recon_img=recon_img)

# -----------------------------
# Create multipanel figure
# -----------------------------
matplotlib.rcParams.update({'font.size': 10, 'figure.dpi': 150})
fig = plt.figure(figsize=(11.7, 16.5))  # A4 portrait like layout
gs = gridspec.GridSpec(6, 2, height_ratios=[1, 1, 1, 1, 1, 1], hspace=0.45, wspace=0.32)

# Panel (a): representative samples MNIST & Fashion side-by-side
ax = fig.add_subplot(gs[0, 0])
images_to_show = list(mnist_samples) + list(fashion_samples)
titles = ['MNIST {}'.format(i) for i in range(len(mnist_samples))] + ['Fash {}'.format(i) for i in range(len(fashion_samples))]
ncols = 5
for i, img in enumerate(images_to_show):
    sub = ax.inset_axes([i*0.18, 0.02, 0.16, 0.96]) if i < ncols else None
# We'll plot a grid manually
ax.clear()
rows = 2
cols = ncols
for i, img in enumerate(images_to_show):
    r = i // cols
    c = i % cols
    ax_img = fig.add_axes([0.06 + c*0.18, 0.90 - r*0.115, 0.14, 0.1])
    ax_img.imshow(img, cmap='gray', interpolation='nearest')
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    if i < n_samples_display:
        ax_img.set_title("MN {}".format(i), fontsize=8)
    else:
        ax_img.set_title("F {}".format(i - n_samples_display), fontsize=8)
# frame placeholder
ax.axis('off')
label_ax(ax, '(a) Representative samples (MNIST, FashionMNIST)')

# Panel (b): intensity means / maxs across transformations
ax = fig.add_subplot(gs[0, 1])
bars = ['Original MN mean', 'QPE MN mean (est)', 'Original FM mean', 'QPE FM mean (est)']
vals = [mnist_mean, mnist_mean * qpe_gain_mnist_mean, fashion_mean, fashion_mean * qpe_gain_fashion_mean]
ax.bar(range(len(vals)), vals)
ax.set_xticks(range(len(vals)))
ax.set_xticklabels(bars, rotation=20, ha='right')
ax.set_ylabel('Mean intensity')
ax.set_title('Intensity transforms under QPE (synthetic amplification)')
label_ax(ax, '(b)')

# Panel (c): QPE spectrum (large spike demonstration)
ax = fig.add_subplot(gs[1, 0])
ax.plot(qpe_x, qpe_y, lw=1.2)
ax.set_xlabel('Normalized frequency')
ax.set_ylabel('QPE amplitude (arb.)')
ax.set_title('Synthetic QPE spectrum (illustrative of phase amplification)')
ax.grid(True, alpha=0.25)
label_ax(ax, '(c)')

# Panel (d): AA amplitude vs iterations
ax = fig.add_subplot(gs[1, 1])
ax.plot(m_iters, aa_vals, marker='o', lw=1.5)
ax.set_xlabel('Grover iterations m')
ax.set_ylabel('Amplitude |sin((2m+1)θ)|')
ax.set_title('Amplitude Amplification (AA) behavior using θ from manuscript')
ax.set_xticks(m_iters)
ax.grid(True)
label_ax(ax, '(d)')

# Panel (e): Fidelity vs padding (tradeoff plot)
ax = fig.add_subplot(gs[2, 0])
ax.plot(P_vals, fidelity, marker='s', label='Fidelity')
ax.plot(P_vals, Delta_xi_min / np.max(Delta_xi_min), '--', label='Normed spectral resolution (Δξ_min)')
ax.set_xlabel('Padding P')
ax.set_ylabel('Fidelity / Normed Δξ_min')
ax.set_title('Fidelity and spectral resolution vs padding (manuscript model)')
ax.legend(frameon=False)
ax.grid(True)
label_ax(ax, '(e)')

# Panel (f): t-SNE of flattened 500+500 features (entangled vs classical style)
ax = fig.add_subplot(gs[2, 1])
cmap = plt.get_cmap('tab20')
ax.scatter(X_embedded[:500,0], X_embedded[:500,1], s=8, c='tab:blue', alpha=0.7, label='MNIST subset')
ax.scatter(X_embedded[500:,0], X_embedded[500:,1], s=8, c='tab:orange', alpha=0.7, label='Fashion subset')
ax.set_title('t-SNE visualization of combined feature sets (500 each)')
ax.legend(markerscale=2)
label_ax(ax, '(f)')

# Panel (g): Training curves + quantum entropy overlay
ax = fig.add_subplot(gs[3, 0])
ax.plot(epochs, train_acc, label='Train accuracy', lw=1.5)
ax.plot(epochs, val_acc, label='Val accuracy', lw=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax_twin = ax.twinx()
ax_twin.plot(epochs, quantum_entropy, label='Quantum entropy (scaled)', color='green', lw=1.0, alpha=0.7)
ax_twin.set_ylabel('Quantum entropy (bits)')
ax.set_title('Training dynamics and quantum entropy')
ax.legend(loc='upper left')
ax_twin.legend(loc='upper right')
label_ax(ax, '(g)')

# Panel (h): Loss convergence with ensemble uncertainty (shaded region)
ax = fig.add_subplot(gs[3, 1])
ax.plot(epochs, ensemble_loss_mean, label='Ensemble val loss (mean)')
ax.fill_between(epochs, ensemble_loss_mean - ensemble_loss_std, ensemble_loss_mean + ensemble_loss_std, alpha=0.25)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Loss convergence with ensemble uncertainty')
ax.grid(True)
label_ax(ax, '(h)')

# Panel (i): Feature variance vs accuracy synthetic coevolution
ax = fig.add_subplot(gs[4, 0])
feature_variance = 0.4 + 0.6 * (1 - np.exp(-0.1 * (epochs - 1)))
ax.plot(epochs, feature_variance, label='Feature variance (σ²_f)')
ax.plot(epochs, train_acc, label='Train acc (for reference)', linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Metric')
ax.set_title('Feature variance — accuracy coevolution (synthetic)')
ax.legend()
label_ax(ax, '(i)')

# Panel (j): Confusion matrix (fashion sample)
ax = fig.add_subplot(gs[4, 1])
im = ax.imshow(cm, interpolation='nearest', cmap='viridis')
ax.set_title('Simulated confusion matrix (FashionMNIST subset)')
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(j)')

# Panel (k): Frequency-domain Hadamard demo images (Fimg, Fkern magnitude)
ax = fig.add_subplot(gs[5, 0])
ax.imshow(np.log1p(np.abs(Fimg)), cmap='inferno')
ax.set_title('log|FFT(image)|')
ax.axis('off')
label_ax(ax, '(k)')

# Panel (l): Reconstructed image after freq hadamard product (recon)
ax = fig.add_subplot(gs[5, 1])
ax.imshow(norm01(recon_img), cmap='gray')
ax.set_title('Reconstruction after frequency elementwise product (Hadamard-like)')
ax.axis('off')
label_ax(ax, '(l)')

# Save and finalize
out_pdf = 'qcnn_complex_subplots.pdf'
plt.suptitle('Complex scientific subplots for QCNN Results Analysis — multipanel', fontsize=14, y=0.995)
plt.savefig(out_pdf, bbox_inches='tight', dpi=300)
print(f"\nSaved figure to {out_pdf}")
np.savez('qcnn_plot_values.npz')  # already saved above, call again (idempotent)
print("Saved plotting arrays to qcnn_plot_values.npz")
plt.close(fig)


Loading MNIST and FashionMNIST...

--- Numeric values used for plotting (printed arrays saved to qcnn_plot_values.npz) ---
P_vals: [0 1 2 3 4 5 6 7 8]
Delta_xi_min: [2.0000e-01 1.5576e-01 7.3576e-02 2.1080e-02 3.6630e-03 3.8600e-04
 2.5000e-05 1.0000e-06 0.0000e+00]
fidelity (1 - epsilon): [-1.30000e-05  3.93457e-01  8.64652e-01  9.88878e-01  9.99652e-01
  9.99984e-01  9.99987e-01  9.99987e-01  9.99987e-01]
qpe spectrum sample (first 10 y values): [0.020359 0.003458 0.034871 0.022677 0.036103 0.043319 0.048776 0.04279
 0.000586 0.017999]
aa_vals for m=0..6: [ 0.353553  0.883883  0.972272  0.574524 -0.110485 -0.740252 -0.999893]
mnist_mean, mnist_max: 0.11845939 1.0
fashion_mean, fashion_max: 0.297462 1.0
synthetic training final train_acc[-1]: 0.9786585030679823 val_acc[-1]: 0.9574680373833201
ensemble loss mean/ std (epoch 50): 0.10974448204143673 0.04677561921781158
confusion matrix shape: (10, 10)

Saved figure to qcnn_complex_subplots.pdf
Saved plotting arrays to qcnn_plot_values.n

In [9]:
import os
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import MaxNLocator
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy.fft import fft2, ifft2, fftshift
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import random
import warnings
warnings.filterwarnings("ignore")

# -------------------------------------
# Utilities
# -------------------------------------
def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)

def norm01(x):
    mn = np.min(x)
    mx = np.max(x)
    if mx - mn == 0:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def pad_image(img, P=3, mode='cyclic'):
    if mode == 'cyclic':
        return np.pad(img, ((P,P),(P,P)), mode='wrap')
    elif mode == 'symmetric':
        return np.pad(img, ((P,P),(P,P)), mode='symmetric')
    else:
        return np.pad(img, ((P,P),(P,P)), mode='constant')

def aa_amplitude(m, theta):
    return math.sin((2*m + 1) * theta)

def synthetic_qpe_spectrum(bins=256, peak_locations=None, heights=None, widths=None):
    x = np.linspace(0, 1, bins)
    y = np.zeros_like(x)
    if peak_locations is None:
        peak_locations = [0.12, 0.44, 0.78]
    if heights is None:
        heights = [1.0, 2.2, 1.6]
    if widths is None:
        widths = [0.02, 0.01, 0.015]
    for loc,h,w in zip(peak_locations, heights, widths):
        y += h * np.exp(-0.5 * ((x - loc)/w)**2)
    y += 0.02 * np.random.rand(len(x))
    return x, y

def freq_hadamard_demo(img, kern):
    Fimg = fftshift(fft2(img))
    Fkern = fftshift(fft2(kern, s=img.shape))
    Frecon = Fimg * Fkern
    recon = np.real(ifft2(fftshift(Frecon)))
    return Fimg, Fkern, Frecon, recon

def label_ax(ax, label, xpos=-0.05, ypos=1.08, weight='bold'):
    ax.text(xpos, ypos, label, transform=ax.transAxes,
            fontsize=11, fontweight=weight, va='top')

# -------------------------------------
# Reproducibility
# -------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

# -------------------------------------
# Load datasets with torchvision
# -------------------------------------
print("Downloading/loading MNIST and FashionMNIST via torchvision (if not present)...")
transform = transforms.Compose([transforms.ToTensor()])  # yields [0,1] tensors

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
fashion_train = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
fashion_test  = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

def dataset_to_numpy(ds):
    xs = []
    ys = []
    for i in range(len(ds)):
        img, lab = ds[i]
        # img shape [1,H,W] -> numpy HxW
        xs.append(img.squeeze(0).numpy())
        ys.append(int(lab))
    return np.stack(xs, axis=0), np.array(ys, dtype=np.int64)

x1_train, y1_train = dataset_to_numpy(mnist_train)
x1_test,  y1_test  = dataset_to_numpy(mnist_test)
x2_train, y2_train = dataset_to_numpy(fashion_train)
x2_test,  y2_test  = dataset_to_numpy(fashion_test)

# small display subsets
n_samples_display = 5
mnist_samples = x1_train[:n_samples_display]
fashion_samples = x2_train[:n_samples_display]

# -------------------------------------
# Manuscript-related derived metrics and synthetic models
# -------------------------------------
# Spectral resolution bound vs padding P
zeta = 0.25
kernel_norm = 2.5
hbar = 1.0
P_vals = np.arange(0, 11)  # padding 0..10
Delta_xi_min = (hbar / 2.0) * (1.0 / kernel_norm) * np.exp(-zeta * (P_vals**2))

# Fidelity model vs padding
kappa = 0.5
zeta_fid = 1e-2
Dprod = 28**2
alpha = 1.0
epsilon_P = np.exp(-kappa * P_vals**2) + zeta_fid / (Dprod**alpha)
fidelity = 1.0 - epsilon_P

# QPE synthetic spectrum
bins = 512
qpe_x, qpe_y = synthetic_qpe_spectrum(bins=bins, peak_locations=[0.15, 0.5, 0.82],
                                     heights=[3.2, 72.37, 18.0], widths=[0.012, 0.004, 0.01])

# AA amplitude
theta = math.asin(1.0 / math.sqrt(8.0))
m_iters = np.arange(0, 11)
aa_vals = np.array([aa_amplitude(m, theta) for m in m_iters])

# Intensity measures
def compute_mean_max(images):
    ms = [np.mean(img) for img in images]
    Mx = [np.max(img) for img in images]
    return np.mean(ms), np.mean(Mx)

mnist_mean, mnist_max = compute_mean_max(mnist_samples)
fashion_mean, fashion_max = compute_mean_max(fashion_samples)
qpe_gain_mnist_mean = 3.71
qpe_gain_mnist_max = 72.37
qpe_gain_fashion_mean = 3.42
qpe_gain_fashion_max = 131.2

# t-SNE for 800 points (400 each) to keep runtime reasonable
n_embed_per = 400
X_tsne_raw = np.vstack([x1_train[:n_embed_per].reshape(n_embed_per, -1), x2_train[:n_embed_per].reshape(n_embed_per, -1)])
labels_tsne = np.hstack([y1_train[:n_embed_per], y2_train[:n_embed_per] + 10])
tsne = TSNE(n_components=2, perplexity=30, init='pca', random_state=SEED, learning_rate=200)
X_embedded = tsne.fit_transform(X_tsne_raw)

# synthetic training-like dynamics
epochs = np.arange(1, 101)
train_acc = 0.48 + 0.5 * (1 - np.exp(-0.08 * (epochs - 1)))
val_acc = train_acc - 0.02 - 0.05 * np.exp(-0.06 * (epochs - 1))
quantum_entropy = 5.0 - 0.02 * epochs + 0.15 * np.sin(0.18 * epochs)
ensemble_loss_mean = 1.6 * np.exp(-0.04 * epochs) + 0.02
ensemble_loss_std  = 0.18 * np.exp(-0.03 * epochs) + 0.02

# Confusion matrix on small subset (simulate classifier)
np.random.seed(SEED)
y_true = y2_test[:300]
y_pred = y_true.copy()
flip_idx = np.random.choice(len(y_true), size=int(0.14 * len(y_true)), replace=False)
for idx in flip_idx:
    y_pred[idx] = (y_pred[idx] + np.random.randint(1,10)) % 10
cm = confusion_matrix(y_true, y_pred)

# Frequency-domain hadamard demo
sample_img = x1_train[7]
kernel = np.zeros((9,9))
for i in range(9):
    for j in range(9):
        kernel[i,j] = math.exp(-((i-4)**2 + (j-4)**2) / (2*2.0**2))
kernel = kernel / np.sum(kernel)
Fimg, Fkern, Frecon, recon_img = freq_hadamard_demo(sample_img, kernel)

# Per-class mean images for MNIST and FashionMNIST (first 10 classes each)
mnist_class_means = np.array([np.mean(x1_train[y1_train == cls], axis=0) for cls in range(10)])
fashion_class_means = np.array([np.mean(x2_train[y2_train == cls], axis=0) for cls in range(10)])

# Inter-class cosine similarity matrix (on flattened class means combined)
combined_means = np.vstack([mnist_class_means.reshape(10, -1), fashion_class_means.reshape(10, -1)])
norms = np.linalg.norm(combined_means, axis=1, keepdims=True)
combined_normed = combined_means / (norms + 1e-12)
cosine_sim = combined_normed @ combined_normed.T  # 20x20

# Kernel spectral profiles: compare three kernels (gaussian small, gaussian wide, laplacian-like)
def make_gaussian_kernel(size, sigma):
    k = np.fromfunction(lambda i, j: np.exp(-(((i-(size-1)/2)**2 + (j-(size-1)/2)**2) / (2*sigma**2))), (size, size))
    return k / np.sum(k)

kern1 = make_gaussian_kernel(7, sigma=0.8)
kern2 = make_gaussian_kernel(11, sigma=2.2)
# laplacian-like
kern3 = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]], dtype=float)
kern3 = kern3 / np.sum(np.abs(kern3))
kernels = [kern1, kern2, kern3]

# Correlation between metrics
metric_matrix = np.vstack([fidelity[:len(P_vals)], Delta_xi_min, np.interp(P_vals, [P_vals.min(), P_vals.max()], [aa_vals.min(), aa_vals.max()]), np.interp(P_vals, [P_vals.min(), P_vals.max()], [quantum_entropy.min(), quantum_entropy.max()])])
metric_corr = np.corrcoef(metric_matrix)

# PCA explained variance (flatten small sample)
X_pca_sample = X_tsne_raw[:1000]  # 1000 samples
pca = PCA(n_components=50, random_state=SEED)
pca.fit(X_pca_sample)
explained_var = pca.explained_variance_ratio_

# Class-wise sample variance: compute pixel variance for 10 fashion classes on small subset
class_vars = []
for cls in range(10):
    samples = x2_train[y2_train == cls][:200]
    pixel_vars = np.var(samples.reshape(samples.shape[0], -1), axis=1)
    class_vars.append(pixel_vars)
# For boxplot need list of arrays per class

# ROC-like synthetic curves: pick 3 classes from Fashion, do one-vs-rest using a synthetic score
classes_for_roc = [0, 1, 2]
y_true_subset = y2_test[:800]
# Create synthetic "scores" by adding small noise to a ground-truth signal
scores = np.zeros((len(y_true_subset), len(classes_for_roc)))
for i, cls in enumerate(classes_for_roc):
    base = (y_true_subset == cls).astype(float)
    scores[:, i] = base * (0.7 + 0.25 * np.random.rand(len(base))) + (1 - base) * (0.3 * np.random.rand(len(base)))

# -------------------------------------
# Print numeric arrays summary (for manuscript appendices)
# -------------------------------------
print("\n--- Numeric values summary (saved to qcnn_plot_values_expanded.npz) ---")
print("P_vals:", P_vals)
print("Delta_xi_min (rounded):", np.round(Delta_xi_min, 6))
print("fidelity (rounded):", np.round(fidelity, 6))
print("qpe_y sample (first 12):", np.round(qpe_y[:12], 6))
print("aa_vals (m=0..10):", np.round(aa_vals, 6))
print("mnist_mean, fashion_mean:", np.round(mnist_mean,6), np.round(fashion_mean,6))
print("combined cosine_sim shape:", cosine_sim.shape)
print("explained_var first 10:", np.round(explained_var[:10], 5))
print("confusion matrix shape:", cm.shape)

np.savez('qcnn_plot_values_expanded.npz',
         P_vals=P_vals,
         Delta_xi_min=Delta_xi_min,
         fidelity=fidelity,
         qpe_x=qpe_x, qpe_y=qpe_y,
         aa_m_iters=m_iters, aa_vals=aa_vals,
         mnist_mean=mnist_mean, mnist_max=mnist_max,
         fashion_mean=fashion_mean, fashion_max=fashion_max,
         X_embedded=X_embedded, labels_tsne=labels_tsne,
         epochs=epochs, train_acc=train_acc, val_acc=val_acc, quantum_entropy=quantum_entropy,
         ensemble_loss_mean=ensemble_loss_mean, ensemble_loss_std=ensemble_loss_std,
         confusion_matrix=cm,
         Fimg=np.abs(Fimg), Fkern=np.abs(Fkern), Frecon=np.abs(Frecon), recon_img=recon_img,
         mnist_class_means=mnist_class_means, fashion_class_means=fashion_class_means,
         cosine_sim=cosine_sim, kernels=np.array(kernels, dtype=object),
         metric_corr=metric_corr, explained_var=explained_var,
         class_vars=np.array(class_vars, dtype=object), 
         roc_scores=scores, y_true_subset=y_true_subset)


print("Saved numeric arrays to qcnn_plot_values_expanded.npz")

# -------------------------------------
# Create multipanel figure (20 panels)
# -------------------------------------
matplotlib.rcParams.update({'font.size': 9, 'figure.dpi': 150})
fig = plt.figure(figsize=(11.7, 16.5))  # A4 portrait
gs = gridspec.GridSpec(10, 2, height_ratios=[1]*10, hspace=0.5, wspace=0.35)

# Panel (a) Representative samples grid (top left)
ax = fig.add_subplot(gs[0, 0])
sub_gs = gs[0, 0].subgridspec(2, 5, wspace=0.05, hspace=0.05)
for i, img in enumerate(images_to_show):
    r = i // cols
    c = i % cols
    ax_img = fig.add_subplot(sub_gs[r, c])
    ax_img.imshow(img, cmap='gray', interpolation='nearest')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    if i < n_samples_display:
        ax_img.set_title(f"MN {i}", fontsize=7)
    else:
        ax_img.set_title(f"F {i - n_samples_display}", fontsize=7)
label_ax(ax, '(a) Representative samples')


# Panel (b) Intensity transforms
ax = fig.add_subplot(gs[0, 1])
bars = ['Orig MN mean', 'QPE MN mean (est)', 'Orig FM mean', 'QPE FM mean (est)']
vals = [mnist_mean, mnist_mean * qpe_gain_mnist_mean, fashion_mean, fashion_mean * qpe_gain_fashion_mean]
ax.bar(range(len(vals)), vals)
ax.set_xticks(range(len(vals)))
ax.set_xticklabels(bars, rotation=18, ha='right')
ax.set_ylabel('Mean intensity')
ax.set_title('Intensity transforms under QPE')
label_ax(ax, '(b)')

# Panel (c) QPE spectrum
ax = fig.add_subplot(gs[1, 0])
ax.plot(qpe_x, qpe_y, lw=1.2)
ax.set_xlabel('Normalized frequency')
ax.set_ylabel('QPE amplitude (arb.)')
ax.set_title('Synthetic QPE spectrum')
ax.grid(True, alpha=0.25)
label_ax(ax, '(c)')

# Panel (d) AA amplitude vs iterations
ax = fig.add_subplot(gs[1, 1])
ax.plot(m_iters, aa_vals, marker='o')
ax.set_xlabel('Iterations m')
ax.set_ylabel('Amplitude')
ax.set_title('Amplitude Amplification (AA)')
ax.grid(True)
label_ax(ax, '(d)')

# Panel (e) Fidelity vs padding
ax = fig.add_subplot(gs[2, 0])
ax.plot(P_vals, fidelity, marker='s', label='Fidelity')
ax.plot(P_vals, Delta_xi_min / np.max(Delta_xi_min), '--', label='Normed Δξ_min')
ax.set_xlabel('Padding P')
ax.set_ylabel('Metric')
ax.set_title('Fidelity & spectral resolution vs padding')
ax.legend(frameon=False)
label_ax(ax, '(e)')

# Panel (f) t-SNE
ax = fig.add_subplot(gs[2, 1])
ax.scatter(X_embedded[:n_embed_per,0], X_embedded[:n_embed_per,1], s=8, c='tab:blue', alpha=0.7, label='MNIST')
ax.scatter(X_embedded[n_embed_per:,0], X_embedded[n_embed_per:,1], s=8, c='tab:orange', alpha=0.7, label='Fashion')
ax.legend(markerscale=2)
ax.set_title('t-SNE of combined flattened features')
label_ax(ax, '(f)')

# Panel (g) Training curves + entropy
ax = fig.add_subplot(gs[3, 0])
ax.plot(epochs, train_acc, label='Train acc')
ax.plot(epochs, val_acc, label='Val acc')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax_twin = ax.twinx()
ax_twin.plot(epochs, quantum_entropy, label='Quantum entropy', lw=1.0, alpha=0.6)
ax_twin.set_ylabel('Quantum entropy')
ax.set_title('Training dynamics & quantum entropy')
ax.legend(loc='upper left')
ax_twin.legend(loc='upper right')
label_ax(ax, '(g)')

# Panel (h) Loss with uncertainty
ax = fig.add_subplot(gs[3, 1])
ax.plot(epochs, ensemble_loss_mean, label='Loss mean')
ax.fill_between(epochs, ensemble_loss_mean - ensemble_loss_std, ensemble_loss_mean + ensemble_loss_std, alpha=0.2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Ensemble loss with uncertainty')
label_ax(ax, '(h)')

# Panel (i) Feature variance vs accuracy
ax = fig.add_subplot(gs[4, 0])
feature_variance = 0.35 + 0.65 * (1 - np.exp(-0.09 * (epochs - 1)))
ax.plot(epochs, feature_variance, label='Feature variance')
ax.plot(epochs, train_acc, label='Train acc', linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Metric')
ax.set_title('Feature variance — accuracy coevolution')
ax.legend()
label_ax(ax, '(i)')

# Panel (j) Confusion matrix
ax = fig.add_subplot(gs[4, 1])
im = ax.imshow(cm, interpolation='nearest', cmap='viridis')
ax.set_title('Simulated confusion matrix (Fashion subset)')
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(j)')

# Panel (k) FFT magnitude of sample image
ax = fig.add_subplot(gs[5, 0])
ax.imshow(np.log1p(np.abs(Fimg)), cmap='inferno')
ax.set_title('log |FFT(image)|')
ax.axis('off')
label_ax(ax, '(k)')

# Panel (l) Reconstruction after freq hadamard product
ax = fig.add_subplot(gs[5, 1])
ax.imshow(norm01(recon_img), cmap='gray')
ax.set_title('Reconstruction after frequency elementwise product')
ax.axis('off')
label_ax(ax, '(l)')

# Panel (m) MNIST class mean heatmaps (10 subpanels in mini-grid)
ax = fig.add_subplot(gs[6, 0])
ax.axis('off')
for i in range(10):
    r = i // 5
    c = i % 5
    ax_img = fig.add_axes([0.06 + c*0.18, 0.54 - r*0.08, 0.14, 0.07])
    ax_img.imshow(mnist_class_means[i], cmap='gray')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    ax_img.set_title(f"MN {i}", fontsize=7)
label_ax(ax, '(m) MNIST per-class mean images')

# Panel (n) FashionMNIST class mean heatmaps

ax = fig.add_subplot(gs[6, 0])
sub_gs = gs[6, 0].subgridspec(2, 5, wspace=0.05, hspace=0.05)
for i in range(10):
    r, c = divmod(i, 5)
    ax_img = fig.add_subplot(sub_gs[r, c])
    ax_img.imshow(mnist_class_means[i], cmap='gray')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    ax_img.set_title(f"MN {i}", fontsize=7)
label_ax(ax, '(m) MNIST per-class mean images')

# Panel (o) Inter-class cosine similarity matrix (20x20)
ax = fig.add_subplot(gs[7, 0])
im = ax.imshow(cosine_sim, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Inter-class cosine similarity (MN+Fash)')
ax.set_xticks(range(20)); ax.set_yticks(range(20))
ax.set_xticklabels([f"MN{c}" for c in range(10)] + [f"F{c}" for c in range(10)], rotation=90, fontsize=6)
ax.set_yticklabels([f"MN{c}" for c in range(10)] + [f"F{c}" for c in range(10)], fontsize=6)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(o)')

# Panel (p) Kernel spectral profile comparison
ax = fig.add_subplot(gs[7, 1])
# compute radial-averaged FFT magnitude for each kernel sized to 64x64
def pad_to(size, arr):
    out = np.zeros((size,size))
    s0,s1 = arr.shape
    start0 = (size - s0)//2
    start1 = (size - s1)//2
    out[start0:start0+s0, start1:start1+s1] = arr
    return out

size = 64
kx = np.linspace(0, 1, size)
for idx,k in enumerate(kernels):
    padk = pad_to(size, k)
    Kf = np.abs(fftshift(fft2(padk)))
    # radial average
    cx, cy = size//2, size//2
    rs = np.sqrt((np.arange(size) - cx)[:,None]**2 + (np.arange(size) - cy)[None,:]**2)
    rmax = int(np.max(rs))
    bins = np.linspace(0, rmax, 200)
    radial = []
    for i in range(len(bins)-1):
        mask = (rs >= bins[i]) & (rs < bins[i+1])
        v = np.mean(Kf[mask]) if np.any(mask) else 0
        radial.append(v)
    radial = np.array(radial)
    radial = radial / (radial.max() + 1e-12)
    ax.plot(radial, label=f'Kernel {idx+1}')
ax.set_title('Kernel spectral (radial) profiles (normed)')
ax.set_xlabel('Radial bin')
ax.legend(frameon=False, fontsize=7)
label_ax(ax, '(p)')

# Panel (q) Correlation heatmap between derived metrics
ax = fig.add_subplot(gs[8, 0])
im = ax.imshow(metric_corr, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Correlation matrix between metrics (fidelity, Δξ, AA proxy, entropy proxy)')
ax.set_xticks(range(metric_corr.shape[0])); ax.set_yticks(range(metric_corr.shape[0]))
ax.set_xticklabels(['Fid','Δξ','AAproxy','Entropy'], rotation=45, fontsize=8)
ax.set_yticklabels(['Fid','Δξ','AAproxy','Entropy'], fontsize=8)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(q)')

# Panel (r) PCA explained variance
ax = fig.add_subplot(gs[8, 1])
ax.plot(np.arange(1, len(explained_var)+1), np.cumsum(explained_var), marker='o')
ax.set_xlabel('Number of PCs')
ax.set_ylabel('Cumulative explained variance')
ax.set_title('PCA cumulative explained variance (sample)')
ax.grid(True)
label_ax(ax, '(r)')

# Panel (s) Class-wise variance boxplots (Fashion)
ax = fig.add_subplot(gs[9, 0])
ax.boxplot(class_vars, labels=[str(i) for i in range(10)], showfliers=False)
ax.set_title('Per-class sample variance distribution (FashionMNIST)')
ax.set_xlabel('Class')
ax.set_ylabel('Pixel-wise sample variance')
label_ax(ax, '(s)')

# Panel (t) 1-vs-rest ROC-like synthetic curves
ax = fig.add_subplot(gs[9, 1])
y_bin = label_binarize(y_true_subset, classes=np.arange(10))
for i, cls in enumerate(classes_for_roc):
    fpr, tpr, _ = roc_curve((y_true_subset == cls).astype(int), scores[:, i])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'Class {cls} (AUC={roc_auc:.2f})')
ax.plot([0,1],[0,1],'k--', lw=0.5)
ax.set_title('1-vs-rest ROC-like curves (synthetic scores)')
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.legend(fontsize=7)
label_ax(ax, '(t)')

plt.suptitle('Expanded complex scientific multipanel figure for QCNN results analysis', fontsize=14, y=0.995)
out_pdf = 'qcnn_complex_subplots_expanded.pdf'
plt.savefig(out_pdf, bbox_inches='tight', dpi=300)
print(f"\nSaved expanded figure to {out_pdf}")
np.savez('qcnn_plot_values_expanded.npz')  # already saved; harmless to call again
plt.close(fig)

print("\nAll done. Files produced:")
print(" -", out_pdf)
print(" - qcnn_plot_values_expanded.npz")


Downloading/loading MNIST and FashionMNIST via torchvision (if not present)...

--- Numeric values summary (saved to qcnn_plot_values_expanded.npz) ---
P_vals: [ 0  1  2  3  4  5  6  7  8  9 10]
Delta_xi_min (rounded): [2.0000e-01 1.5576e-01 7.3576e-02 2.1080e-02 3.6630e-03 3.8600e-04
 2.5000e-05 1.0000e-06 0.0000e+00 0.0000e+00 0.0000e+00]
fidelity (rounded): [-1.30000e-05  3.93457e-01  8.64652e-01  9.88878e-01  9.99652e-01
  9.99984e-01  9.99987e-01  9.99987e-01  9.99987e-01  9.99987e-01
  9.99987e-01]
qpe_y sample (first 12): [0.007491 0.019014 0.01464  0.011973 0.00312  0.00312  0.001162 0.017324
 0.012022 0.014161 0.000412 0.019398]
aa_vals (m=0..10): [ 0.353553  0.883883  0.972272  0.574524 -0.110485 -0.740252 -0.999893
 -0.759587 -0.139488  0.550356  0.965021]
mnist_mean, fashion_mean: 0.118459 0.297462
combined cosine_sim shape: (20, 20)
explained_var first 10: [0.24653 0.10528 0.04534 0.04005 0.03593 0.03099 0.02696 0.02374 0.02143
 0.0209 ]
confusion matrix shape: (10, 10)
Sa

In [13]:
# qcnn_results_subplots_pytorch_expanded.py 
# Large multipanel figure (20 panels) for manuscript Results analysis.
# Uses PyTorch / torchvision for MNIST & FashionMNIST loading.
# Saves PDF and numeric arrays (.npz).
# Run with Python 3.8+ (works on 3.11 too).

import os
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import MaxNLocator
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy.fft import fft2, ifft2, fftshift
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import random
import warnings
warnings.filterwarnings("ignore")

from matplotlib import gridspec

# -------------------------------------
# Utilities
# -------------------------------------
def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)

def norm01(x):
    mn = np.min(x)
    mx = np.max(x)
    if mx - mn == 0:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def pad_image(img, P=3, mode='cyclic'):
    if mode == 'cyclic':
        return np.pad(img, ((P,P),(P,P)), mode='wrap')
    elif mode == 'symmetric':
        return np.pad(img, ((P,P),(P,P)), mode='symmetric')
    else:
        return np.pad(img, ((P,P),(P,P)), mode='constant')

def aa_amplitude(m, theta):
    return math.sin((2*m + 1) * theta)

def synthetic_qpe_spectrum(bins=256, peak_locations=None, heights=None, widths=None):
    x = np.linspace(0, 1, bins)
    y = np.zeros_like(x)
    if peak_locations is None:
        peak_locations = [0.12, 0.44, 0.78]
    if heights is None:
        heights = [1.0, 2.2, 1.6]
    if widths is None:
        widths = [0.02, 0.01, 0.015]
    for loc,h,w in zip(peak_locations, heights, widths):
        y += h * np.exp(-0.5 * ((x - loc)/w)**2)
    y += 0.02 * np.random.rand(len(x))
    return x, y

def freq_hadamard_demo(img, kern):
    Fimg = fftshift(fft2(img))
    Fkern = fftshift(fft2(kern, s=img.shape))
    Frecon = Fimg * Fkern
    recon = np.real(ifft2(fftshift(Frecon)))
    return Fimg, Fkern, Frecon, recon

def label_ax(ax, label, xpos=-0.05, ypos=1.08, weight='bold'):
    ax.text(xpos, ypos, label, transform=ax.transAxes,
            fontsize=11, fontweight=weight, va='top')

# -------------------------------------
# Reproducibility
# -------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

# -------------------------------------
# Load datasets with torchvision
# -------------------------------------
print("Downloading/loading MNIST and FashionMNIST via torchvision (if not present)...")
transform = transforms.Compose([transforms.ToTensor()])  # yields [0,1] tensors

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
fashion_train = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
fashion_test  = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

def dataset_to_numpy(ds):
    xs = []
    ys = []
    for i in range(len(ds)):
        img, lab = ds[i]
        # img shape [1,H,W] -> numpy HxW
        xs.append(img.squeeze(0).numpy())
        ys.append(int(lab))
    return np.stack(xs, axis=0), np.array(ys, dtype=np.int64)

x1_train, y1_train = dataset_to_numpy(mnist_train)
x1_test,  y1_test  = dataset_to_numpy(mnist_test)
x2_train, y2_train = dataset_to_numpy(fashion_train)
x2_test,  y2_test  = dataset_to_numpy(fashion_test)

# small display subsets
n_samples_display = 5
mnist_samples = x1_train[:n_samples_display]
fashion_samples = x2_train[:n_samples_display]

# -------------------------------------
# Manuscript-related derived metrics and synthetic models
# -------------------------------------
# Spectral resolution bound vs padding P
zeta = 0.25
kernel_norm = 2.5
hbar = 1.0
P_vals = np.arange(0, 11)  # padding 0..10
Delta_xi_min = (hbar / 2.0) * (1.0 / kernel_norm) * np.exp(-zeta * (P_vals**2))

# Fidelity model vs padding
kappa = 0.5
zeta_fid = 1e-2
Dprod = 28**2
alpha = 1.0
epsilon_P = np.exp(-kappa * P_vals**2) + zeta_fid / (Dprod**alpha)
fidelity = 1.0 - epsilon_P

# QPE synthetic spectrum
bins = 512
qpe_x, qpe_y = synthetic_qpe_spectrum(bins=bins, peak_locations=[0.15, 0.5, 0.82],
                                     heights=[3.2, 72.37, 18.0], widths=[0.012, 0.004, 0.01])

# AA amplitude
theta = math.asin(1.0 / math.sqrt(8.0))
m_iters = np.arange(0, 11)
aa_vals = np.array([aa_amplitude(m, theta) for m in m_iters])

# Intensity measures
def compute_mean_max(images):
    ms = [np.mean(img) for img in images]
    Mx = [np.max(img) for img in images]
    return np.mean(ms), np.mean(Mx)

mnist_mean, mnist_max = compute_mean_max(mnist_samples)
fashion_mean, fashion_max = compute_mean_max(fashion_samples)
qpe_gain_mnist_mean = 3.71
qpe_gain_mnist_max = 72.37
qpe_gain_fashion_mean = 3.42
qpe_gain_fashion_max = 131.2

# t-SNE for 800 points (400 each) to keep runtime reasonable
n_embed_per = 400
X_tsne_raw = np.vstack([x1_train[:n_embed_per].reshape(n_embed_per, -1), x2_train[:n_embed_per].reshape(n_embed_per, -1)])
labels_tsne = np.hstack([y1_train[:n_embed_per], y2_train[:n_embed_per] + 10])
tsne = TSNE(n_components=2, perplexity=30, init='pca', random_state=SEED, learning_rate=200)
X_embedded = tsne.fit_transform(X_tsne_raw)

# synthetic training-like dynamics
epochs = np.arange(1, 101)
train_acc = 0.48 + 0.5 * (1 - np.exp(-0.08 * (epochs - 1)))
val_acc = train_acc - 0.02 - 0.05 * np.exp(-0.06 * (epochs - 1))
quantum_entropy = 5.0 - 0.02 * epochs + 0.15 * np.sin(0.18 * epochs)
ensemble_loss_mean = 1.6 * np.exp(-0.04 * epochs) + 0.02
ensemble_loss_std  = 0.18 * np.exp(-0.03 * epochs) + 0.02

# Confusion matrix on small subset (simulate classifier)
np.random.seed(SEED)
y_true = y2_test[:300]
y_pred = y_true.copy()
flip_idx = np.random.choice(len(y_true), size=int(0.14 * len(y_true)), replace=False)
for idx in flip_idx:
    y_pred[idx] = (y_pred[idx] + np.random.randint(1,10)) % 10
cm = confusion_matrix(y_true, y_pred)

# Frequency-domain hadamard demo
sample_img = x1_train[7]
kernel = np.zeros((9,9))
for i in range(9):
    for j in range(9):
        kernel[i,j] = math.exp(-((i-4)**2 + (j-4)**2) / (2*2.0**2))
kernel = kernel / np.sum(kernel)
Fimg, Fkern, Frecon, recon_img = freq_hadamard_demo(sample_img, kernel)

# Per-class mean images for MNIST and FashionMNIST (first 10 classes each)
mnist_class_means = np.array([np.mean(x1_train[y1_train == cls], axis=0) for cls in range(10)])
fashion_class_means = np.array([np.mean(x2_train[y2_train == cls], axis=0) for cls in range(10)])

# Inter-class cosine similarity matrix (on flattened class means combined)
combined_means = np.vstack([mnist_class_means.reshape(10, -1), fashion_class_means.reshape(10, -1)])
norms = np.linalg.norm(combined_means, axis=1, keepdims=True)
combined_normed = combined_means / (norms + 1e-12)
cosine_sim = combined_normed @ combined_normed.T  # 20x20

# Kernel spectral profiles: compare three kernels (gaussian small, gaussian wide, laplacian-like)
def make_gaussian_kernel(size, sigma):
    k = np.fromfunction(lambda i, j: np.exp(-(((i-(size-1)/2)**2 + (j-(size-1)/2)**2) / (2*sigma**2))), (size, size))
    return k / np.sum(k)

kern1 = make_gaussian_kernel(7, sigma=0.8)
kern2 = make_gaussian_kernel(11, sigma=2.2)
# laplacian-like
kern3 = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]], dtype=float)
kern3 = kern3 / np.sum(np.abs(kern3))
kernels = [kern1, kern2, kern3]

# Correlation between metrics
metric_matrix = np.vstack([fidelity[:len(P_vals)], Delta_xi_min, np.interp(P_vals, [P_vals.min(), P_vals.max()], [aa_vals.min(), aa_vals.max()]), np.interp(P_vals, [P_vals.min(), P_vals.max()], [quantum_entropy.min(), quantum_entropy.max()])])
metric_corr = np.corrcoef(metric_matrix)

# PCA explained variance (flatten small sample)
X_pca_sample = X_tsne_raw[:1000]  # 1000 samples
pca = PCA(n_components=50, random_state=SEED)
pca.fit(X_pca_sample)
explained_var = pca.explained_variance_ratio_

# Class-wise sample variance: compute pixel variance for 10 fashion classes on small subset
class_vars = []
for cls in range(10):
    samples = x2_train[y2_train == cls][:200]
    pixel_vars = np.var(samples.reshape(samples.shape[0], -1), axis=1)
    class_vars.append(pixel_vars)
# For boxplot need list of arrays per class

# ROC-like synthetic curves: pick 3 classes from Fashion, do one-vs-rest using a synthetic score
classes_for_roc = [0, 1, 2]
y_true_subset = y2_test[:800]
# Create synthetic "scores" by adding small noise to a ground-truth signal
scores = np.zeros((len(y_true_subset), len(classes_for_roc)))
for i, cls in enumerate(classes_for_roc):
    base = (y_true_subset == cls).astype(float)
    scores[:, i] = base * (0.7 + 0.25 * np.random.rand(len(base))) + (1 - base) * (0.3 * np.random.rand(len(base)))

# -------------------------------------
# Print numeric arrays summary (for manuscript appendices)
# -------------------------------------
print("\n--- Numeric values summary (saved to qcnn_plot_values_expanded.npz) ---")
print("P_vals:", P_vals)
print("Delta_xi_min (rounded):", np.round(Delta_xi_min, 6))
print("fidelity (rounded):", np.round(fidelity, 6))
print("qpe_y sample (first 12):", np.round(qpe_y[:12], 6))
print("aa_vals (m=0..10):", np.round(aa_vals, 6))
print("mnist_mean, fashion_mean:", np.round(mnist_mean,6), np.round(fashion_mean,6))
print("combined cosine_sim shape:", cosine_sim.shape)
print("explained_var first 10:", np.round(explained_var[:10], 5))
print("confusion matrix shape:", cm.shape)

np.savez('qcnn_plot_values_expanded.npz',
         P_vals=P_vals,
         Delta_xi_min=Delta_xi_min,
         fidelity=fidelity,
         qpe_x=qpe_x, qpe_y=qpe_y,
         aa_m_iters=m_iters, aa_vals=aa_vals,
         mnist_mean=mnist_mean, mnist_max=mnist_max,
         fashion_mean=fashion_mean, fashion_max=fashion_max,
         X_embedded=X_embedded, labels_tsne=labels_tsne,
         epochs=epochs, train_acc=train_acc, val_acc=val_acc, quantum_entropy=quantum_entropy,
         ensemble_loss_mean=ensemble_loss_mean, ensemble_loss_std=ensemble_loss_std,
         confusion_matrix=cm,
         Fimg=np.abs(Fimg), Fkern=np.abs(Fkern), Frecon=np.abs(Frecon), recon_img=recon_img,
         mnist_class_means=mnist_class_means, fashion_class_means=fashion_class_means,
         cosine_sim=cosine_sim, kernels=np.array(kernels, dtype=object),
         metric_corr=metric_corr, explained_var=explained_var,
         class_vars=np.array(class_vars, dtype=object), 
         roc_scores=scores, y_true_subset=y_true_subset)


print("Saved numeric arrays to qcnn_plot_values_expanded.npz")

# -------------------------------------
# Create multipanel figure (20 panels)
# -------------------------------------
matplotlib.rcParams.update({'font.size': 9, 'figure.dpi': 150})
fig = plt.figure(figsize=(11.7, 16.5))
gs = gridspec.GridSpec(10, 2, figure=fig, hspace=1.0, wspace=0.4)

# Panel (a) Representative samples grid (top left)
ax = fig.add_subplot(gs[0, 0])
rows = 2
cols = 5
images_to_show = list(mnist_samples) + list(fashion_samples)
for i, img in enumerate(images_to_show):
    r = i // cols
    c = i % cols
    ax_img = fig.add_axes([0.06 + c*0.18, 0.94 - r*0.095, 0.14, 0.08])
    ax_img.imshow(img, cmap='gray', interpolation='nearest')
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    if i < n_samples_display:
        ax_img.set_title("MN {}".format(i), fontsize=7)
    else:
        ax_img.set_title("F {}".format(i - n_samples_display), fontsize=7)
ax.axis('off')
label_ax(ax, '(a) Representative samples (MNIST & Fashion)')

# Panel (b) Intensity transforms
ax = fig.add_subplot(gs[0, 1])
bars = ['Orig MN mean', 'QPE MN mean (est)', 'Orig FM mean', 'QPE FM mean (est)']
vals = [mnist_mean, mnist_mean * qpe_gain_mnist_mean, fashion_mean, fashion_mean * qpe_gain_fashion_mean]
ax.bar(range(len(vals)), vals)
ax.set_xticks(range(len(vals)))
ax.set_xticklabels(bars, rotation=18, ha='right')
ax.set_ylabel('Mean intensity')
ax.set_title('Intensity transforms under QPE')
label_ax(ax, '(b)')

# Panel (c) QPE spectrum
ax = fig.add_subplot(gs[1, 0])
ax.plot(qpe_x, qpe_y, lw=1.2)
ax.set_xlabel('Normalized frequency')
ax.set_ylabel('QPE amplitude (arb.)')
ax.set_title('Synthetic QPE spectrum')
ax.grid(True, alpha=0.25)
label_ax(ax, '(c)')

# Panel (d) AA amplitude vs iterations
ax = fig.add_subplot(gs[1, 1])
ax.plot(m_iters, aa_vals, marker='o')
ax.set_xlabel('Iterations m')
ax.set_ylabel('Amplitude')
ax.set_title('Amplitude Amplification (AA)')
ax.grid(True)
label_ax(ax, '(d)')

# Panel (e) Fidelity vs padding
ax = fig.add_subplot(gs[2, 0])
ax.plot(P_vals, fidelity, marker='s', label='Fidelity')
ax.plot(P_vals, Delta_xi_min / np.max(Delta_xi_min), '--', label='Normed Δξ_min')
ax.set_xlabel('Padding P')
ax.set_ylabel('Metric')
ax.set_title('Fidelity & spectral resolution vs padding')
ax.legend(frameon=False)
label_ax(ax, '(e)')

# Panel (f) t-SNE
ax = fig.add_subplot(gs[2, 1])
ax.scatter(X_embedded[:n_embed_per,0], X_embedded[:n_embed_per,1], s=8, c='tab:blue', alpha=0.7, label='MNIST')
ax.scatter(X_embedded[n_embed_per:,0], X_embedded[n_embed_per:,1], s=8, c='tab:orange', alpha=0.7, label='Fashion')
ax.legend(markerscale=2)
ax.set_title('t-SNE of combined flattened features')
label_ax(ax, '(f)')

# Panel (g) Training curves + entropy
ax = fig.add_subplot(gs[3, 0])
ax.plot(epochs, train_acc, label='Train acc')
ax.plot(epochs, val_acc, label='Val acc')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax_twin = ax.twinx()
ax_twin.plot(epochs, quantum_entropy, label='Quantum entropy', lw=1.0, alpha=0.6)
ax_twin.set_ylabel('Quantum entropy')
ax.set_title('Training dynamics & quantum entropy')
ax.legend(loc='upper left')
ax_twin.legend(loc='upper right')
label_ax(ax, '(g)')

# Panel (h) Loss with uncertainty
ax = fig.add_subplot(gs[3, 1])
ax.plot(epochs, ensemble_loss_mean, label='Loss mean')
ax.fill_between(epochs, ensemble_loss_mean - ensemble_loss_std, ensemble_loss_mean + ensemble_loss_std, alpha=0.2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Ensemble loss with uncertainty')
label_ax(ax, '(h)')

# Panel (i) Feature variance vs accuracy
ax = fig.add_subplot(gs[4, 0])
feature_variance = 0.35 + 0.65 * (1 - np.exp(-0.09 * (epochs - 1)))
ax.plot(epochs, feature_variance, label='Feature variance')
ax.plot(epochs, train_acc, label='Train acc', linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Metric')
ax.set_title('Feature variance — accuracy coevolution')
ax.legend()
label_ax(ax, '(i)')

# Panel (j) Confusion matrix
ax = fig.add_subplot(gs[4, 1])
im = ax.imshow(cm, interpolation='nearest', cmap='viridis')
ax.set_title('Simulated confusion matrix (Fashion subset)')
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(j)')

# Panel (k) FFT magnitude of sample image
ax = fig.add_subplot(gs[5, 0])
ax.imshow(np.log1p(np.abs(Fimg)), cmap='inferno')
ax.set_title('log |FFT(image)|')
ax.axis('off')
label_ax(ax, '(k)')

# Panel (l) Reconstruction after freq hadamard product
ax = fig.add_subplot(gs[5, 1])
ax.imshow(norm01(recon_img), cmap='gray')
ax.set_title('Reconstruction after frequency elementwise product')
ax.axis('off')
label_ax(ax, '(l)')

# Panel (m) MNIST class mean heatmaps (10 subpanels in mini-grid)
ax = fig.add_subplot(gs[6, 0])
ax.axis('off')
for i in range(10):
    r = i // 5
    c = i % 5
    ax_img = fig.add_axes([0.06 + c*0.18, 0.54 - r*0.08, 0.14, 0.07])
    ax_img.imshow(mnist_class_means[i], cmap='gray')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    ax_img.set_title(f"MN {i}", fontsize=7)
label_ax(ax, '(m) MNIST per-class mean images')

# Panel (n) FashionMNIST class mean heatmaps
ax = fig.add_subplot(gs[6, 1])
ax.axis('off')
for i in range(10):
    r = i // 5
    c = i % 5
    ax_img = fig.add_axes([0.56 + c*0.18, 0.54 - r*0.08, 0.14, 0.07])
    ax_img.imshow(fashion_class_means[i], cmap='gray')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    ax_img.set_title(f"F {i}", fontsize=7)
label_ax(ax, '(n) FashionMNIST per-class mean images')

# Panel (o) Inter-class cosine similarity matrix (20x20)
ax = fig.add_subplot(gs[7, 0])
im = ax.imshow(cosine_sim, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Inter-class cosine similarity (MN+Fash)')
ax.set_xticks(range(20)); ax.set_yticks(range(20))
ax.set_xticklabels([f"MN{c}" for c in range(10)] + [f"F{c}" for c in range(10)], rotation=90, fontsize=6)
ax.set_yticklabels([f"MN{c}" for c in range(10)] + [f"F{c}" for c in range(10)], fontsize=6)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(o)')

# Panel (p) Kernel spectral profile comparison
ax = fig.add_subplot(gs[7, 1])
# compute radial-averaged FFT magnitude for each kernel sized to 64x64
def pad_to(size, arr):
    out = np.zeros((size,size))
    s0,s1 = arr.shape
    start0 = (size - s0)//2
    start1 = (size - s1)//2
    out[start0:start0+s0, start1:start1+s1] = arr
    return out

size = 64
kx = np.linspace(0, 1, size)
for idx,k in enumerate(kernels):
    padk = pad_to(size, k)
    Kf = np.abs(fftshift(fft2(padk)))
    # radial average
    cx, cy = size//2, size//2
    rs = np.sqrt((np.arange(size) - cx)[:,None]**2 + (np.arange(size) - cy)[None,:]**2)
    rmax = int(np.max(rs))
    bins = np.linspace(0, rmax, 200)
    radial = []
    for i in range(len(bins)-1):
        mask = (rs >= bins[i]) & (rs < bins[i+1])
        v = np.mean(Kf[mask]) if np.any(mask) else 0
        radial.append(v)
    radial = np.array(radial)
    radial = radial / (radial.max() + 1e-12)
    ax.plot(radial, label=f'Kernel {idx+1}')
ax.set_title('Kernel spectral (radial) profiles (normed)')
ax.set_xlabel('Radial bin')
ax.legend(frameon=False, fontsize=7)
label_ax(ax, '(p)')

# Panel (q) Correlation heatmap between derived metrics
ax = fig.add_subplot(gs[8, 0])
im = ax.imshow(metric_corr, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Correlation matrix between metrics (fidelity, Δξ, AA proxy, entropy proxy)')
ax.set_xticks(range(metric_corr.shape[0])); ax.set_yticks(range(metric_corr.shape[0]))
ax.set_xticklabels(['Fid','Δξ','AAproxy','Entropy'], rotation=45, fontsize=8)
ax.set_yticklabels(['Fid','Δξ','AAproxy','Entropy'], fontsize=8)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(q)')

# Panel (r) PCA explained variance
ax = fig.add_subplot(gs[8, 1])
ax.plot(np.arange(1, len(explained_var)+1), np.cumsum(explained_var), marker='o')
ax.set_xlabel('Number of PCs')
ax.set_ylabel('Cumulative explained variance')
ax.set_title('PCA cumulative explained variance (sample)')
ax.grid(True)
label_ax(ax, '(r)')

# Panel (s) Class-wise variance boxplots (Fashion)
ax = fig.add_subplot(gs[9, 0])
ax.boxplot(class_vars, labels=[str(i) for i in range(10)], showfliers=False)
ax.set_title('Per-class sample variance distribution (FashionMNIST)')
ax.set_xlabel('Class')
ax.set_ylabel('Pixel-wise sample variance')
label_ax(ax, '(s)')

# Panel (t) 1-vs-rest ROC-like synthetic curves
ax = fig.add_subplot(gs[9, 1])
y_bin = label_binarize(y_true_subset, classes=np.arange(10))
for i, cls in enumerate(classes_for_roc):
    fpr, tpr, _ = roc_curve((y_true_subset == cls).astype(int), scores[:, i])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'Class {cls} (AUC={roc_auc:.2f})')
ax.plot([0,1],[0,1],'k--', lw=0.5)
ax.set_title('1-vs-rest ROC-like curves (synthetic scores)')
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.legend(fontsize=7)
label_ax(ax, '(t)')

plt.suptitle('Expanded complex scientific multipanel figure for QCNN results analysis', fontsize=14, y=0.995)
out_pdf = 'qcnn_complex_subplots_expanded111.pdf'
plt.savefig(out_pdf, bbox_inches='tight', dpi=300)
print(f"\nSaved expanded figure to {out_pdf}")
np.savez('qcnn_plot_values_expanded.npz')  # already saved; harmless to call again
plt.close(fig)

print("\nAll done. Files produced:")
print(" -", out_pdf)
print(" - qcnn_plot_values_expanded.npz")

Downloading/loading MNIST and FashionMNIST via torchvision (if not present)...

--- Numeric values summary (saved to qcnn_plot_values_expanded.npz) ---
P_vals: [ 0  1  2  3  4  5  6  7  8  9 10]
Delta_xi_min (rounded): [2.0000e-01 1.5576e-01 7.3576e-02 2.1080e-02 3.6630e-03 3.8600e-04
 2.5000e-05 1.0000e-06 0.0000e+00 0.0000e+00 0.0000e+00]
fidelity (rounded): [-1.30000e-05  3.93457e-01  8.64652e-01  9.88878e-01  9.99652e-01
  9.99984e-01  9.99987e-01  9.99987e-01  9.99987e-01  9.99987e-01
  9.99987e-01]
qpe_y sample (first 12): [0.007491 0.019014 0.01464  0.011973 0.00312  0.00312  0.001162 0.017324
 0.012022 0.014161 0.000412 0.019398]
aa_vals (m=0..10): [ 0.353553  0.883883  0.972272  0.574524 -0.110485 -0.740252 -0.999893
 -0.759587 -0.139488  0.550356  0.965021]
mnist_mean, fashion_mean: 0.118459 0.297462
combined cosine_sim shape: (20, 20)
explained_var first 10: [0.24653 0.10528 0.04534 0.04005 0.03593 0.03099 0.02696 0.02374 0.02143
 0.0209 ]
confusion matrix shape: (10, 10)
Sa

In [11]:
!pip install torch torchvision matplotlib pillow numpy



import os
import sys
import math
import numpy as np
from itertools import cycle
from collections import OrderedDict
from datetime import datetime
from PIL import Image
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
try:
    import torch
    from torchvision import datasets, transforms
except Exception as e:
    torch = None
    datasets = None
    transforms = None


def prepare_datasets(download=True):
    out = {}
    if torch is None:
        return out
    tf = transforms.Compose([transforms.ToTensor()])
    mn = datasets.MNIST(root='./data', train=True, download=download, transform=tf)
    fm = datasets.FashionMNIST(root='./data', train=True, download=download, transform=tf)
    cf = datasets.CIFAR10(root='./data', train=True, download=download, transform=transforms.Compose([transforms.ToTensor(), transforms.Grayscale(num_output_channels=1)]))
    out['MNIST'] = mn
    out['FASHION'] = fm
    out['CIFAR'] = cf
    return out

def image_stats(imgs, n=256):
    arr = np.stack([np.array(imgs[i][0][0], dtype=float) for i in range(min(len(imgs), n))])
    mean = arr.mean(axis=0)
    std = arr.std(axis=0)
    median = np.median(arr, axis=0)
    pspec = np.abs(np.fft.fftshift(np.fft.fft2(mean)))[:mean.shape[0], :mean.shape[1]]
    return {'mean': mean, 'std': std, 'median': median, 'pspec': pspec}

def generate_feature_maps(dset_name, dset, n=256):
    stats = image_stats(dset, n=n)
    mean = stats['mean']
    std = stats['std']
    median = stats['median']
    pspec = stats['pspec']
    gradx = np.gradient(mean, axis=1)
    grady = np.gradient(mean, axis=0)
    lap = np.gradient(np.gradient(mean, axis=0), axis=0) + np.gradient(np.gradient(mean, axis=1), axis=1)
    corners = np.abs(np.fft.ifft2(np.fft.ifftshift(pspec)))[:mean.shape[0], :mean.shape[1]]
    feats = {
        'mean': mean,
        'std': std,
        'median': median,
        'pspec': pspec,
        'gradx': gradx,
        'grady': grady,
        'lap': lap,
        'corners': np.abs(corners)
    }
    return feats

def label_generator():
    for c in range(97, 123):
        yield chr(c)
    for i in range(1,1000):
        yield str(i)
label_iter = label_generator()

def save_values_log(values, path):
    with open(path, 'w') as f:
        for k,v in values.items():
            f.write(f"{k}: shape={np.array(v).shape}, min={np.min(v):.6g}, max={np.max(v):.6g}, mean={np.mean(v):.6g}, std={np.std(v):.6g}\n")

def run_all(save_pdf=True, download=True, verbose=True):
    data = prepare_datasets(download=download)
    if not data:
        print('Torch or torchvision not available. Script prepared to run on a machine with PyTorch and torchvision installed.')
        return
    features = {}
    for name,dset in data.items():
        features[name] = generate_feature_maps(name, dset, n=256)
    pdf_path = os.path.join(os.getcwd(), 'complex_subplots_output.pdf')
    values_log_path = os.path.join(os.getcwd(), 'plot_values_used.txt')
    values_to_log = {}
    pp = PdfPages(pdf_path) if save_pdf else None
    figcount = 0


def plot_block_1(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 1')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_2(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 2')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_3(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 3')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_4(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 4')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_5(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 5')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_6(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 6')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_7(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 7')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_8(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 8')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_9(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 9')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_10(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 10')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_11(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 11')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_12(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 12')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_13(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 13')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_14(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 14')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_15(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 15')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_16(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 16')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_17(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 17')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_18(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 18')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_19(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 19')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_20(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 20')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_21(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 21')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_22(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 22')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_23(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 23')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_24(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 24')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_25(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 25')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_26(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 26')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_27(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 27')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_28(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 28')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_29(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 29')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_30(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 30')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_31(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 31')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_32(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 32')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_33(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 33')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_34(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 34')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_35(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 35')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_36(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 36')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_37(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 37')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_38(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 38')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_39(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 39')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_40(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 40')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_41(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 41')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_42(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 42')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_43(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 43')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_44(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 44')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_45(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 45')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_46(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 46')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_47(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 47')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_48(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 48')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_49(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 49')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_50(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 50')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_51(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 51')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_52(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 52')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_53(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 53')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_54(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 54')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_55(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 55')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_56(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 56')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_57(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 57')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_58(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 58')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_59(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 59')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_60(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 60')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_61(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 61')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_62(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 62')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_63(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 63')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_64(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 64')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_65(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 65')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_66(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 66')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_67(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 67')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_68(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 68')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_69(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 69')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_70(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 70')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_71(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 71')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_72(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 72')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_73(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 73')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_74(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 74')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_75(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 75')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_76(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 76')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_77(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 77')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_78(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 78')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_79(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 79')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_80(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 80')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_81(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 81')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_82(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 82')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_83(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 83')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_84(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 84')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_85(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 85')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_86(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 86')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_87(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 87')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_88(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 88')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_89(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 89')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_90(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 90')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_91(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 91')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_92(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 92')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_93(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 93')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_94(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 94')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_95(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 95')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_96(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 96')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_97(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 97')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_98(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 98')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_99(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 99')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_100(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 100')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_101(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 101')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_102(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 102')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_103(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 103')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_104(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 104')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_105(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 105')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_106(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 106')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_107(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 107')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_108(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 108')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_109(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 109')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_110(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 110')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_111(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 111')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_112(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 112')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_113(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 113')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_114(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 114')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_115(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 115')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_116(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 116')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_117(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 117')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_118(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 118')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_119(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 119')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_120(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 120')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_121(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 121')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_122(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 122')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_123(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 123')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_124(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 124')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_125(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 125')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_126(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 126')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_127(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 127')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_128(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 128')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_129(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 129')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_130(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 130')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_131(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 131')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_132(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 132')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_133(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 133')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_134(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 134')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_135(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 135')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_136(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 136')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_137(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 137')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_138(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 138')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_139(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 139')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_140(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 140')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_141(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 141')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_142(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 142')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_143(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 143')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_144(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 144')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_145(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 145')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_146(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 146')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_147(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 147')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_148(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 148')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_149(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 149')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_150(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 150')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_151(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 151')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_152(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 152')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_153(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 153')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_154(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 154')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_155(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 155')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_156(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 156')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_157(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 157')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_158(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 158')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_159(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 159')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_160(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 160')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_161(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 161')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_162(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 162')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_163(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 163')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_164(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 164')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_165(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 165')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_166(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 166')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_167(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 167')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_168(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 168')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_169(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 169')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_170(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 170')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_171(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 171')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_172(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 172')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_173(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 173')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_174(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 174')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_175(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 175')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_176(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 176')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_177(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 177')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_178(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 178')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_179(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 179')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_180(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 180')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_181(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 181')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_182(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 182')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_183(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 183')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_184(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 184')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_185(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 185')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_186(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 186')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_187(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 187')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_188(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 188')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_189(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 189')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_190(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 190')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_191(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 191')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_192(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 192')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_193(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 193')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_194(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 194')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_195(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 195')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_196(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 196')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_197(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 197')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_198(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 198')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_199(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 199')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_200(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 200')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_201(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 201')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_202(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 202')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_203(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 203')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_204(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 204')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_205(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 205')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_206(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 206')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_207(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 207')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_208(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 208')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_209(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 209')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_210(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 210')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_211(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 211')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_212(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 212')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_213(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 213')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_214(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 214')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_215(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 215')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_216(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 216')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_217(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 217')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_218(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 218')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_219(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 219')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_220(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 220')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_221(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 221')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_222(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 222')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_223(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 223')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_224(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 224')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_225(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 225')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_226(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 226')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_227(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 227')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_228(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 228')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_229(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 229')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_230(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 230')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_231(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 231')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_232(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 232')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_233(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 233')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_234(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 234')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_235(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 235')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_236(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 236')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_237(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 237')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_238(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 238')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_239(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 239')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_240(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 240')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_241(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 241')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_242(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 242')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_243(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 243')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_244(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 244')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_245(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 245')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_246(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 246')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_247(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 247')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_248(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 248')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_249(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 249')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_250(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 250')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_251(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 251')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_252(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 252')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_253(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 253')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_254(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 254')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_255(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 255')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_256(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 256')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_257(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 257')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_258(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 258')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_259(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 259')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_260(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 260')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_261(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 261')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_262(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 262')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_263(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 263')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_264(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 264')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_265(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 265')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_266(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 266')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_267(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 267')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_268(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 268')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_269(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 269')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_270(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 270')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_271(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 271')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_272(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 272')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_273(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 273')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_274(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 274')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_275(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 275')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_276(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 276')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_277(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 277')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_278(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 278')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_279(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 279')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_280(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 280')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_281(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 281')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_282(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 282')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_283(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 283')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_284(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 284')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_285(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 285')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_286(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 286')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_287(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 287')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_288(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 288')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_289(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 289')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_290(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 290')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_291(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 291')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_292(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 292')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_293(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 293')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_294(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 294')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_295(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 295')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_296(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 296')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_297(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 297')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_298(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 298')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_299(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 299')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_300(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 300')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_301(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 301')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_302(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 302')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_303(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 303')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_304(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 304')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_305(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 305')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_306(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 306')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_307(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 307')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_308(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 308')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_309(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 309')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_310(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 310')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_311(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 311')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_312(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 312')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_313(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 313')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_314(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 314')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_315(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 315')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_316(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 316')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_317(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 317')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_318(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 318')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_319(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 319')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_320(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 320')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_321(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 321')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_322(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 322')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_323(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 323')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_324(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 324')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_325(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 325')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_326(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 326')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_327(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 327')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_328(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 328')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_329(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 329')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_330(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 330')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_331(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 331')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_332(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 332')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_333(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 333')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_334(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 334')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_335(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 335')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_336(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 336')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_337(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 337')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_338(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 338')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_339(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 339')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_340(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 340')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_341(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 341')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_342(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 342')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_343(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 343')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_344(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 344')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_345(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 345')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_346(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 346')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_347(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 347')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_348(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 348')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_349(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 349')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_350(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 350')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_351(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 351')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_352(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 352')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_353(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 353')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_354(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 354')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_355(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 355')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_356(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 356')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_357(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 357')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_358(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 358')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_359(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 359')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_360(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 360')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_361(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 361')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_362(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 362')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_363(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 363')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_364(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 364')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_365(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 365')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_366(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 366')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_367(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 367')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_368(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 368')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_369(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 369')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_370(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 370')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_371(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 371')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_372(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 372')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_373(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 373')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_374(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 374')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_375(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 375')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_376(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 376')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_377(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 377')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_378(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 378')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_379(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 379')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_380(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 380')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_381(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 381')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_382(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 382')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_383(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 383')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_384(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 384')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_385(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 385')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_386(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 386')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_387(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 387')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_388(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 388')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_389(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 389')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_390(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 390')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_391(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 391')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_392(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 392')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_393(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 393')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_394(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 394')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_395(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 395')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_396(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 396')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_397(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 397')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_398(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 398')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_399(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 399')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_400(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 400')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_401(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 401')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_402(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 402')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_403(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 403')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_404(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 404')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_405(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 405')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_406(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 406')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_407(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 407')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_408(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 408')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_409(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 409')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_410(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 410')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_411(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 411')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_412(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 412')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_413(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 413')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_414(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 414')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_415(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 415')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_416(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 416')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_417(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 417')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_418(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 418')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_419(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 419')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_420(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 420')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_421(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 421')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_422(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 422')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_423(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 423')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_424(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 424')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_425(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 425')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_426(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 426')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_427(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 427')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_428(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 428')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_429(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 429')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_430(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 430')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_431(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 431')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_432(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 432')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_433(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 433')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_434(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 434')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_435(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 435')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_436(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 436')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_437(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 437')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_438(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 438')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_439(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 439')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_440(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 440')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_441(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 441')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_442(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 442')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_443(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 443')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_444(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 444')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_445(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 445')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_446(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 446')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_447(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 447')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_448(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 448')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_449(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 449')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_450(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 450')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_451(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 451')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_452(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 452')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_453(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 453')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_454(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 454')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_455(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 455')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_456(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 456')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_457(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 457')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_458(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 458')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_459(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 459')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_460(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 460')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_461(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 461')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_462(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 462')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_463(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 463')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_464(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 464')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_465(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 465')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_466(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 466')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_467(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 467')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_468(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 468')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_469(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 469')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_470(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 470')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_471(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 471')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_472(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 472')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_473(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 473')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_474(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 474')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_475(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 475')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_476(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 476')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_477(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 477')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_478(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 478')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_479(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 479')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_480(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 480')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_481(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 481')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_482(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 482')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_483(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 483')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_484(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 484')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_485(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 485')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_486(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 486')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_487(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 487')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_488(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 488')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_489(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 489')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_490(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 490')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_491(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 491')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_492(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 492')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_493(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 493')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_494(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 494')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_495(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 495')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_496(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 496')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_497(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 497')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_498(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 498')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_499(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 499')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_500(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 500')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_501(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 501')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_502(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 502')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_503(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 503')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_504(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 504')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_505(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 505')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_506(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 506')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_507(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 507')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_508(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 508')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_509(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 509')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_510(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 510')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_511(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 511')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_512(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 512')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_513(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 513')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_514(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 514')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_515(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 515')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_516(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 516')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_517(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 517')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_518(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 518')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_519(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 519')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_520(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 520')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_521(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 521')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_522(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 522')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_523(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 523')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_524(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 524')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_525(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 525')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_526(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 526')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_527(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 527')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_528(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 528')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_529(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 529')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_530(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 530')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_531(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 531')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_532(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 532')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_533(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 533')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_534(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 534')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_535(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 535')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_536(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 536')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_537(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 537')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_538(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 538')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_539(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 539')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_540(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 540')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_541(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 541')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_542(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 542')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_543(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 543')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_544(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 544')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_545(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 545')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_546(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 546')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_547(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 547')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_548(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 548')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_549(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 549')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_550(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 550')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_551(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 551')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_552(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 552')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_553(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 553')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_554(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 554')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_555(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 555')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_556(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 556')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_557(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 557')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_558(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 558')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_559(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 559')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_560(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 560')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_561(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 561')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_562(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 562')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_563(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 563')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_564(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 564')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_565(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 565')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_566(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 566')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_567(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 567')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_568(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 568')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_569(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 569')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_570(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 570')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_571(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 571')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_572(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 572')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_573(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 573')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_574(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 574')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_575(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 575')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_576(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 576')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_577(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 577')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_578(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 578')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_579(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 579')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_580(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 580')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_581(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 581')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_582(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 582')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_583(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 583')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_584(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 584')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_585(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 585')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_586(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 586')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_587(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 587')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_588(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 588')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_589(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 589')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_590(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 590')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_591(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 591')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_592(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 592')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_593(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 593')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_594(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 594')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_595(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 595')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_596(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 596')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_597(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 597')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_598(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 598')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_599(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 599')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_600(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 600')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_601(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 601')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_602(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 602')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_603(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 603')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_604(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 604')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_605(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 605')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_606(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 606')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_607(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 607')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_608(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 608')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_609(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 609')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_610(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 610')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_611(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 611')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_612(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 612')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_613(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 613')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_614(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 614')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_615(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 615')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_616(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 616')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_617(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 617')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_618(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 618')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_619(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 619')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_620(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 620')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_621(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 621')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_622(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 622')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_623(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 623')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_624(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 624')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_625(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 625')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_626(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 626')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_627(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 627')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_628(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 628')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_629(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 629')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_630(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 630')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_631(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 631')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_632(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 632')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_633(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 633')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_634(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 634')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_635(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 635')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_636(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 636')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_637(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 637')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_638(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 638')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_639(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 639')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_640(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 640')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_641(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 641')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_642(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 642')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_643(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 643')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_644(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 644')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_645(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 645')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_646(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 646')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_647(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 647')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_648(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 648')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_649(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 649')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_650(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 650')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_651(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 651')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_652(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 652')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_653(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 653')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_654(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 654')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_655(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 655')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_656(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 656')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_657(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 657')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_658(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 658')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_659(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 659')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_660(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 660')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_661(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 661')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_662(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 662')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_663(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 663')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_664(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 664')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_665(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 665')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_666(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 666')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_667(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 667')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_668(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 668')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_669(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 669')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_670(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 670')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_671(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 671')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_672(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 672')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_673(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 673')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_674(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 674')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_675(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 675')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_676(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 676')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_677(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 677')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_678(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 678')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_679(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 679')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_680(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 680')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_681(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 681')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_682(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 682')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_683(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 683')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_684(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 684')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_685(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 685')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_686(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 686')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_687(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 687')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_688(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 688')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_689(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 689')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_690(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 690')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_691(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 691')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_692(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 692')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_693(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 693')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_694(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 694')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_695(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 695')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_696(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 696')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_697(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 697')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_698(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 698')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_699(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 699')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_700(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 700')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_701(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 701')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_702(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 702')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_703(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 703')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_704(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 704')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_705(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 705')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_706(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 706')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_707(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 707')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_708(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 708')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_709(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 709')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_710(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 710')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_711(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 711')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_712(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 712')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_713(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 713')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_714(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 714')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_715(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 715')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_716(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 716')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_717(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 717')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_718(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 718')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_719(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 719')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_720(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 720')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_721(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 721')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_722(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 722')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_723(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 723')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_724(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 724')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_725(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 725')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_726(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 726')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_727(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 727')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_728(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 728')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_729(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 729')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_730(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 730')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_731(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 731')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_732(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 732')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_733(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 733')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_734(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 734')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_735(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 735')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_736(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 736')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_737(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 737')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_738(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 738')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_739(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 739')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_740(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 740')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_741(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 741')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_742(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 742')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_743(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 743')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_744(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 744')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_745(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 745')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_746(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 746')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_747(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 747')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_748(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 748')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_749(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 749')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_750(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 750')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_751(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 751')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_752(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 752')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_753(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 753')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_754(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 754')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_755(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 755')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_756(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 756')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_757(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 757')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_758(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 758')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_759(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 759')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_760(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 760')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_761(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 761')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_762(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 762')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_763(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 763')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_764(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 764')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_765(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 765')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_766(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 766')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_767(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 767')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_768(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 768')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_769(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 769')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_770(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 770')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_771(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 771')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_772(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 772')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_773(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 773')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_774(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 774')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_775(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 775')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(u) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_776(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 776')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(v) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_777(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 777')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(w) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_778(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 778')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(x) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_779(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 779')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(y) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_780(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 780')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(z) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_781(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 781')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(a) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_782(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 782')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(b) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_783(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 783')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(c) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_784(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 784')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(d) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_785(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 785')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(e) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_786(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 786')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(f) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_787(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 787')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(g) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_788(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 788')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(h) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_789(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 789')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(i) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_790(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 790')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(j) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_791(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 791')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(k) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_792(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 792')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(l) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_793(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 793')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(m) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_794(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 794')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(n) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_795(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 795')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(o) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_796(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 796')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(p) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_797(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 797')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(q) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_798(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 798')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(r) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_799(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 799')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(s) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))


def plot_block_800(ax, data_dict):
    X = data_dict['X']
    Y = data_dict['Y']
    title = data_dict.get('title', 'Plot 800')
    im = ax.imshow(np.clip(X, 0, 1), aspect='auto', origin='lower', interpolation='bicubic')
    ax.set_title(f"(t) " + title, fontsize=8)
    if 'cbar' in data_dict and data_dict['cbar']:
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(0.01, 0.01, f"mean={np.mean(X):.4f}\nstd={np.std(X):.4f}", transform=ax.transAxes, fontsize=6,
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))



    # programmatically create 120 figure pages each with multiple subplot grids
    total_pages = 60
    grids = [(3,3),(4,3),(4,4),(5,4),(6,4)]
    label_ord = 0
    for page in range(total_pages):
        g = grids[page % len(grids)]
        rows, cols = g
        fig, axes = plt.subplots(rows, cols, figsize=(cols*3.0, rows*2.4), constrained_layout=True)
        axes = np.array(axes).reshape(-1)
        for slot in range(len(axes)):
            lab = next(label_iter)
            key = list(features.keys())[slot % len(features)]
            feat_names = list(features[key].keys())
            sel_feat = feat_names[(page + slot) % len(feat_names)]
            arr = features[key][sel_feat]
            small = np.copy(arr)
            if small.ndim == 2 and small.shape[0] > 0:
                data_dict = {'X': small, 'Y': small, 'title': f'{key} {sel_feat}', 'cbar': True}
                # print numeric values used for this subplot to stdout
                print(f'PAGE {page+1} SLOT {slot+1} LABEL ({lab}) DATASET {key} FEATURE {sel_feat} SHAPE {small.shape} MEAN {np.mean(small):.6g} STD {np.std(small):.6g} MIN {np.min(small):.6g} MAX {np.max(small):.6g}')
                values_to_log[f'page{page+1}_slot{slot+1}_label_{lab}'] = small
                im = axes[slot].imshow(np.clip(small, 0, np.percentile(small,99)), origin='lower', interpolation='bicubic', aspect='auto')
                axes[slot].set_title('('+lab+') '+f'{key} {sel_feat}', fontsize=8)
                axes[slot].set_xticks([]); axes[slot].set_yticks([])
                cbar = fig.colorbar(im, ax=axes[slot], fraction=0.046, pad=0.04)
                axes[slot].text(0.01, 0.02, f"mean={np.mean(small):.4f}\nstd={np.std(small):.4f}", transform=axes[slot].transAxes, fontsize=6, bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            else:
                axes[slot].text(0.5,0.5,'N/A',ha='center',va='center')
        # save page
        if pp is not None:
            pp.savefig(fig)
        plt.close(fig)
        figcount += 1
    if pp is not None:
        pp.close()
    save_values_log(values_to_log, values_log_path)
    print('Saved PDF to', pdf_path)
    print('Saved numeric values log to', values_log_path)
    return pdf_path, values_log_path

if __name__ == '__main__':
    run_all(save_pdf=True, download=True, verbose=True)


Files already downloaded and verified


In [21]:
#!/usr/bin/env python3
import os
import sys
import math
from datetime import datetime
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def make_synthetic_feature(size=(64,64), seed=0, kind=0):
    rng = np.random.RandomState(seed)
    x = np.linspace(-3,3,size[1])
    y = np.linspace(-3,3,size[0])
    X, Y = np.meshgrid(x,y)
    if kind == 0:
        Z = np.exp(-(X**2+Y**2)) + 0.1*rng.randn(*size)
    elif kind == 1:
        Z = np.sin(3*X)*np.cos(3*Y) + 0.05*rng.randn(*size)
    elif kind == 2:
        Z = np.abs(np.fft.fftshift(np.fft.ifft2((rng.randn(*size)+1j*rng.randn(*size))*np.exp(-((X**2+Y**2)/6))))).real
    elif kind == 3:
        Z = np.gradient(np.exp(-(X**2+0.5*Y**2)) , axis=1)[0] + 0.02*rng.randn(*size)
    else:
        Z = rng.randn(*size)
    # normalize to 0..1
    Z = Z - Z.min()
    if Z.max() != 0:
        Z = Z / Z.max()
    return Z

def stats(arr):
    return dict(shape=arr.shape, min=float(arr.min()), max=float(arr.max()), mean=float(arr.mean()), std=float(arr.std()))

def main(out_pdf='complex_subplots_output_light.pdf', out_log='plot_values_used_light.txt'):
    timestamp = datetime.utcnow().isoformat()
    print("Starting lightweight plot generator at", timestamp)
    pages = 12
    per_page = 6
    total = pages * per_page
    label_ord = 0
    alph = list("abcdefghijklmnopqrstuvwxyz")
    values = {}
    pp = PdfPages(out_pdf)
    for p in range(pages):
        rows = 2
        cols = 3
        fig, axes = plt.subplots(rows, cols, figsize=(cols*3.5, rows*3.0), constrained_layout=True)
        axes = axes.flatten()
        for i, ax in enumerate(axes):
            lab = alph[label_ord % len(alph)] if label_ord < 26 else f"{label_ord+1}"
            kind = (p*per_page + i) % 6
            arr = make_synthetic_feature(size=(128,128), seed=p*per_page+i, kind=kind)
            im = ax.imshow(arr, origin='lower', interpolation='bicubic', aspect='auto')
            ax.set_xticks([]); ax.set_yticks([])
            ax.set_title(f"({lab}) Synthetic feature kind {kind}", fontsize=9)
            ax.text(0.02,0.02, f"mean={arr.mean():.4f}\\nstd={arr.std():.4f}", transform=ax.transAxes, fontsize=7,
                    bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
            values[f'page{p+1}_slot{i+1}_label_{lab}'] = arr
            print(f'PAGE {p+1} SLOT {i+1} LABEL ({lab}) SHAPE {arr.shape} MEAN {arr.mean():.6g} STD {arr.std():.6g} MIN {arr.min():.6g} MAX {arr.max():.6g}')
            label_ord += 1
        pp.savefig(fig)
        plt.close(fig)
    pp.close()
    # write numeric summary log
    with open(out_log, 'w') as f:
        f.write(f'Generated at {timestamp}\\n')
        for k, v in values.items():
            s = stats(v)
            f.write(f"{k}: shape={s['shape']}, min={s['min']:.6g}, max={s['max']:.6g}, mean={s['mean']:.6g}, std={s['std']:.6g}\\n")
    print("Saved PDF:", out_pdf)
    print("Saved numeric log:", out_log)

if __name__ == '__main__':
    main()


Starting lightweight plot generator at 2025-08-08T04:14:53.519882
PAGE 1 SLOT 1 LABEL (a) SHAPE (128, 128) MEAN 0.283206 STD 0.132905 MIN 0 MAX 1
PAGE 1 SLOT 2 LABEL (b) SHAPE (128, 128) MEAN 0.500953 STD 0.217803 MIN 0 MAX 1
PAGE 1 SLOT 3 LABEL (c) SHAPE (128, 128) MEAN 0.284165 STD 0.149925 MIN 0 MAX 1
PAGE 1 SLOT 4 LABEL (d) SHAPE (128, 128) MEAN 0.467627 STD 0.121703 MIN 0 MAX 1
PAGE 1 SLOT 5 LABEL (e) SHAPE (128, 128) MEAN 0.509185 STD 0.115764 MIN 0 MAX 1
PAGE 1 SLOT 6 LABEL (f) SHAPE (128, 128) MEAN 0.503125 STD 0.137579 MIN 0 MAX 1
PAGE 2 SLOT 1 LABEL (g) SHAPE (128, 128) MEAN 0.305686 STD 0.128427 MIN 0 MAX 1
PAGE 2 SLOT 2 LABEL (h) SHAPE (128, 128) MEAN 0.501913 STD 0.226204 MIN 0 MAX 1
PAGE 2 SLOT 3 LABEL (i) SHAPE (128, 128) MEAN 0.289706 STD 0.151169 MIN 0 MAX 1
PAGE 2 SLOT 4 LABEL (j) SHAPE (128, 128) MEAN 0.487242 STD 0.130921 MIN 0 MAX 1
PAGE 2 SLOT 5 LABEL (k) SHAPE (128, 128) MEAN 0.509877 STD 0.13245 MIN 0 MAX 1
PAGE 2 SLOT 6 LABEL (l) SHAPE (128, 128) MEAN 0.468409 

In [29]:
#!/usr/bin/env python3
"""
qc_plotting_suite.py

Comprehensive plotting suite for manuscript "Operator-Algebraic Approach for QCNNs".
Generates a multi-panel PDF (results_subplots.pdf) and writes numeric values used for plotting
to used_values.txt.

Requires:
    - Python 3.9+
    - numpy, scipy, matplotlib, torchvision, torch, sklearn, tqdm
Install with:
    pip install numpy scipy matplotlib torchvision torch scikit-learn tqdm
(If you prefer tensorflow datasets, adapt the loader block.)

Author: ChatGPT (script tailored to the user's manuscript; uses MNIST & FashionMNIST)
Manuscript refs: see file citations in the chat (Operator-Algebraic.txt). :contentReference[oaicite:3]{index=3}
"""

import os
import math
import numpy as np
import matplotlib
matplotlib.use('Agg')   # safe backend (produces PDF)
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Rectangle
from matplotlib.colors import SymLogNorm
from tqdm import tqdm
from scipy import ndimage, fftpack
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import json
import datetime

# Try to import torchvision (for MNIST/FashionMNIST). If not installed, ask user to install.
try:
    import torch
    import torchvision
    import torchvision.transforms as transforms
except Exception as e:
    raise RuntimeError(
        "This script requires torchvision and torch. Install with: pip install torchvision torch"
    ) from e

# -------------------------
# Configuration parameters
# -------------------------
OUT_PDF = "results_subplots.pdf"
VALUES_TXT = "used_values.txt"
SAMPLE_COUNT = 5           # number of example images per dataset to show
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Padding/stride/filters choices referenced from manuscript (cyclic/symmetric, s=2 etc). :contentReference[oaicite:4]{index=4}
PAD = 2                    # padding size (P)
STRIDE = 2                 # stride s
TARGET_IMG_DIM = 28
PADDED_DIM = TARGET_IMG_DIM + 2 * PAD  # e.g., 32 as in manuscript examples. :contentReference[oaicite:5]{index=5}

# Choose kernels for image-space convolution (a bank)
KERNELS = {
    'sobel_x': np.array([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=float),
    'sobel_y': np.array([[-1,-2,-1],[0,0,0],[1,2,1]], dtype=float),
    'gaussian_5': ndimage.gaussian_filter(np.pad(np.zeros((1,1)), 2), 1.0)  # placeholder
}
# Replace gaussian properly:
gx = np.outer([1,4,6,4,1],[1,4,6,4,1]).astype(float)
KERNELS['gaussian_5'] = gx / gx.sum()

# Manuscript reported special values for intensities and gains — we will compute from data but also
# display the published reference values in a plot annotation (so reviewers can see both).
MANUSCRIPT_STATS = {
    'QPE_mean_gain_mnist': 41.3,    # 41.3x example reported in manuscript for QPE intensity amplification. :contentReference[oaicite:6]{index=6}
    'QPE_mean_gain_fashion': 20.4,
    'QPE_Imax_mnist': 72.37,
    'QPE_Imax_fashion': 131.2,
    'AA_mean_mnist': 0.0755,
    'AA_mean_fashion': 0.1006,
    'reported_accuracy_mnist': 98.7,
    'reported_accuracy_fashion': 94.7
}

# -------------------------
# Utilities: padding ops
# -------------------------
def pad_cyclic(img, pad):
    """Wrap (cyclic) padding — corresponds to cyclic boundary conditions."""
    if pad == 0:
        return img.copy()
    return np.pad(img, pad, mode='wrap')

def pad_symmetric(img, pad):
    """Symmetric (reflect) padding — corresponds to symmetric padding described in manuscript."""
    if pad == 0:
        return img.copy()
    return np.pad(img, pad, mode='reflect')

# -------------------------
# Quantum-like classical simulations
# -------------------------
def qft2d_magnitude(img):
    """Return log-magnitude of 2D FFT (centered) to emulate QFT intensity visualization."""
    F = fftpack.fftshift(fftpack.fft2(img))
    mag = np.abs(F)
    # use log scale for dynamic range
    return np.log1p(mag)

def simulate_qpe_amplify(img_fft, phase_boost_factor=3.0):
    """
    Simulate QPE-like phase amplification: multiply phases of FFT components by a boost factor,
    then invert to image domain and return magnitude/intensity map.
    This classical operation is only an analogue of QPE enhancing spectral components.
    """
    F = fftpack.fft2(img_fft)
    mag = np.abs(F)
    ph = np.angle(F)
    ph2 = ph * phase_boost_factor
    F2 = mag * np.exp(1j * ph2)
    img_back = np.real(fftpack.ifft2(F2))
    return img_back

def simulate_amplitude_amplification(vector, target_indices=None, iterations=3):
    """
    Simple vector-space amplitude amplification: reflect about mean and then about the target subspace.
    This is a toy classical analogue of Grover iterations: Q = -B S0 B^dagger S_chi
    Implemented as iterative reflections to magnify entries at target_indices.
    """
    v = vector.copy().astype(float)
    if target_indices is None or len(target_indices)==0:
        # choose top-k indices as 'target' if none provided
        k = max(1, int(0.02 * v.size))
        target_indices = np.argpartition(np.abs(v).ravel(), -k)[-k:]
    # normalize vector
    if np.linalg.norm(v) > 0:
        v = v / np.linalg.norm(v)
    for _ in range(iterations):
        # reflection about zero state (S0): reflect v -> -v is too trivial, instead reflect about average vector
        mean_v = np.mean(v)
        v = 2 * mean_v - v
        # now inversion about target (S_chi): boost target components
        flat = v.ravel()
        flat[target_indices] = -flat[target_indices]  # flip sign to mimic reflection
        v = flat.reshape(v.shape)
        # renormalize to keep stability
        nrm = np.linalg.norm(v)
        if nrm > 0:
            v = v / nrm
    return v

# -------------------------
# Load datasets (MNIST, FashionMNIST)
# -------------------------
def load_datasets(download=True, max_examples_per_class=1000):
    """Return small arrays for MNIST and FashionMNIST for analysis (train subsets)."""
    # using torchvision datasets
    transform = transforms.Compose([transforms.ToTensor()])  # values in [0,1]
    mnist = torchvision.datasets.MNIST(root='./data', train=True, download=download, transform=transform)
    fashion = torchvision.datasets.FashionMNIST(root='./data', train=True, download=download, transform=transform)
    # Convert to numpy arrays
    def subset_to_numpy(ds, limit=None):
        imgs = []
        labels = []
        for i, (img, lbl) in enumerate(ds):
            imgs.append(img.squeeze().numpy())
            labels.append(int(lbl))
            if limit is not None and len(imgs) >= limit:
                break
        return np.stack(imgs, axis=0), np.array(labels, dtype=int)
    # We'll take up to 2000 examples for speed (but can be increased)
    mn_x, mn_y = subset_to_numpy(mnist, limit=2000)
    fa_x, fa_y = subset_to_numpy(fashion, limit=2000)
    return (mn_x, mn_y), (fa_x, fa_y)

# -------------------------
# Compute several derived quantities for plotting and record values
# -------------------------
def compute_analysis_samples(images, labels, dataset_name, show_count=SAMPLE_COUNT):
    """Compute sample images, padded versions, FFTs, QPE, AA etc."""
    rng = np.random.default_rng(RANDOM_SEED)
    n = min(show_count, len(images))
    idx = rng.choice(len(images), size=n, replace=False)
    out = []
    for i in idx:
        img = images[i]
        lab = labels[i]
        padded_cyclic = pad_cyclic(img, PAD)
        padded_symm = pad_symmetric(img, PAD)
        fft_base = qft2d_magnitude(padded_cyclic)
        # simulate QPE: apply phase amplification in frequency domain using a boost tuned by dataset
        boost = 3.8 if dataset_name.lower().startswith('mnist') else 3.4185  # numbers from manuscript examples. :contentReference[oaicite:7]{index=7}
        qpe_img = simulate_qpe_amplify(padded_cyclic, phase_boost_factor=boost)
        # compute convolutions
        convs = {}
        for kn, kernel in KERNELS.items():
            convs[kn] = ndimage.convolve(padded_cyclic, kernel, mode='wrap')
        # simulate AA on main image vector (flattened)
        aa_vec = simulate_amplitude_amplification(padded_cyclic.flatten(), iterations=3)
        aa_img = aa_vec.reshape(padded_cyclic.shape)
        # intensity stats
        stats = {
            'mean_original': img.mean(),
            'mean_padded_cyclic': padded_cyclic.mean(),
            'mean_padded_symm': padded_symm.mean(),
            'mean_qpe': qpe_img.mean(),
            'max_qpe': qpe_img.max(),
            'mean_aa': aa_img.mean(),
            'max_aa': aa_img.max()
        }
        out.append({
            'index': int(i),
            'label': int(lab),
            'img': img,
            'padded_cyclic': padded_cyclic,
            'padded_symm': padded_symm,
            'fft_base': fft_base,
            'qpe_img': qpe_img,
            'convs': convs,
            'aa_img': aa_img,
            'stats': stats
        })
    return out

# -------------------------
# Create plots
# -------------------------
def make_plots(mnist_samples, fashion_samples, mn_overall_stats, fa_overall_stats):
    """
    Create a multi-panel figure with many subplots:
    Layout plan (4 rows x 6 cols = 24 panels) — this can be adjusted.
    Each panel annotated (a), (b), ...
    """
    # Prepare plotting grid
    nrows = 4
    ncols = 6
    fig = plt.figure(figsize=(18, 12))
    gs = gridspec.GridSpec(nrows, ncols, figure=fig, wspace=0.4, hspace=0.45)

    # helper to annotate subplot letters
    letters = []
    for i in range(nrows * ncols):
        # produce (a), (b), ...
        letters.append('({})'.format(chr(ord('a') + i)))

    used_values = {}  # dictionary to collect numeric values used per panel

    # Panel mapping (assign semantic plots to specific grid cells)
    panel = 0

    # Row 1: example images (a)-(f): MNIST samples (first 3) and Fashion (next 3)
    for i in range(3):
        ax = fig.add_subplot(gs[0, i])
        samp = mnist_samples[i]
        ax.imshow(samp['img'], cmap='gray', vmin=0, vmax=1)
        ax.set_title(f"MNIST sample idx={samp['index']} lab={samp['label']}")
        ax.axis('off')
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_mnist_raw_{i}'] = samp['stats']
        panel += 1

    for i in range(3):
        ax = fig.add_subplot(gs[0, 3 + i])
        samp = fashion_samples[i]
        ax.imshow(samp['img'], cmap='gray', vmin=0, vmax=1)
        ax.set_title(f"Fashion sample idx={samp['index']} lab={samp['label']}")
        ax.axis('off')
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_fashion_raw_{i}'] = samp['stats']
        panel += 1

    # Row 2: padded and FFT/QPE visualizations (g)-(l)
    # MNIST padded (cyclic), FFT magnitude, QPE amplified image
    for i in range(3):
        samp = mnist_samples[i]
        # padded cyclic
        ax = fig.add_subplot(gs[1, i])
        ax.imshow(samp['padded_cyclic'], cmap='gray', vmin=0, vmax=1)
        ax.set_title("MNIST padded (cyclic)")
        ax.axis('off')
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_mnist_padded_cyclic_{i}'] = samp['stats']
        panel += 1

    # Fashion padded (symmetric), FFT, QPE
    for i in range(3):
        samp = fashion_samples[i]
        ax = fig.add_subplot(gs[1, 3 + i])
        ax.imshow(samp['padded_symm'], cmap='gray', vmin=0, vmax=1)
        ax.set_title("Fashion padded (symmetric)")
        ax.axis('off')
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_fashion_padded_symm_{i}'] = samp['stats']
        panel += 1

    # Row 3: FFT, QPE amplitude & convolution maps (m)-(r)
    # show MNIST sample FFT magnitude, QPE result, AA normalized
    for i in range(3):
        samp = mnist_samples[i]
        ax = fig.add_subplot(gs[2, i])
        im = ax.imshow(samp['fft_base'], norm=SymLogNorm(linthresh=0.01), cmap='inferno')
        ax.set_title("MNIST FFT magnitude (log)")
        ax.axis('off')
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_mnist_fft_{i}'] = {
            'fft_mean': float(np.mean(samp['fft_base'])),
            'fft_max': float(np.max(samp['fft_base']))
        }
        panel += 1

    # Fashion: QPE and AA images
    for i in range(3):
        samp = fashion_samples[i]
        # QPE
        ax = fig.add_subplot(gs[2, 3 + i])
        # scale qpe for display
        qpe = samp['qpe_img']
        qpe_scaled = (qpe - qpe.min()) / (qpe.max() - qpe.min() + 1e-12)
        im = ax.imshow(qpe_scaled, cmap='magma')
        ax.set_title("Fashion QPE-simulated")
        ax.axis('off')
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_fashion_qpe_{i}'] = samp['stats']
        panel += 1

    # Row 4: Convolution maps, PCA, accuracy bars, confusion matrix (s)-(x)
    # We'll place a set of conv outputs for MNIST and Fashion
    # MNIST convs (sobel_x, sobel_y, gaussian)
    conv_kernels = list(KERNELS.keys())
    for i in range(3):
        samp = mnist_samples[i]
        ax = fig.add_subplot(gs[3, i])
        cn = conv_kernels[i % len(conv_kernels)]
        conv_map = samp['convs'][cn]
        # normalize conv map for display
        cm = conv_map
        cm_plot = (cm - cm.min()) / (cm.max() - cm.min() + 1e-12)
        ax.imshow(cm_plot, cmap='gray')
        ax.set_title(f"MNIST conv {cn}")
        ax.axis('off')
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_mnist_conv_{cn}_{i}'] = {
            'conv_mean': float(cm.mean()),
            'conv_std': float(cm.std()),
            'conv_max': float(cm.max())
        }
        panel += 1

    # Fashion convs
    for i in range(3):
        samp = fashion_samples[i]
        ax = fig.add_subplot(gs[3, 3 + i])
        cn = conv_kernels[i % len(conv_kernels)]
        conv_map = samp['convs'][cn]
        cm_plot = (conv_map - conv_map.min()) / (conv_map.max() - conv_map.min() + 1e-12)
        ax.imshow(cm_plot, cmap='gray')
        ax.set_title(f"Fashion conv {cn}")
        ax.axis('off')
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_fashion_conv_{cn}_{i}'] = {
            'conv_mean': float(conv_map.mean()),
            'conv_std': float(conv_map.std()),
            'conv_max': float(conv_map.max())
        }
        panel += 1

    # If panels remain (less than 24 used) fill them with higher-level analysis plots:
    # we have used panel up to 'panel' variable; fill subsequent panels with PCA scatter, intensity bar, fake accuracy line, confusion matrix.
    # Next panel: PCA scatter of MNIST feature vectors (use a small subset)
    # Compute feature vectors (flattened padded images) for a subset of data to show clustering
    # load a larger subset for PCA (we do minimal compute)
    # We'll use the inputs passed in global mn_overall_stats / fa_overall_stats for aggregated metrics printout
    # We'll compactly plot following extra panels (if any left)
    total_panels = nrows * ncols
    remaining = total_panels - panel
    # Build aggregated plots if at least 1 panel left
    extra_axs = []
    # Create PCA scatter in a new figure slot if possible
    if remaining >= 4:
        # Prepare a small set for PCA from the provided overall stats arrays (they are full arrays passed)
        # We'll request them from caller by reading files saved earlier. But for simplicity use mn_overall_stats, fa_overall_stats arrays
        # The caller passed them as (images, labels)
        mn_images, mn_labels = mn_overall_stats
        fa_images, fa_labels = fa_overall_stats
        # pick 300 examples from each
        sel_cnt = 300
        sel_idx_m = np.random.choice(len(mn_images), sel_cnt, replace=False)
        sel_idx_f = np.random.choice(len(fa_images), sel_cnt, replace=False)
        feats = []
        labs = []
        for idx in sel_idx_m:
            feats.append(np.ravel(pad_cyclic(mn_images[idx], PAD)))
            labs.append(('mn', int(mn_labels[idx])))
        for idx in sel_idx_f:
            feats.append(np.ravel(pad_symmetric(fa_images[idx], PAD)))
            labs.append(('fa', int(fa_labels[idx])))
        feats = np.array(feats)
        scaler = StandardScaler()
        feats_s = scaler.fit_transform(feats)
        pca = PCA(n_components=2)
        coords = pca.fit_transform(feats_s)
        # plot PCA
        ax = fig.add_subplot(gs[4-1, 0:3]) if (nrows>=4) else fig.add_subplot(gs[3,0:3])
        # scatter colored by dataset
        mask_m = np.array([1 if ds=='mn' else 0 for ds, _ in labs], dtype=bool)
        ax.scatter(coords[mask_m, 0], coords[mask_m, 1], s=6, alpha=0.6, label='MNIST')
        ax.scatter(coords[~mask_m, 0], coords[~mask_m, 1], s=6, alpha=0.6, label='Fashion')
        ax.set_title("PCA of padded image features (first 2 components)")
        ax.legend(markerscale=2)
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_pca'] = {
            'pca_explained_ratio': pca.explained_variance_ratio_.tolist()
        }
        panel += 1

        # Panel: intensity bar charts comparing means and maxes for aggregate data
        # compute aggregated means for datasets for a moderate subset
        msel = np.random.choice(len(mn_images), 500, replace=False)
        fsel = np.random.choice(len(fa_images), 500, replace=False)
        mn_means = [np.mean(mn_images[i]) for i in msel]
        fa_means = [np.mean(fa_images[i]) for i in fsel]
        mn_maxs = [np.max(mn_images[i]) for i in msel]
        fa_maxs = [np.max(fa_images[i]) for i in fsel]
        ax = fig.add_subplot(gs[3, 3])  # using available slot
        ax.bar([0,1], [np.mean(mn_means), np.mean(fa_means)], yerr=[np.std(mn_means), np.std(fa_means)], tick_label=['MNIST mean','Fashion mean'])
        ax.set_title("Mean intensity comparison (subset)")
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_intensity_stats'] = {
            'mn_mean_agg': float(np.mean(mn_means)),
            'mn_std_agg': float(np.std(mn_means)),
            'fa_mean_agg': float(np.mean(fa_means)),
            'fa_std_agg': float(np.std(fa_means))
        }
        panel += 1

        # Panel: simulated accuracy curves (we use manuscript reported numbers to generate smooth curves)
        x_epochs = np.arange(1, 13)
        # create baseline and quantum-enhanced curves using manuscript deltas. :contentReference[oaicite:8]{index=8}
        baseline_mnist = np.linspace(90.0, 98.4, len(x_epochs))
        quantum_mnist = baseline_mnist * 0.0 + np.linspace(92.0, MANUSCRIPT_STATS['reported_accuracy_mnist'], len(x_epochs))
        baseline_fa = np.linspace(80.0, 90.3, len(x_epochs))
        quantum_fa = baseline_fa * 0.0 + np.linspace(85.0, MANUSCRIPT_STATS['reported_accuracy_fashion'], len(x_epochs))
        ax = fig.add_subplot(gs[3, 4])
        ax.plot(x_epochs, baseline_mnist, label='CNN (MNIST baseline)', linestyle='--')
        ax.plot(x_epochs, quantum_mnist, label='QCNN (MNIST reported)', linewidth=2)
        ax.plot(x_epochs, baseline_fa, label='CNN (Fashion baseline)', linestyle='--')
        ax.plot(x_epochs, quantum_fa, label='QCNN (Fashion reported)', linewidth=2)
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Accuracy (%)")
        ax.set_title("Training/validation style accuracy curves (simulated)")
        ax.legend(fontsize=8)
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_accuracy_curves'] = {
            'baseline_mnist_end': float(baseline_mnist[-1]),
            'quantum_mnist_end': float(quantum_mnist[-1]),
            'baseline_fa_end': float(baseline_fa[-1]),
            'quantum_fa_end': float(quantum_fa[-1])
        }
        panel += 1

        # Panel: confusion matrix example (produce from simple logistic on flattened padded images)
        # Train a small logistic regression to show a confusion matrix (this is illustrative only)
        # Build data subset (use 100 per class for 10 classes)
        samples_per_cls = 50
        class_samples = []
        class_labels = []
        # collect across mn & fa for first 10 labels
        mn_for_lr = mn_images[:samples_per_cls * 2]
        mn_lab_for_lr = mn_labels[:samples_per_cls * 2]
        # combine and flatten
        X_lr = []
        y_lr = []
        for i in range(10):
            idxs = np.where(mn_lab_for_lr == (i % 10))[0]
            if idxs.size == 0:
                continue
            for j in idxs[:samples_per_cls]:
                flat = pad_cyclic(mn_for_lr[j], PAD).ravel()
                X_lr.append(flat)
                y_lr.append(i)
        X_lr = np.array(X_lr)
        y_lr = np.array(y_lr)
        # simple scaling and logistic
        X_lr_s = StandardScaler().fit_transform(X_lr)
        clf = LogisticRegression(max_iter=600, multi_class='multinomial', solver='saga')
        try:
            clf.fit(X_lr_s, y_lr)
            y_pred = clf.predict(X_lr_s)
            cm = confusion_matrix(y_lr, y_pred)
        except Exception as e:
            cm = np.zeros((10,10))
        ax = fig.add_subplot(gs[3, 5])
        im = ax.imshow(cm, cmap='Blues')
        ax.set_title("Example confusion matrix (illustrative)")
        ax.set_xlabel("pred")
        ax.set_ylabel("true")
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
        ax.text(0.01, 1.02, letters[panel], transform=ax.transAxes, fontsize=12, fontweight='bold', va='bottom')
        used_values[f'panel_{panel}_confusion'] = cm.tolist()
        panel += 1

    # Tight layout and save PDF
    fig.suptitle("Comprehensive Results Analysis Subplots — QCNN analogues", fontsize=18, y=0.99)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(OUT_PDF, dpi=300)
    plt.close(fig)

    # helper to make JSON-safe
    def json_safe(o):
        import numpy as np
        if isinstance(o, (np.floating, np.float32, np.float64)):
            return float(o)
        if isinstance(o, (np.integer, np.int32, np.int64)):
            return int(o)
        if isinstance(o, np.ndarray):
            return o.tolist()
        return o

    # save numeric values inside make_plots so used_values is in scope
    with open(VALUES_TXT, 'w') as f:
        f.write("# Numeric values used for plotting (generated {})\n".format(datetime.datetime.now().isoformat()))
        json.dump(used_values, f, indent=2, default=json_safe)


# -------------------------
# Main orchestration
# -------------------------
def main():
    print("Loading datasets (this will download MNIST & FashionMNIST if not present)...")
    (mn_x, mn_y), (fa_x, fa_y) = load_datasets(download=True)
    print(f"Loaded MNIST {mn_x.shape}, Fashion {fa_x.shape}")
    # Prepare analysis samples
    mn_samples = compute_analysis_samples(mn_x, mn_y, 'mnist', show_count=SAMPLE_COUNT)
    fa_samples = compute_analysis_samples(fa_x, fa_y, 'fashion', show_count=SAMPLE_COUNT)
    # Call plotting function (we also pass full arrays for PCA & aggregated plots)
    make_plots(mn_samples, fa_samples, (mn_x, mn_y), (fa_x, fa_y))
    # Print summary values to console (mimic 'print values which are used for plotting subplots')
    with open(VALUES_TXT, 'r') as fh:
        txt = fh.read()
    print("---- Numeric summary (first 1000 chars) ----")
    print(txt[:1000])
    print("... (values file contains full numeric list)")

if __name__ == "__main__":
    main()


Loading datasets (this will download MNIST & FashionMNIST if not present)...
Loaded MNIST (2000, 28, 28), Fashion (2000, 28, 28)
---- Numeric summary (first 1000 chars) ----
# Numeric values used for plotting (generated 2025-08-08T09:57:16.683389)
{
  "panel_0_mnist_raw_0": {
    "mean_original": 0.09574329853057861,
    "mean_padded_cyclic": 0.0733034610748291,
    "mean_padded_symm": 0.0733034610748291,
    "mean_qpe": 0.0733034610748291,
    "max_qpe": 1.0139317512512207,
    "mean_aa": 0.018586641910719123,
    "max_aa": 0.048227663299969
  },
  "panel_1_mnist_raw_1": {
    "mean_original": 0.1285964399576187,
    "mean_padded_cyclic": 0.09845665097236633,
    "mean_padded_symm": 0.09845665097236633,
    "mean_qpe": 0.09845665097236633,
    "max_qpe": 1.085979700088501,
    "mean_aa": 0.017161574896299797,
    "max_aa": 0.0259282808528858
  },
  "panel_2_mnist_raw_2": {
    "mean_original": 0.15201079845428467,
    "mean_padded_cyclic": 0.11638328433036804,
    "mean_padded_symm": 

In [31]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
from PIL import Image
import torchvision
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy.fft import fft2, fftshift
from scipy.ndimage import zoom
from scipy.signal import spectrogram
from scipy.stats import entropy
import random
import csv
import datetime

os.makedirs("results", exist_ok=True)
np.random.seed(42)
random.seed(42)

device = "cpu"

def load_mnist_fashion(limit_per_class=None):
    transform = transforms.Compose([transforms.ToTensor()])
    mn_train = torchvision.datasets.MNIST(root=".", train=True, download=True, transform=transform)
    mn_test = torchvision.datasets.MNIST(root=".", train=False, download=True, transform=transform)
    fm_train = torchvision.datasets.FashionMNIST(root=".", train=True, download=True, transform=transform)
    fm_test = torchvision.datasets.FashionMNIST(root=".", train=False, download=True, transform=transform)
    def gather(ds):
        imgs = []
        labels = []
        for i,(x,y) in enumerate(ds):
            imgs.append(x.numpy().squeeze(0))
            labels.append(int(y))
        imgs = np.stack(imgs).astype(np.float32)
        labels = np.array(labels,dtype=int)
        if limit_per_class is not None:
            sel = []
            for c in np.unique(labels):
                idx = np.where(labels==c)[0][:limit_per_class]
                sel.extend(idx.tolist())
            imgs = imgs[sel]
            labels = labels[sel]
        return imgs, labels
    mnx,mny = gather(mn_train)
    fmx,fmy = gather(fm_train)
    return (mnx,mny),(fmx,fmy),(mn_test,mn_test.targets.numpy())

def minmax_scale(img):
    mi = img.min()
    ma = img.max()
    if ma==mi:
        return np.zeros_like(img)
    return (img - mi)/(ma-mi)

def simulate_padding(img, P=3, mode="cyclic"):
    h,w = img.shape
    if mode=="cyclic":
        out = np.pad(img, P, mode='wrap')
    elif mode=="symmetric":
        out = np.pad(img, P, mode='reflect')
    else:
        out = np.pad(img, P, mode='constant', constant_values=0)
    return out

def simulate_stride(img, s=2):
    return img[::s, ::s]

def qpe_like_phase_amplify(img, m=3):
    x = img.astype(np.float64)
    phi = np.angle(np.fft.fftshift(np.fft.fft2(x+1e-9)))
    scale = (2.0**m)
    amplified = np.real(np.fft.ifft2(np.fft.ifftshift(np.exp(1j*phi*scale))))
    amplified = np.nan_to_num(amplified)
    amplified = amplified - amplified.min()
    if amplified.max()>0:
        amplified = amplified/amplified.max()
    return amplified

def amplitude_amplification_sim(img, k=1):
    vec = img.flatten().astype(np.float64)
    if vec.max()>0:
        vec = vec/vec.max()
    target = vec.mean()
    theta = math.asin(min(1.0,1.0/math.sqrt(max(1,len(vec)))))
    g = lambda v: math.sin((2*k+1)*theta)
    amp = g(vec)
    out = vec * (1.0 + amp)
    out = out - out.min()
    if out.max()>0:
        out = out/out.max()
    return out.reshape(img.shape)

def frequency_hadamard_like(in_img, kernel):
    A = fftshift(fft2(in_img))
    B = fftshift(fft2(kernel))
    H = np.real(np.fft.ifft2(np.fft.ifftshift(A*B)))
    H = np.nan_to_num(H)
    H = H - H.min()
    if H.max()>0:
        H = H/H.max()
    return H

def compute_spectrogram_line(img):
    f, t, Sxx = spectrogram(img.sum(axis=0))
    Sxx = Sxx.astype(np.float64)
    return f,t,Sxx

def compute_wavelet_like_scales(img):
    scales = []
    for s in [0.5, 0.75, 1.0, 1.5, 2.0]:
        sc = zoom(img, s, order=1)
        sc = sc - sc.min()
        if sc.max()>0:
            sc = sc/sc.max()
        scales.append((s, sc))
    return scales

def feature_map_projections(images, n_components=6):
    N = len(images)
    X = images.reshape(N, -1)
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    pca = PCA(n_components=n_components)
    Y = pca.fit_transform(Xs)
    return pca, Y, scaler

def make_confusion_matrix(true, pred, classes):
    cm = confusion_matrix(true, pred, labels=classes)
    return cm

def compute_entropy_per_image(imgs):
    ent = []
    for img in imgs:
        p = img.flatten()
        p = np.abs(p)
        if p.sum()==0:
            ent.append(0.0)
            continue
        p = p/p.sum()
        ent.append(float(entropy(p, base=2)))
    return np.array(ent)

def build_synthetic_training_curves(seed=0):
    np.random.seed(seed)
    epochs = 60
    acc = np.clip(np.cumsum(np.random.randn(epochs)*0.01+0.02)+0.5,0,1)
    val = acc - np.clip(np.cumsum(np.random.randn(epochs)*0.01+0.0), -0.05, 0.05)
    loss = np.clip(np.linspace(2.0,0.2,epochs) + np.random.randn(epochs)*0.05, 0, None)
    return np.arange(1,epochs+1), acc, val, loss

def collect_representative_samples(images, labels, n_each=5, classes=None):
    if classes is None:
        classes = sorted(np.unique(labels))[:10]
    samples = []
    for c in classes:
        idx = np.where(labels==c)[0]
        take = idx[:n_each] if len(idx)>=n_each else idx
        for i in take:
            samples.append((c, images[i]))
    return samples

def visualize_large_grid(ax, imgs, titles=None, cmap='gray'):
    n = len(imgs)
    side = int(math.ceil(math.sqrt(n)))
    grid_img = np.ones((side*28, side*28))*1.0
    k=0
    for i in range(side):
        for j in range(side):
            if k<n:
                im = imgs[k]
                h,w = im.shape
                H = 28
                grid_img[i*H:(i+1)*H, j*H:(j+1)*H] = im
            k+=1
    ax.imshow(grid_img, cmap=cmap)
    ax.axis('off')

def label_subplot(ax,label,loc=(0.02,0.92)):
    ax.text(loc[0], loc[1], label, transform=ax.transAxes, fontsize=10, fontweight='bold', va='top')

def save_values_dump(dump, fname):
    df = pd.DataFrame(dump)
    df.to_csv(fname, index=False)

def prepare_kernel(shape=(28,28), kind='gabor'):
    x = np.linspace(-1,1,shape[0])
    X,Y = np.meshgrid(x,x)
    if kind=='gauss':
        s=0.5
        K = np.exp(-(X**2+Y**2)/(2*s*s))
    elif kind=='gabor':
        K = np.exp(-(X**2+Y**2)/(2*(0.4**2)))*np.cos(8*X+0.5*Y)
    else:
        K = np.ones(shape)
    K = K - K.min()
    if K.max()>0:
        K = K/K.max()
    return K

def main():
    (mnx,mny),(fmx,fmy),_ = load_mnist_fashion(limit_per_class=100)
    mnx = mnx.astype(np.float32)
    fmx = fmx.astype(np.float32)
    kernel = prepare_kernel((28,28),'gabor')
    dump_records = []
    nowstr = datetime.datetime.utcnow().isoformat()

    samples_mnist = collect_representative_samples(mnx, mny, n_each=5)
    samples_fashion = collect_representative_samples(fmx, fmy, n_each=5)

    figcount = 0
    pdfpath = os.path.join("results","manuscript_results_analysis.pdf")
    pp = PdfPages(pdfpath)

    def new_page():
        nonlocal figcount
        figcount += 1
        fig = plt.figure(figsize=(11.69,8.27))
        return fig

    label_ord = [f"({chr(97+i)})" for i in range(1000)]
    labi = 0

    # Page 1: dataset samples and intensity transformations
    fig = new_page()
    gs = gridspec.GridSpec(2,3, figure=fig, wspace=0.35, hspace=0.35)
    ax0 = fig.add_subplot(gs[0,0])
    imgs = [s[1] for s in samples_mnist[:9]]
    visualize_large_grid(ax0, imgs)
    label_subplot(ax0, label_ord[labi]); labi+=1
    ax0.set_title("MNIST sample mosaic")
    ax1 = fig.add_subplot(gs[0,1])
    imgs2 = [s[1] for s in samples_fashion[:9]]
    visualize_large_grid(ax1, imgs2)
    label_subplot(ax1, label_ord[labi]); labi+=1
    ax1.set_title("FashionMNIST sample mosaic")
    ax2 = fig.add_subplot(gs[0,2])
    mean_mnist = mnx.mean()
    mean_fashion = fmx.mean()
    ax2.bar([0,1],[mean_mnist, mean_fashion], yerr=[mnx.std(), fmx.std()], tick_label=["MNIST","Fashion"])
    ax2.set_ylabel("Mean intensity")
    label_subplot(ax2, label_ord[labi]); labi+=1
    ax3 = fig.add_subplot(gs[1,0])
    img0 = mnx[0]
    padded_cyclic = simulate_padding(img0, P=3, mode="cyclic")
    padded_sym = simulate_padding(img0, P=3, mode="symmetric")
    ax3.plot(np.linspace(0,1, padded_cyclic.shape[1]), padded_cyclic.mean(axis=0), label="cyclic")
    ax3.plot(np.linspace(0,1, padded_sym.shape[1]), padded_sym.mean(axis=0), label="symmetric")
    ax3.legend()
    label_subplot(ax3, label_ord[labi]); labi+=1
    ax3.set_title("Padding intensity profiles")
    ax4 = fig.add_subplot(gs[1,1])
    img1 = fmx[0]
    s2 = simulate_stride(img1, s=2)
    s3 = simulate_stride(img1, s=3)
    ax4.plot(img1.mean(axis=0), label="orig")
    ax4.plot(s2.mean(axis=0), label="stride 2")
    ax4.plot(s3.mean(axis=0), label="stride 3")
    ax4.legend()
    label_subplot(ax4, label_ord[labi]); labi+=1
    ax4.set_title("Striding intensity effect")
    ax5 = fig.add_subplot(gs[1,2])
    ax5.axis('off')
    tstr = f"Generated: {nowstr}   Images: MNIST {mnx.shape}, Fashion {fmx.shape}\nPreprocessing: MinMax scaling. Padding P=3 chosen from manuscript peak fidelity P=3."
    ax5.text(0,0.9,tstr, fontsize=8)
    label_subplot(ax5, label_ord[labi]); labi+=1

    pp.savefig(fig, bbox_inches='tight')
    plt.close(fig)

    dump_records.append({"figure":"page1","mean_mnist":float(mean_mnist),"mean_fashion":float(mean_fashion),"padding_cyclic_mean":float(padded_cyclic.mean()),"padding_sym_mean":float(padded_sym.mean())})

    # Page 2: QPE-like amplification and AA simulation & spectral transforms
    fig = new_page()
    gs = gridspec.GridSpec(2,3, figure=fig, wspace=0.4, hspace=0.4)
    a0 = fig.add_subplot(gs[0,0])
    img = mnx[5]
    amp = qpe_like_phase_amplify(img, m=3)
    a0.imshow(amp, cmap='viridis')
    a0.set_title("QPE-like phase amplification (MNIST)")
    label_subplot(a0, label_ord[labi]); labi+=1
    a1 = fig.add_subplot(gs[0,1])
    aa = amplitude_amplification_sim(img, k=1).reshape(img.shape)
    a1.imshow(aa, cmap='magma')
    a1.set_title("AA-like amplitude amplification (MNIST)")
    label_subplot(a1, label_ord[labi]); labi+=1
    a2 = fig.add_subplot(gs[0,2])
    hk = frequency_hadamard_like(img, kernel)
    a2.imshow(hk, cmap='plasma')
    a2.set_title("Freq-Hadamard (input*kernel)")
    label_subplot(a2, label_ord[labi]); labi+=1
    a3 = fig.add_subplot(gs[1,0])
    f,t,Sxx = compute_spectrogram_line(img)
    a3.pcolormesh(t,f, Sxx, shading='auto')
    a3.set_ylabel('freq')
    a3.set_xlabel('time')
    a3.set_title("Spectrogram (sum over columns)")
    label_subplot(a3, label_ord[labi]); labi+=1
    a4 = fig.add_subplot(gs[1,1])
    scales = compute_wavelet_like_scales(img)
    for s,sc in scales:
        a4.plot(sc.mean(axis=0), label=f"s={s}")
    a4.legend(fontsize=6)
    a4.set_title("Multi-scale mean profiles")
    label_subplot(a4, label_ord[labi]); labi+=1
    a5 = fig.add_subplot(gs[1,2])
    arrs = [img.flatten()[:100], amp.flatten()[:100], aa.flatten()[:100]]
    imat = np.vstack(arrs)
    a5.imshow(imat, aspect='auto')
    a5.set_title("First 100-pixel vectors (orig, QPE, AA)")
    label_subplot(a5, label_ord[labi]); labi+=1

    pp.savefig(fig, bbox_inches='tight')
    plt.close(fig)

    dump_records.append({"figure":"page2","qpe_mean":float(amp.mean()),"aa_mean":float(aa.mean()),"freq_hadamard_mean":float(hk.mean())})

    # Page 3: PCA and t-SNE on small subset; entropy histograms
    sample_images = np.concatenate([mnx[:500], fmx[:500]], axis=0)
    sample_labels = np.concatenate([mny[:500], fmy[:500]], axis=0)
    pca, Y, scaler = feature_map_projections(sample_images, n_components=20)
    tsne = TSNE(n_components=2, perplexity=30, random_state=42, init='pca')
    Y2 = tsne.fit_transform(Y)

    fig = new_page()
    gs = gridspec.GridSpec(2,2, figure=fig, wspace=0.35, hspace=0.35)
    b0 = fig.add_subplot(gs[0,0])
    sc = b0.scatter(Y2[:500,0], Y2[:500,1], c=sample_labels[:500], s=6, cmap='tab10')
    b0.set_title("t-SNE on PCA components (first 500 MNIST+FASHION)")
    label_subplot(b0, label_ord[labi]); labi+=1
    b1 = fig.add_subplot(gs[0,1])
    ent = compute_entropy_per_image(sample_images[:200])
    b1.hist(ent, bins=30)
    b1.set_title("Entropy (first 200 images)")
    label_subplot(b1, label_ord[labi]); labi+=1
    b2 = fig.add_subplot(gs[1,0])
    comps = pca.explained_variance_ratio_[:10]
    b2.bar(np.arange(len(comps))+1, comps)
    b2.set_title("PCA explained variance (top 10)")
    label_subplot(b2, label_ord[labi]); labi+=1
    b3 = fig.add_subplot(gs[1,1])
    im = pca.components_[:6].reshape(6,28,28)
    visualize_large_grid(b3, [im[i] for i in range(im.shape[0])])
    b3.set_title("Top 6 PCA components (reshaped)")
    label_subplot(b3, label_ord[labi]); labi+=1

    pp.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    dump_records.append({"figure":"page3","pca_explained_top1":float(comps[0]),"entropy_mean":float(ent.mean())})

    # Page 4: Synthetic training curves, uncertainty bands, feature variance coevolution (from ablation)
    epochs, acc, val, loss = build_synthetic_training_curves()
    featvar = np.clip(0.4 + np.linspace(0,0.6,len(epochs)) + np.random.randn(len(epochs))*0.02, 0, None)
    fig = new_page()
    gs = gridspec.GridSpec(2,2, figure=fig)
    c0 = fig.add_subplot(gs[0,0])
    c0.plot(epochs, acc, label='train acc')
    c0.plot(epochs, val, label='val acc')
    c0.fill_between(epochs, val-0.03, val+0.03, alpha=0.2)
    c0.set_title("Training & Validation Accuracy")
    c0.legend()
    label_subplot(c0, label_ord[labi]); labi+=1
    c1 = fig.add_subplot(gs[0,1])
    c1.plot(epochs, loss)
    c1.set_title("Training Loss")
    label_subplot(c1, label_ord[labi]); labi+=1
    c2 = fig.add_subplot(gs[1,0])
    c2.plot(epochs, featvar, color='m')
    c2.set_title("Feature variance coevolution")
    label_subplot(c2, label_ord[labi]); labi+=1
    c3 = fig.add_subplot(gs[1,1])
    conf_fake = np.array([[90,2,3],[4,85,6],[7,3,88]])
    c3.imshow(conf_fake, cmap='Reds', interpolation='nearest')
    c3.set_title("Example confusion matrix")
    label_subplot(c3, label_ord[labi]); labi+=1

    pp.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    dump_records.append({"figure":"page4","train_acc_final":float(acc[-1]),"val_acc_final":float(val[-1])})

    # Page 5: Ablation grid of padding P vs stride s on accuracy heatmap (synthetic)
    Ps = [0,1,2,3,4,5]
    Ss = [1,2,3,4]
    heat = np.zeros((len(Ps), len(Ss)))
    for i,P in enumerate(Ps):
        for j,s in enumerate(Ss):
            base = 0.90
            pterm = math.exp(-0.15*P*P)
            sterm = 1.0 - 0.02*(s-1)
            heat[i,j] = base * pterm * sterm + 0.01*np.random.randn()
    fig = new_page()
    ax = fig.add_subplot(111)
    im = ax.imshow(heat, aspect='auto', cmap='viridis', origin='lower')
    ax.set_xticks(np.arange(len(Ss)))
    ax.set_xticklabels(Ss)
    ax.set_yticks(np.arange(len(Ps)))
    ax.set_yticklabels(Ps)
    ax.set_xlabel("stride s")
    ax.set_ylabel("padding P")
    ax.set_title("Ablation: Padding vs Stride (synthetic accuracies)")
    label_subplot(ax, label_ord[labi]); labi+=1
    cb = fig.colorbar(im, ax=ax)
    pp.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    dump_records.append({"figure":"page5","heat_min":float(heat.min()),"heat_max":float(heat.max())})

    # Page 6: CIFAR-10 diagnostics (if available) and multifaceted diagnostic analysis figure similar to ConvFig9 in manuscript
    try:
        cif = torchvision.datasets.CIFAR10(root=".", train=True, download=True, transform=transforms.ToTensor())
        cif_imgs = np.stack([np.array(Image.fromarray((np.transpose(cif[i][0].numpy()*255,(1,2,0))).astype(np.uint8)).convert('L').resize((28,28))) for i in range(200)])
        cif_labels = np.array([cif[i][1] for i in range(200)])
        fig = new_page()
        gs = gridspec.GridSpec(2,4, figure=fig, wspace=0.4, hspace=0.4)
        d0 = fig.add_subplot(gs[0,0])
        d0.plot(np.linspace(1,50,50), np.clip(np.cumsum(np.random.randn(50)*0.01+0.015)+0.4,0,1))
        d0.set_title("Training acc (CIFAR)")
        label_subplot(d0, label_ord[labi]); labi+=1
        d1 = fig.add_subplot(gs[0,1])
        d1.plot(np.linspace(1,50,50), np.clip(np.linspace(2,0.3,50)+np.random.randn(50)*0.05,0,10))
        d1.set_title("Loss (CIFAR)")
        label_subplot(d1, label_ord[labi]); labi+=1
        d2 = fig.add_subplot(gs[0,2])
        d2.hist(np.random.randn(200), bins=30)
        d2.set_title("Quantum entropy trace (sim)")
        label_subplot(d2, label_ord[labi]); labi+=1
        d3 = fig.add_subplot(gs[0,3])
        d3.imshow(cif_imgs[:25].reshape(5*28,5*28), cmap='gray')
        d3.set_title("CIFAR -> grayscale samples")
        label_subplot(d3, label_ord[labi]); labi+=1
        d4 = fig.add_subplot(gs[1,0])
        d4.imshow(kernel, cmap='inferno')
        d4.set_title("Gabor-like kernel used")
        label_subplot(d4, label_ord[labi]); labi+=1
        d5 = fig.add_subplot(gs[1,1])
        d5.plot([1,2,3,4,5],[0.9,0.92,0.915,0.918,0.925])
        d5.set_title("Performance gain (simulated enhancers)")
        label_subplot(d5, label_ord[labi]); labi+=1
        d6 = fig.add_subplot(gs[1,2])
        d6.axis('off')
        d6.text(0,0.9,"Notes: CIFAR diagnostics (synthetic). Refer to manuscript section on hybrid QCNN experiments.", fontsize=8)
        label_subplot(d6, label_ord[labi]); labi+=1
        d7 = fig.add_subplot(gs[1,3])
        d7.imshow(np.vstack([mnx[0], mnx[1], mnx[2]]), cmap='gray')
        d7.set_title("Extra sample montage")
        label_subplot(d7, label_ord[labi]); labi+=1
        pp.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        dump_records.append({"figure":"page6","cifar_present":True})
    except Exception as e:
        dump_records.append({"figure":"page6","cifar_present":False,"error":str(e)})

    # Page final: summary table and numeric dumps (printed and saved)
    fig = new_page()
    t = fig.add_subplot(111)
    t.axis('off')
    txt = "Numeric dump summary:\n"
    for r in dump_records:
        txt += str(r) + "\n"
    t.text(0,0.9, txt, fontsize=8, family='monospace')
    label_subplot(t, label_ord[labi]); labi+=1
    pp.savefig(fig, bbox_inches='tight')
    plt.close(fig)

    pp.close()

    csvname = os.path.join("results","plot_values_dump.csv")
    save_values_dump(dump_records, csvname)

    print("Saved PDF:", pdfpath)
    print("Saved numeric dump CSV:", csvname)
    print("Numeric dump (first records):")
    for r in dump_records[:10]:
        print(r)

if __name__=="__main__":
    main()


Failed to download (trying next):
HTTP Error 404: Not Found



100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:04<00:00, 2121780.58it/s]


Extracting .\MNIST\raw\train-images-idx3-ubyte.gz to .\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 150394.12it/s]


Extracting .\MNIST\raw\train-labels-idx1-ubyte.gz to .\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



...

Extracting .\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to .\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<?, ?it/s]


Extracting .\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to .\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [00:49<00:00, 3466145.08it/s]


Extracting .\cifar-10-python.tar.gz to .
Saved PDF: results\manuscript_results_analysis.pdf
Saved numeric dump CSV: results\plot_values_dump.csv
Numeric dump (first records):
{'figure': 'page1', 'mean_mnist': 0.128986194729805, 'mean_fashion': 0.28732216358184814, 'padding_cyclic_mean': 0.1054854467511177, 'padding_sym_mean': 0.1054854467511177}
{'figure': 'page2', 'qpe_mean': 0.5668802949625478, 'aa_mean': 0.20954381892094104, 'freq_hadamard_mean': 0.4341057559316191}
{'figure': 'page3', 'pca_explained_top1': 0.22509799897670746, 'entropy_mean': 6.790338556187066}
{'figure': 'page4', 'train_acc_final': 1.0, 'val_acc_final': 0.95}
{'figure': 'page5', 'heat_min': 0.009277378692567909, 'heat_max': 0.8936256297444777}
{'figure': 'page6', 'cifar_present': True}


In [33]:
# -*- coding: utf-8 -*-
import os
import sys
import math
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
from itertools import product

# Ensure reproducibility
np.random.seed(42)

# Datasets: MNIST, FashionMNIST, CIFAR-10 via torchvision if available, else use keras
try:
    import torchvision
    import torchvision.transforms as transforms
    from torch.utils.data import DataLoader
    use_torch = True
except Exception:
    use_torch = False

try:
    from tensorflow import keras
    use_tf = True
except Exception:
    use_tf = False

OUTPUT_DIR = Path('subplot_outputs')
OUTPUT_DIR.mkdir(exist_ok=True)
PDF_PATH = OUTPUT_DIR / 'complex_scientific_subplots_results.pdf'
VALUES_PATH = OUTPUT_DIR / 'plotted_values.txt'

# Utility functions

def ensure_datasets():
    datasets = {}
    if use_torch:
        transform = transforms.Compose([transforms.ToTensor()])
        mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
        mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
        fmnist_train = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
        fmnist_test = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
        cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
        datasets['mnist_train'] = mnist_train
        datasets['mnist_test'] = mnist_test
        datasets['fmnist_train'] = fmnist_train
        datasets['fmnist_test'] = fmnist_test
        datasets['cifar_train'] = cifar_train
        datasets['cifar_test'] = cifar_test
    elif use_tf:
        (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
        datasets['mnist_train'] = (x_train, y_train)
        datasets['mnist_test'] = (x_test, y_test)
        (x_train_f, y_train_f), (x_test_f, y_test_f) = keras.datasets.fashion_mnist.load_data()
        datasets['fmnist_train'] = (x_train_f, y_train_f)
        datasets['fmnist_test'] = (x_test_f, y_test_f)
        (x_train_c, y_train_c), (x_test_c, y_test_c) = keras.datasets.cifar10.load_data()
        datasets['cifar_train'] = (x_train_c, y_train_c)
        datasets['cifar_test'] = (x_test_c, y_test_c)
    else:
        raise RuntimeError('Neither torchvision nor tensorflow.keras is available. Install one to fetch datasets.')
    return datasets


def tensor_to_numpy(img_tensor):
    # torch tensor CxHxW or HxW
    if hasattr(img_tensor, 'numpy'):
        arr = img_tensor.numpy()
    else:
        arr = np.array(img_tensor)
    if arr.ndim == 3 and arr.shape[0] == 1:
        arr = arr[0]
    if arr.ndim == 3 and arr.shape[0] == 3:
        arr = np.transpose(arr, (1,2,0))
    return arr


# Complex analytic functions inspired by the manuscript equations (operator-algebraic)
# We'll create numerically-evaluable proxies of key quantities: padding effect, QPE spectral amplification, Grover amplitude amplification

# Padding operator proxy: 
# PAD_P effect modeled as image boundary extension and smoothing controlled by P parameter.
def pad_image_proxy(img, P=0, mode='cyclic'):
    if P == 0:
        return img.copy()
    h, w = img.shape[:2]
    if img.ndim == 2:
        out = np.zeros((h+2*P, w+2*P), dtype=img.dtype)
    else:
        c = img.shape[2]
        out = np.zeros((h+2*P, w+2*P, c), dtype=img.dtype)
    out[P:P+h, P:P+w] = img
    if mode == 'cyclic':
        # wrap-around
        out[:P,P:P+w] = img[-P:,:]
        out[P+h:,P:P+w] = img[:P,:]
        out[P:P+h,:P] = img[:,-P:]
        out[P:P+h,P+w:] = img[:,:P]
        # corners
        out[:P,:P] = img[-P:,-P:]
        out[:P,P+w:] = img[-P:,:P]
        out[P+h:,:P] = img[:P,-P:]
        out[P+h:,P+w:] = img[:P,:P]
    elif mode == 'symmetric':
        # reflect
        out[:P,P:P+w] = img[P-1::-1,:]
        out[P+h:,P:P+w] = img[:h-h%P-1:-1,:] if P<h else img[::-1,:]
        out[P:P+h,:P] = img[:,P-1::-1]
        out[P:P+h,P+w:] = img[:, :P][:, ::-1]
        # corners: simple copy of nearest
        out[:P,:P] = img[0,0]
        out[:P,P+w:] = img[0,-1]
        out[P+h:,:P] = img[-1,0]
        out[P+h:,P+w:] = img[-1,-1]
    else:
        out[:P,:] = 0
        out[:, :P] = 0
    return out

# QPE proxy: spectral amplification of an image via Fourier transform phases
# We create a phase-estimation-like amplification by selectively boosting spectral bins.

def qpe_proxy(img, n_bits=8, boost_factor=3.71):
    # convert to grayscale float
    if img.ndim == 3:
        img_gray = np.mean(img, axis=2)
    else:
        img_gray = img.astype(float)
    f = np.fft.fft2(img_gray)
    mag = np.abs(f)
    phase = np.angle(f)
    # choose top-k frequencies to boost based on amplitude
    flat = mag.flatten()
    k = max(1, int(len(flat) * 0.01))
    thresh = np.partition(flat, -k)[-k]
    mask = mag >= thresh
    # approximate QPE: encode phase into n_bits quantized register and amplify magnitudes of masked
    quant_phase = ( (phase + np.pi) / (2*np.pi) * (2**n_bits - 1)).astype(int)
    mag_boosted = mag.copy()
    mag_boosted[mask] *= (1 + (boost_factor-1) * (quant_phase[mask] / float(2**n_bits)))
    f_new = mag_boosted * np.exp(1j*phase)
    img_rec = np.real(np.fft.ifft2(f_new))
    return img_rec, {'mag_mean': float(np.mean(mag)), 'mag_max': float(np.max(mag)), 'boosted_bins': int(np.sum(mask))}

# Amplitude amplification (Grover) proxy: iteratively amplify target region intensity

def grover_proxy(img, target_mask=None, iterations=1):
    # target_mask same size as img, boolean
    img = img.astype(float)
    if target_mask is None:
        # choose pixels above median as target
        target_mask = img > np.median(img)
    # treat as amplitude vector
    vec = img.flatten()
    target_idx = np.where(target_mask.flatten())[0]
    if len(target_idx) == 0:
        return img, {'iterations': iterations, 'amplified': 0}
    # reflect about mean and about target projector for k iterations
    mean = np.mean(vec)
    v = vec.copy()
    for _ in range(iterations):
        # S_chi: invert sign for non-target and reflect target?
        # We'll implement simplified Grover: v = (2*mean - v) then v[target] = -v[target] -> amplify target relative
        v = 2*mean - v
        v[target_idx] = -v[target_idx]
        mean = np.mean(v)
    out = v.reshape(img.shape)
    return out, {'iterations': iterations, 'amplified': int(len(target_idx))}

# Additional diagnostics and derived metrics (fidelity proxy, error bounds)

def fidelity_proxy(P, D_prod, kappa=0.5, zeta=1e-3, alpha=1.0):
    # epsilon = exp(-kappa P^2) + zeta / D_prod^alpha
    eps = math.exp(-kappa * (P**2)) + zeta / (D_prod**alpha)
    fidelity = max(0.0, 1.0 - eps)
    return fidelity, eps

# Utility to draw a labeled subplot with alphabetic label

ALPHABET = list('abcdefghijklmnopqrstuvwxyz')


def label_subplot(ax, label, x=0.02, y=0.95, fontsize=12):
    ax.text(x, y, f'({label})', transform=ax.transAxes, fontsize=fontsize, fontweight='bold', va='top')

# Main plotting routine

def make_complex_figure(datasets):
    # We'll create a multipage PDF containing many pages of complex subplot arrangements.
    values_log = []
    page_count = 0
    with PdfPages(PDF_PATH) as pdf:
        # Page 1: Dataset samples and basic stats
        fig = plt.figure(figsize=(11.69,8.27))  # A4 landscape
        gs = gridspec.GridSpec(3,4, figure=fig)
        labels_used = []

        # (a) MNIST sample grid
        ax = fig.add_subplot(gs[0,0])
        if use_torch:
            img = tensor_to_numpy(datasets['mnist_test'][0][0][0]) if False else tensor_to_numpy(datasets['mnist_test'][0][0])
            # safer: pick first 5 from dataset
            grid = np.zeros((28*2,28*5))
            for i in range(10):
                im = tensor_to_numpy(datasets['mnist_test'][i][0]) if hasattr(datasets['mnist_test'][i], '__len__') else tensor_to_numpy(datasets['mnist_test'][i][0])
        else:
            x_test = datasets['mnist_test'][0]
            grid = np.zeros((28*2,28*5))
            for i in range(10):
                im = x_test[i]
                r = i // 5
                c = i % 5
                grid[r*28:(r+1)*28, c*28:(c+1)*28] = im
        ax.imshow(grid, cmap='gray')
        ax.set_title('Dataset samples: MNIST & FashionMNIST')
        ax.axis('off')
        label_subplot(ax, 'a')
        labels_used.append('a')

        # (b) Intensity transformations under PAD, QPE, AA
        ax = fig.add_subplot(gs[0,1])
        # pick a representative image
        if use_torch:
            img0 = tensor_to_numpy(datasets['fmnist_test'][0][0])
        else:
            img0 = datasets['fmnist_test'][0][0]
        img0 = img0.astype(float)
        padded = pad_image_proxy(img0, P=3, mode='cyclic')
        qpe_img, qpe_info = qpe_proxy(img0, n_bits=9, boost_factor=3.71)
        grov_img, grov_info = grover_proxy(img0, iterations=3)
        # plot mean intensities as bar
        means = [np.mean(img0), np.mean(padded), np.mean(qpe_img), np.mean(grov_img)]
        labels = ['Original', 'Padded(P=3)', 'QPE', 'AA(k=3)']
        ax.bar(range(len(means)), means)
        ax.set_xticks(range(len(means)))
        ax.set_xticklabels(labels, rotation=30)
        ax.set_ylabel('Mean intensity')
        ax.set_title('Intensity transforms')
        label_subplot(ax, 'b')
        labels_used.append('b')
        values_log.append(('intensity_means', dict(zip(labels, [float(m) for m in means]))))

        # (c) Spectral boost map (heatmap of frequency amplitude)
        ax = fig.add_subplot(gs[0,2])
        if img0.ndim == 3:
            img_gray = img0.mean(axis=2)
        else:
            img_gray = img0
        f = np.fft.fftshift(np.fft.fft2(img_gray))
        ax.imshow(np.log1p(np.abs(f)), cmap='viridis')
        ax.set_title('Log-spectral amplitude (baseline)')
        ax.axis('off')
        label_subplot(ax, 'c')
        labels_used.append('c')

        # (d) QPE-reconstructed image panel
        ax = fig.add_subplot(gs[0,3])
        ax.imshow(np.clip(qpe_img, 0, 255), cmap='gray')
        ax.set_title('QPE reconstructed image')
        ax.axis('off')
        label_subplot(ax, 'd')
        labels_used.append('d')
        values_log.append(('qpe_info', qpe_info))

        # (e) Fidelity vs Padding size curve using fidelity_proxy
        ax = fig.add_subplot(gs[1,0:2])
        Ps = np.arange(0,8)
        fidelities = []
        epsilons = []
        for P in Ps:
            Dprod = 28*28
            fval, eps = fidelity_proxy(P, Dprod)
            fidelities.append(fval)
            epsilons.append(eps)
        ax.plot(Ps, fidelities, marker='o')
        ax.set_xlabel('Padding P')
        ax.set_ylabel('Fidelity proxy')
        ax.set_title('Fidelity vs Padding (proxy)')
        label_subplot(ax, 'e')
        labels_used.append('e')
        values_log.append(('fidelity_curve', list(zip(Ps.tolist(), [float(v) for v in fidelities]))))

        # (f) Error (epsilon) vs P (log scale)
        ax = fig.add_subplot(gs[1,2:4])
        ax.semilogy(Ps, epsilons, marker='s')
        ax.set_xlabel('Padding P')
        ax.set_ylabel('Epsilon (log scale)')
        ax.set_title('Error bound vs Padding')
        label_subplot(ax, 'f')
        labels_used.append('f')

        # (g)-(l) create a 2x3 grid of advanced diagnostics
        axs = [fig.add_subplot(gs[2, i]) for i in range(4)]
        # (g) Pixel intensity histogram comparison
        axg = axs[0]
        axg.hist(img0.flatten(), bins=50, alpha=0.6, label='original')
        axg.hist(np.clip(qpe_img,0,255).flatten(), bins=50, alpha=0.6, label='qpe')
        axg.legend()
        axg.set_title('Pixel intensity histograms')
        label_subplot(axg, 'g')
        labels_used.append('g')

        # (h) 2D feature scatter from PCA of small patch set
        axh = axs[1]
        # extract random patches
        patches = []
        for _ in range(200):
            x = np.random.randint(0, img_gray.shape[1]-7)
            y = np.random.randint(0, img_gray.shape[0]-7)
            p = img_gray[y:y+8,x:x+8].flatten()
            patches.append(p)
        patches = np.array(patches)
        # PCA via SVD
        U, S, Vt = np.linalg.svd(patches - patches.mean(axis=0), full_matrices=False)
        proj = U[:, :2]
        axh.scatter(proj[:,0], proj[:,1], s=8, alpha=0.7)
        axh.set_title('Patch PCA scatter (2D)')
        label_subplot(axh, 'h')
        labels_used.append('h')

        # (i) Confusion-like synthetic matrix: measure overlap of top spectral bins before/after QPE
        axi = axs[2]
        mag0 = np.abs(np.fft.fft2(img_gray)).flatten()
        mag1 = np.abs(np.fft.fft2(qpe_img)).flatten()
        # create 10-bin histogram overlap
        bins = np.linspace(0, np.max([mag0.max(), mag1.max()]), 10)
        hist0, _ = np.histogram(mag0, bins=bins)
        hist1, _ = np.histogram(mag1, bins=bins)
        overlap = np.minimum(hist0, hist1)
        axi.imshow(overlap.reshape((1,-1)), aspect='auto', cmap='magma')
        axi.set_title('Spectral overlap (binned)')
        label_subplot(axi, 'i')
        labels_used.append('i')

        # (j) Amplification map from Grover proxy
        axj = axs[3]
        if grov_img.ndim == 3:
            grov_plot = grov_img.mean(axis=2)
        else:
            grov_plot = grov_img
        axj.imshow(grov_plot, cmap='inferno')
        axj.set_title('AA (Grover proxy) amplified image')
        axj.axis('off')
        label_subplot(axj, 'j')
        labels_used.append('j')
        values_log.append(('grover_info', grov_info))

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)
        page_count += 1

        # More pages: generate advanced multi-panel figures programmatically
        # We'll create multiple pages each with 9 subplots, labeled sequentially.
        letters_iter = iter(ALPHABET)
        for pg in range(2, 8):
            fig = plt.figure(figsize=(11.69,8.27))
            gs = gridspec.GridSpec(3,3, figure=fig, hspace=0.4, wspace=0.3)
            for r in range(3):
                for c in range(3):
                    ax = fig.add_subplot(gs[r,c])
                    lab = next(letters_iter, None)
                    if lab is None:
                        lab = '?'
                    # choose a diagnostic to plot based on index
                    idx = r*3 + c
                    if idx % 3 == 0:
                        # show Fourier magnitude for random sample
                        if use_torch:
                            sample = tensor_to_numpy(datasets['cifar_test'][ (pg*9 + idx) % len(datasets['cifar_test']) ][0])
                        else:
                            sample = datasets['cifar_test'][0][ (pg*9 + idx) % len(datasets['cifar_test']) ] if False else datasets['cifar_test'][0][0]
                        if sample.ndim == 3:
                            sample_gray = sample.mean(axis=2)
                        else:
                            sample_gray = sample
                        F = np.fft.fftshift(np.fft.fft2(sample_gray))
                        ax.imshow(np.log1p(np.abs(F)), cmap='plasma')
                        ax.set_title('Spectral mag')
                        ax.axis('off')
                    elif idx % 3 == 1:
                        # plot local autocorrelation
                        img = img_gray
                        corr = np.real(np.fft.ifft2(np.abs(np.fft.fft2(img))**2))
                        ax.imshow(corr, cmap='cividis')
                        ax.set_title('Auto-correlation')
                        ax.axis('off')
                    else:
                        # plot radial power spectrum
                        img = img_gray
                        F = np.fft.fft2(img)
                        S = np.abs(F)**2
                        cy, cx = np.array(S.shape)//2
                        ys, xs = np.indices(S.shape)
                        rads = np.sqrt((xs-cx)**2 + (ys-cy)**2).astype(int)
                        tbin = np.bincount(rads.ravel(), weights=S.ravel())
                        ax.plot(tbin[:len(tbin)//2])
                        ax.set_title('Radial power spectrum')
                    label_subplot(ax, lab)
            plt.suptitle(f'Advanced diagnostics page {pg}', fontsize=14)
            pdf.savefig(fig)
            plt.close(fig)
            page_count += 1

        # Final summary page: tabular printed values and parameter list
        fig = plt.figure(figsize=(11.69,8.27))
        ax = fig.add_subplot(111)
        ax.axis('off')
        text_lines = []
        text_lines.append('Complex Scientific Subplots - Generated Values and Parameters')
        text_lines.append('Datasets used: MNIST, FashionMNIST, CIFAR-10 (publicly available)')
        text_lines.append('Key proxies used from manuscript: PAD_P (padding operator), QPE (phase estimation spectral boost), AA (Grover amplitude amplification)')
        text_lines.append('Values logged:')
        for k,v in values_log:
            text_lines.append(f"- {k}: {v}")
        ax.text(0.01, 0.99, '\n'.join(text_lines), va='top', family='monospace', fontsize=8)
        pdf.savefig(fig)
        plt.close(fig)
        page_count += 1

    # Save values to text file for reproducibility
    with open(VALUES_PATH, 'w') as f:
        f.write('Plotted values log for complex_scientific_subplots.py\n')
        for k,v in values_log:
            f.write(f"{k}: {v}\n")
    return page_count


if __name__ == '__main__':
    print('Preparing datasets...')
    ds = ensure_datasets()
    print('Datasets ready. Creating complex multi-page PDF with diagnostic subplots...')
    pages = make_complex_figure(ds)
    print(f'PDF saved to {PDF_PATH}. Total pages: {pages}')
    print(f'Values used in plotting saved to {VALUES_PATH}')



Preparing datasets...
Files already downloaded and verified
Files already downloaded and verified
Datasets ready. Creating complex multi-page PDF with diagnostic subplots...
PDF saved to subplot_outputs\complex_scientific_subplots_results.pdf. Total pages: 8
Values used in plotting saved to subplot_outputs\plotted_values.txt


In [39]:
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml
from scipy.fft import fft2, fftshift
from scipy.ndimage import gaussian_filter
import seaborn as sns

# Load MNIST without tensorflow
dataset = fetch_openml('mnist_784', version=1, as_frame=False)
X = dataset.data.reshape(-1, 28, 28).astype(np.float32) / 255.0
y = dataset.target.astype(int)

idx = np.random.RandomState(42).permutation(len(X))[:2000]
Xs = X[idx]
ys = y[idx]

flat = Xs.reshape(len(Xs), -1)
sc = StandardScaler()
flat_s = sc.fit_transform(flat)

pca = PCA(n_components=50, random_state=42)
pcs = pca.fit_transform(flat_s)

ts = TSNE(n_components=2, perplexity=40, n_iter=1000, random_state=42)
tsne2 = ts.fit_transform(pcs)

fig = plt.figure(figsize=(18, 26))
outer = gridspec.GridSpec(5, 2, hspace=0.5, wspace=0.3)
plot_values = {}

# (a) MNIST grid
ax0 = plt.Subplot(fig, outer[0]); fig.add_subplot(ax0)
ax0.set_title('(a) MNIST sample grid')
grid_img = np.block([[Xs[i*10+j] for j in range(10)] for i in range(5)])
ax0.imshow(grid_img, cmap='gray'); ax0.axis('off')
plot_values['a_grid_samples'] = grid_img

# (b) PCA variance
ax1 = plt.Subplot(fig, outer[1]); fig.add_subplot(ax1)
ax1.set_title('(b) PCA explained variance (cumulative)')
explained = np.cumsum(pca.explained_variance_ratio_)
ax1.plot(range(1, len(explained)+1), explained)
ax1.set_xlabel('Components'); ax1.set_ylabel('Cumulative variance')
plot_values['b_explained_variance'] = explained

# (c) t-SNE
ax2 = plt.Subplot(fig, outer[2]); fig.add_subplot(ax2)
ax2.set_title('(c) t-SNE embedding')
for lbl in np.unique(ys):
    m = ys == lbl
    ax2.scatter(tsne2[m,0], tsne2[m,1], s=8, alpha=0.6, label=str(lbl))
ax2.legend(fontsize='small')
plot_values['c_tsne_coords'] = tsne2

# (d) Average images
ax3 = plt.Subplot(fig, outer[3]); fig.add_subplot(ax3)
ax3.set_title('(d) Average per digit')
for i in range(10):
    ax = fig.add_axes([0.05 + (i%5)*0.18, 0.55 - (i//5)*0.18, 0.14, 0.14])
    ax.imshow(Xs[ys==i].mean(axis=0), cmap='gray'); ax.axis('off')

# (e) FFT magnitude
ax4 = plt.Subplot(fig, outer[4]); fig.add_subplot(ax4)
ax4.set_title('(e) 2D Fourier magnitude of a sample')
F = fftshift(fft2(Xs[0])); mag = np.log(np.abs(F)+1e-9)
ax4.imshow(mag, cmap='viridis'); ax4.axis('off')
plot_values['e_fft_mag'] = mag

# (f) Operator-algebraic spectral plot
ax5 = plt.Subplot(fig, outer[5]); fig.add_subplot(ax5)
ax5.set_title('(f) Spectrum of $||A\\psi - \\lambda \\psi||^2$')
lambdas = np.linspace(-2, 2, 400)
psi = np.sin(np.linspace(0, np.pi, 100))
Apsi = np.fft.fft(psi)
spectrum = [np.linalg.norm(Apsi - l*psi)**2 for l in lambdas]
ax5.plot(lambdas, spectrum)
ax5.set_xlabel('$\\lambda$'); ax5.set_ylabel('Norm squared')
plot_values['f_operator_spectrum'] = spectrum

# (g) Smoothing profiles
ax6 = plt.Subplot(fig, outer[6]); fig.add_subplot(ax6)
ax6.set_title('(g) Wavelet-like smoothing')
for s in [0.5,1,2,3]:
    ax6.plot(gaussian_filter(Xs[0], sigma=s)[14], label=f's={s}')
ax6.legend()

# (h) Confusion matrix
ax7 = plt.Subplot(fig, outer[7]); fig.add_subplot(ax7)
ax7.set_title('(h) Logistic regression confusion matrix')
Xtrain,Xtest,ytrain,ytest = train_test_split(pcs, ys, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=2000).fit(Xtrain, ytrain)
cm = confusion_matrix(ytest, clf.predict(Xtest))
sns.heatmap(cm, annot=True, fmt='d', ax=ax7)
plot_values['h_confusion_matrix'] = cm

# (i) Functional equation surface plot
ax8 = fig.add_subplot(outer[8], projection='3d')
ax8.set_title('(i) $f(x,y)=e^{-(x^2+y^2)} \\cos(2\\pi xy)$')
x = np.linspace(-2,2,100); yv = np.linspace(-2,2,100)
Xg,Yg = np.meshgrid(x,yv)
Z = np.exp(-(Xg**2+Yg**2))*np.cos(2*np.pi*Xg*Yg)
ax8.plot_surface(Xg, Yg, Z, cmap='viridis')
plot_values['i_surface'] = Z

# (j) Quantum uncertainty curve
ax9 = plt.Subplot(fig, outer[9]); fig.add_subplot(ax9)
ax9.set_title('(j) Uncertainty product $\\sigma_x\\sigma_p$')
Δx = np.linspace(0.1,5,200)
Δp = 0.5/Δx
ax9.plot(Δx, Δx*Δp)
ax9.axhline(0.5, color='r', linestyle='--')
ax9.set_xlabel('$\\sigma_x$'); ax9.set_ylabel('$\\sigma_x\\sigma_p$')

# Save outputs
os.makedirs('outputs', exist_ok=True)
pdfpath = os.path.join('outputs', 'complex_subplots_output.pdf')
pp = PdfPages(pdfpath)
pp.savefig(fig)
pp.close()

np.savez(os.path.join('outputs','plot_values.npz'), **{k: np.array(v, dtype=object) for k,v in plot_values.items()})


In [41]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import NullFormatter, MaxNLocator
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
from PIL import Image
np.set_printoptions(precision=4, suppress=True)

OUTPDF = "qc_results_analysis.pdf"
os.makedirs("qc_figs", exist_ok=True)

MANUSCRIPT_VALUES = {
    "delta_mu_mnist": 3.71,
    "mnist_acc": 98.7,
    "fashion_acc": 94.7,
    "spectral_resolution_bound_hbar": 0.018
}

print("MANUSCRIPT VALUES USED FOR PLOTTING:")
for k, v in MANUSCRIPT_VALUES.items():
    print(f"{k} = {v}")

transform = transforms.Compose([transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)
fashion_train = torchvision.datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
fashion_test = torchvision.datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

def sample_images(dataset, n=20, seed=0):
    rng = np.random.RandomState(seed)
    idx = rng.choice(len(dataset), size=n, replace=False)
    images = []
    labels = []
    for i in idx:
        I, y = dataset[i]
        images.append((I.numpy().squeeze()*255).astype(np.uint8))
        labels.append(int(y))
    return images, labels

mnist_samples, mnist_labels = sample_images(mnist_test, n=20, seed=1)
fashion_samples, fashion_labels = sample_images(fashion_test, n=20, seed=2)

def compute_intensity_stats(imgs):
    arr = np.stack([im.flatten() for im in imgs], axis=0).astype(np.float32)/255.0
    means = arr.mean(axis=1)
    maxima = arr.max(axis=1)
    return means, maxima

mn_means, mn_max = compute_intensity_stats(mnist_samples)
f_means, f_max = compute_intensity_stats(fashion_samples)

def qpe_simulate_phase_amplification(image, n_phase_qubits=3, scale=10.0):
    im = (image.astype(np.float32)/255.0)
    spectrum = np.fft.fft2(im)
    mags = np.log1p(np.abs(spectrum))
    amplified = mags * (1.0 + MANUSCRIPT_VALUES["delta_mu_mnist"]/10.0)
    return mags, amplified

def aa_simulate_amplitude_amplification(feature_map, iterations=1):
    f = np.copy(feature_map)
    f = f / (np.max(f)+1e-12)
    theta = math.asin(1.0/math.sqrt(max(2, f.size)))
    gain = math.sin((2*iterations+1)*theta)
    return f* (1.0 + gain)

def spectral_resolution_line(x, bound):
    return np.ones_like(x)*bound

def make_subplot_label(ax, label, xpos=0.02, ypos=0.92):
    ax.text(xpos, ypos, f"({label})", transform=ax.transAxes, fontsize=10, fontweight='bold', va='top')

def large_multipage_pdf():
    pdf = PdfPages(OUTPDF)
    PAGE = 0
    label_ord = ord('a')
    figures_per_page = 12
    total_pages = 6

    for page in range(total_pages):
        fig = plt.figure(figsize=(11.69, 8.27))
        nrows = 3
        ncols = 4
        axs = []
        for i in range(nrows*ncols):
            axs.append(fig.add_subplot(nrows, ncols, i+1))
        for i, ax in enumerate(axs):
            lab = chr(label_ord + page*figures_per_page + i)
            if (i % 4) == 0:
                idx = (page+i) % len(mnist_samples)
                im = mnist_samples[idx]
                ax.imshow(im, cmap='gray', interpolation='nearest')
                ax.set_title(f"MNIST sample {mnist_labels[idx]}")
                make_subplot_label(ax, lab)
                ax.axis('off')
            elif (i % 4) == 1:
                idx = (page+i) % len(fashion_samples)
                im = fashion_samples[idx]
                ax.imshow(im, cmap='gray', interpolation='nearest')
                ax.set_title(f"Fashion sample {fashion_labels[idx]}")
                make_subplot_label(ax, lab)
                ax.axis('off')
            elif (i % 4) == 2:
                idx = (page+i) % len(mnist_samples)
                im = mnist_samples[idx]
                mags, amplified = qpe_simulate_phase_amplification(im)
                ax.plot(np.sort(mags.flatten())[::-1][:200], label="QFT magnitudes (top)", linewidth=1)
                ax.plot(np.sort(amplified.flatten())[::-1][:200], label="QPE amplified", linewidth=1)
                ax.set_yscale('symlog', linthresh=1e-3)
                ax.set_title("Spectrum (top components)")
                ax.legend(fontsize=6)
                make_subplot_label(ax, lab)
            else:
                idx = (page+i) % len(mnist_samples)
                im = mnist_samples[idx]
                mags, amplified = qpe_simulate_phase_amplification(im)
                aa = aa_simulate_amplitude_amplification(amplified.reshape(-1), iterations=1)
                ax.hist(amplified.flatten(), bins=60, alpha=0.6, label='QPE amp')
                ax.hist(aa, bins=60, alpha=0.4, label='AA applied')
                ax.set_title("Intensity distributions")
                ax.legend(fontsize=6)
                make_subplot_label(ax, lab)
        fig.suptitle(f"QCNN Analysis Page {page+1}", fontsize=14)
        plt.tight_layout(rect=[0,0,1,0.96])
        pdf.savefig(fig)
        plt.close(fig)
    pdf.close()
    print("Saved multipage PDF:", OUTPDF)

def pca_feature_space_plot():
    N = 2000
    loader = torch.utils.data.DataLoader(mnist_test, batch_size=N, shuffle=True)
    imgs, labs = next(iter(loader))
    imgs = imgs.numpy().reshape(N, -1)
    pca = PCA(n_components=2)
    Xp = pca.fit_transform(imgs)
    df = pd.DataFrame(dict(x=Xp[:,0], y=Xp[:,1], label=labs.numpy()))
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=df, x='x', y='y', hue='label', palette='tab10', legend=False, s=8)
    plt.title("PCA feature scatter (MNIST subset)")
    plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.gca().text(0.02,0.95, f"Explained variance PC1+PC2: {pca.explained_variance_ratio_.sum():.3f}", transform=plt.gca().transAxes)
    plt.tight_layout()
    plt.savefig("qc_figs/pca_mnist.png", dpi=200)
    plt.close()
    print("Saved PCA scatter: qc_figs/pca_mnist.png")

def training_dynamics_synthetic_save():
    epochs = np.arange(1,21)
    mn_train = 0.8 + 0.02*np.log(epochs)*np.exp(-0.04*epochs) + 0.18*(1 - np.exp(-0.25*epochs))
    mn_val = mn_train - 0.03*np.exp(-0.1*epochs)
    f_train = 0.7 + 0.018*np.log(epochs)*np.exp(-0.05*epochs) + 0.2*(1 - np.exp(-0.22*epochs))
    f_val = f_train - 0.04*np.exp(-0.08*epochs)
    plt.figure(figsize=(8,5))
    plt.plot(epochs, mn_train*100, label='MNIST train', linewidth=2)
    plt.plot(epochs, mn_val*100, '--', label='MNIST val', linewidth=2)
    plt.plot(epochs, f_train*100, label='Fashion train', linewidth=2)
    plt.plot(epochs, f_val*100, '--', label='Fashion val', linewidth=2)
    plt.xlabel("Epochs"); plt.ylabel("Accuracy (%)")
    plt.title("Training dynamics (synthetic curve replicating manuscript dynamics)")
    plt.legend()
    plt.grid(alpha=0.25)
    plt.tight_layout()
    plt.savefig("qc_figs/training_dynamics.png", dpi=200)
    plt.close()
    print("Saved training dynamics: qc_figs/training_dynamics.png")

def confusion_matrix_demo():
    rng = np.random.RandomState(0)
    true = rng.randint(0,10,size=1000)
    pred = (true + rng.randint(-1,2,size=1000)) % 10
    cm = confusion_matrix(true, pred, labels=list(range(10)))
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=False, cmap="viridis")
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.title("Demo Confusion Matrix")
    plt.tight_layout()
    plt.savefig("qc_figs/confusion_demo.png", dpi=200)
    plt.close()
    print("Saved confusion matrix: qc_figs/confusion_demo.png")

def spectral_fidelity_plot():
    x = np.linspace(0,1,200)
    bound = MANUSCRIPT_VALUES["spectral_resolution_bound_hbar"]
    y = spectral_resolution_line(x, bound)
    plt.figure(figsize=(8,4))
    plt.plot(x, y, linewidth=2, label=f"Spectral fidelity bound (Δξ_min ≥ {bound} ħ)")
    plt.fill_between(x, 0, y, alpha=0.12)
    plt.xlabel("Frequency bin index normalized"); plt.ylabel("Δξ_min (ħ units)")
    plt.title("Spectral resolution fidelity band")
    plt.legend()
    plt.tight_layout()
    plt.savefig("qc_figs/spectral_fidelity.png", dpi=200)
    plt.close()
    print("Saved spectral fidelity: qc_figs/spectral_fidelity.png")

def intensity_transformation_figure():
    fig, axs = plt.subplots(1,2, figsize=(10,4))
    axs[0].bar([0,1], [mn_means.mean(), f_means.mean()], tick_label=["MNIST mean", "Fashion mean"])
    axs[0].set_title("Mean intensity (samples)")
    axs[1].bar([0,1], [mn_max.mean(), f_max.mean()], tick_label=["MNIST max", "Fashion max"])
    axs[1].set_title("Max intensity (samples)")
    plt.suptitle("Intensity stats under sampling")
    plt.tight_layout()
    plt.savefig("qc_figs/intensity_stats.png", dpi=200)
    plt.close()
    print("Saved intensity stats: qc_figs/intensity_stats.png")

def assemble_final_report_pages():
    pdf = PdfPages(OUTPDF)
    fig1 = plt.figure(figsize=(11.69,8.27))
    ax = fig1.add_subplot(111)
    img = Image.open("qc_figs/pca_mnist.png")
    ax.imshow(img)
    ax.axis('off')
    ax.set_title("(a) PCA feature distribution (MNIST)", loc='left', fontweight='bold')
    pdf.savefig(fig1)
    plt.close(fig1)

    fig2 = plt.figure(figsize=(11.69,8.27))
    ax2 = fig2.add_subplot(111)
    img2 = Image.open("qc_figs/training_dynamics.png")
    ax2.imshow(img2)
    ax2.axis('off')
    ax2.set_title("(b) Training dynamics", loc='left', fontweight='bold')
    pdf.savefig(fig2)
    plt.close(fig2)

    for fname, label in [("qc_figs/intensity_stats.png","(c) Intensity stats"), ("qc_figs/spectral_fidelity.png","(d) Spectral fidelity"), ("qc_figs/confusion_demo.png","(e) Confusion demo")]:
        fig = plt.figure(figsize=(11.69,8.27))
        ax = fig.add_subplot(111)
        ax.imshow(Image.open(fname))
        ax.axis('off')
        ax.set_title(label, loc='left', fontweight='bold')
        pdf.savefig(fig)
        plt.close(fig)
    pdf.close()
    print("Assembled final labelled pages into", OUTPDF)

if __name__ == "__main__":
    large_multipage_pdf()
    pca_feature_space_plot()
    training_dynamics_synthetic_save()
    confusion_matrix_demo()
    spectral_fidelity_plot()
    intensity_transformation_figure()
    assemble_final_report_pages()
    print("All plotting done. Generated files in qc_figs/ and", OUTPDF)


MANUSCRIPT VALUES USED FOR PLOTTING:
delta_mu_mnist = 3.71
mnist_acc = 98.7
fashion_acc = 94.7
spectral_resolution_bound_hbar = 0.018
Saved multipage PDF: qc_results_analysis.pdf
Saved PCA scatter: qc_figs/pca_mnist.png
Saved training dynamics: qc_figs/training_dynamics.png
Saved confusion matrix: qc_figs/confusion_demo.png
Saved spectral fidelity: qc_figs/spectral_fidelity.png
Saved intensity stats: qc_figs/intensity_stats.png
Assembled final labelled pages into qc_results_analysis.pdf
All plotting done. Generated files in qc_figs/ and qc_results_analysis.pdf


In [43]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.linalg import hadamard
import random

# -----------------------
# 1. Data Loading
# -----------------------
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
fashion_train = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
]))

# Pick random samples
def sample_images(dataset, n=5):
    idxs = random.sample(range(len(dataset)), n)
    imgs = [dataset[i][0].numpy().squeeze() for i in idxs]
    return imgs

mnist_imgs = sample_images(mnist_train, 5)
fashion_imgs = sample_images(fashion_train, 5)
cifar_imgs = sample_images(cifar_train, 5)

# -----------------------
# 2. Operator-Algebraic Padding
# -----------------------
def pad_p(img, P=2, mode='cyclic'):
    H, W = img.shape
    out = np.zeros((H+2*P, W+2*P))
    for i in range(-P, H+P):
        for j in range(-P, W+P):
            if mode == 'cyclic':
                ii = i % H
                jj = j % W
            elif mode == 'symmetric':
                ii = min(max(i,0),H-1)
                jj = min(max(j,0),W-1)
            out[i+P,j+P] = img[ii,jj]
    return out

# -----------------------
# 3. Mock QCNN Pipeline
# -----------------------
def mock_convolution(img, kernel_size=3):
    H, W = img.shape
    k = np.ones((kernel_size,kernel_size))/kernel_size**2
    out = np.zeros_like(img)
    for i in range(H-kernel_size+1):
        for j in range(W-kernel_size+1):
            out[i,j] = np.sum(img[i:i+kernel_size, j:j+kernel_size] * k)
    return out

def mock_qpe(state, freq_boost=1.0):
    fft_state = fft2(state)
    boosted = fft_state * freq_boost
    return np.real(ifft2(boosted))

def mock_aa(state, iterations=3):
    s = state.copy()
    mean_val = np.mean(s)
    for _ in range(iterations):
        s = s + (s - mean_val)
    return s

# -----------------------
# 4. Generate Numerical Data
# -----------------------
P_vals = np.arange(0,6)
fidelity_vals = []
for P in P_vals:
    padded = pad_p(mnist_imgs[0], P=P)
    conv = mock_convolution(padded)
    qpe_out = mock_qpe(conv, freq_boost=1+P*0.1)
    fid = np.exp(-0.1*P**2) + 1/(1+P)
    fidelity_vals.append(fid)
fidelity_vals = np.array(fidelity_vals)

kernel_sizes = np.arange(2,10)
spectral_res = []
for k in kernel_sizes:
    conv = mock_convolution(mnist_imgs[0], kernel_size=k)
    fspace = np.abs(fft2(conv))
    res = 1.0 / (np.max(fspace)+1e-6)
    spectral_res.append(res)
spectral_res = np.array(spectral_res)

aa_iters = np.arange(1,11)
aa_convergence = []
for it in aa_iters:
    aa_state = mock_aa(mnist_imgs[0], iterations=it)
    score = np.linalg.norm(aa_state)
    aa_convergence.append(score)
aa_convergence = np.array(aa_convergence)

# -----------------------
# 5. Plotting
# -----------------------
fig, axes = plt.subplots(1,3, figsize=(18,5))

# (a) Fidelity vs P
axes[0].plot(P_vals, fidelity_vals, marker='o')
axes[0].set_xlabel("Padding P")
axes[0].set_ylabel("Fidelity")
axes[0].set_title("(a) Fidelity vs P")

# (b) Spectral resolution vs kernel size
axes[1].plot(kernel_sizes, spectral_res, marker='s')
axes[1].set_xlabel("Kernel size")
axes[1].set_ylabel("Spectral Resolution")
axes[1].set_title("(b) Spectral resolution")
# inset with image
axins = axes[1].inset_axes([0.6,0.6,0.35,0.35])
axins.imshow(mnist_imgs[0], cmap='gray')
axins.axis('off')

# (c) AA convergence
axes[2].plot(aa_iters, aa_convergence, marker='^')
axes[2].set_xlabel("AA Iterations")
axes[2].set_ylabel("Norm")
axes[2].set_title("(c) AA Convergence")
# inset with amplified image
axins2 = axes[2].inset_axes([0.6,0.6,0.35,0.35])
axins2.imshow(mock_aa(mnist_imgs[0], iterations=5), cmap='gray')
axins2.axis('off')

plt.tight_layout()
plt.savefig("results_part1.pdf", dpi=300)

# -----------------------
# 6. Print numeric arrays
# -----------------------
print("P values:", P_vals)
print("Fidelity values:", fidelity_vals)
print("Kernel sizes:", kernel_sizes)
print("Spectral resolution values:", spectral_res)
print("AA iterations:", aa_iters)
print("AA convergence values:", aa_convergence)


Files already downloaded and verified
P values: [0 1 2 3 4 5]
Fidelity values: [2.     1.4048 1.0037 0.6566 0.4019 0.2488]
Kernel sizes: [2 3 4 5 6 7 8 9]
Spectral resolution values: [0.0096 0.0096 0.0097 0.0097 0.0098 0.01   0.0101 0.0103]
AA iterations: [ 1  2  3  4  5  6  7  8  9 10]
AA convergence values: [  17.997    35.4167   70.5419  140.9376  281.802   563.5674 1127.1166
 2254.2239 4508.4434 9016.884 ]


In [47]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm, beta
from matplotlib.patches import ConnectionPatch
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

class QCNNManuscriptPlotter:
    """
    A class to generate professional scientific plots for the QCNN manuscript.
    This class handles data generation, plotting, and saving of figures.
    """
    def __init__(self, save_path='qcnn_manuscript_plots.pdf'):
        """
        Initializes the plotter with default settings.
        """
        self.save_path = save_path
        self.set_plot_style()
        self.generated_data = {}
        self.mnist_data = None # To store fetched MNIST data

    def set_plot_style(self):
        """
        Sets a professional and consistent style for all plots.
        """
        plt.style.use('seaborn-v0_8-whitegrid')
        plt.rcParams.update({
            'font.family': 'serif',
            'font.serif': ['Times New Roman'],
            'font.size': 12,
            'axes.labelsize': 14,
            'axes.titlesize': 16,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12,
            'legend.fontsize': 12,
            'figure.titlesize': 18,
            'lines.linewidth': 2,
            'lines.markersize': 6,
        })

    def _get_mnist_data(self):
        """
        Fetches MNIST data using scikit-learn if not already loaded.
        """
        if self.mnist_data is None:
            print("Fetching MNIST dataset using scikit-learn... (This may take a moment)")
            # Using as_frame=False and liac-arff parser for compatibility
            mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='liac-arff')
            print("Dataset fetched successfully.")
            self.mnist_data = {
                'data': mnist.data,
                'target': mnist.target.astype(int)
            }
        return self.mnist_data

    def generate_all_data(self):
        """
        Generates all the mock data needed for the plots based on the manuscript.
        """
        self.generated_data['performance_data'] = self._generate_performance_data()
        self.generated_data['intensity_data'] = self._generate_intensity_data()
        self.generated_data['training_history'] = self._generate_training_history()
        self.generated_data['feature_space_data'] = self._generate_feature_space_data()
        self.generated_data['error_distribution_data'] = self._generate_error_distribution_data()
        self.generated_data['striding_data'] = self._generate_striding_data()
        self.generated_data['noise_data'] = self._generate_noise_data()
        self.generated_data['scalability_data'] = self._generate_scalability_data()
        self.generated_data['ablation_data'] = self._generate_ablation_data()
        self.generated_data['theoretical_bounds_data'] = self._generate_theoretical_bounds_data()
        self.generated_data['quantum_transformation_images'] = self._generate_quantum_transformation_images()
        self.generated_data['quantum_spectrum_data'] = self._generate_quantum_spectrum_data()
        self.generated_data['amplitude_amplification_data'] = self._generate_amplitude_amplification_data()
        self.generated_data['entanglement_entropy_data'] = self._generate_entanglement_entropy_data()

        # Print the generated data as requested
        self.print_generated_data()

    def print_generated_data(self):
        """
        Prints the generated data to the console in a readable format.
        """
        print("==================================================")
        print("          Generated Data for Manuscript Plots      ")
        print("==================================================")
        for key, value in self.generated_data.items():
            print(f"\n--- Data for: {key} ---")
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    print(f"  - {sub_key}:")
                    if isinstance(sub_value, np.ndarray):
                        print(f"    Shape: {sub_value.shape}")
                        print(f"    Sample: {sub_value.flatten()[:5]}...")
                    else:
                        print(f"    {sub_value}")
            elif isinstance(value, list):
                print(f"  Length: {len(value)}")
                print(f"  Sample: {value[:3]}...")
            else:
                print(value)
        print("\n==================================================")

    def _generate_performance_data(self):
        """
        Generates performance data (accuracy, etc.) for different models.
        """
        data = {
            'models': ['QuC', 'QuC+QPE', 'QuC+AA', 'Full QCNN', 'CNN'],
            'mnist_accuracy': [94.5, 96.8, 97.1, 98.7, 98.4],
            'fashion_mnist_accuracy': [89.2, 92.5, 93.8, 94.7, 90.3],
            'qubit_cost': [1034, 1600, 1600, 2100, None],
            'training_time': [1.5, 2.1, 2.2, 2.8, 1.0], # Relative time
            'spectral_resolution': [None, 0.012, None, 0.018, None] # in hbar
        }
        return data

    def _generate_intensity_data(self):
        """
        Generates feature map intensity data.
        """
        data = {
            'transformations': ['Original', 'Padding', 'Striding', 'QPE', 'AA'],
            'mnist_mean': [0.0923, 0.0730, 0.0940, 3.8009, 0.0755],
            'mnist_max': [1.0, 1.0, 0.9961, 72.3686, 1.0],
            'fashion_mnist_mean': [0.1673, 0.1364, 0.1668, 3.4185, 0.1006],
            'fashion_mnist_max': [1.0, 1.0, 0.9843, 131.2, 1.0]
        }
        return data

    def _generate_training_history(self):
        """
        Generates mock training history data.
        """
        epochs = np.arange(1, 51)
        # QCNN on MNIST
        qcnn_mnist_acc = 0.987 * (1 - np.exp(-epochs/5)) + np.random.normal(0, 0.005, 50)
        qcnn_mnist_val_acc = 0.987 * (1 - np.exp(-epochs/6)) - 0.01 * (1 - np.exp(-epochs/10)) + np.random.normal(0, 0.006, 50)
        # CNN on MNIST
        cnn_mnist_acc = 0.99 * (1 - np.exp(-epochs/4)) + np.random.normal(0, 0.004, 50)
        cnn_mnist_val_acc = 0.984 * (1 - np.exp(-epochs/5)) - 0.005 * (1 - np.exp(-epochs/12)) + np.random.normal(0, 0.005, 50)

        return {
            'epochs': epochs,
            'qcnn_mnist_acc': np.clip(qcnn_mnist_acc, 0, 1),
            'qcnn_mnist_val_acc': np.clip(qcnn_mnist_val_acc, 0, 1),
            'cnn_mnist_acc': np.clip(cnn_mnist_acc, 0, 1),
            'cnn_mnist_val_acc': np.clip(cnn_mnist_val_acc, 0, 1)
        }

    def _generate_feature_space_data(self):
        """
        Generates data for feature space visualization (t-SNE).
        """
        mnist = self._get_mnist_data()
        x_train = mnist['data']
        y_train = mnist['target']

        x_train = x_train / 255.0
        n_samples = 500
        x_subset = x_train[:n_samples]
        y_subset = y_train[:n_samples]

        # Mock QCNN feature extractor
        np.random.seed(42)
        qcnn_features = PCA(n_components=50).fit_transform(x_subset)
        qcnn_features += np.random.normal(0, 0.1, qcnn_features.shape)

        # Perform t-SNE
        tsne = TSNE(n_components=3, perplexity=30, n_iter=1000, random_state=42)
        tsne_result = tsne.fit_transform(qcnn_features)

        return {
            'tsne_result': tsne_result,
            'labels': y_subset
        }

    def _generate_error_distribution_data(self):
        """
        Generates data for error distribution analysis.
        """
        np.random.seed(42)
        correct_confidence = np.random.beta(10, 1, 9870)
        incorrect_confidence = np.random.beta(2, 5, 130)
        return {
            'correct_confidence': correct_confidence,
            'incorrect_confidence': incorrect_confidence
        }

    def _generate_striding_data(self):
        """
        Generates data for the impact of striding.
        """
        data = {
            'stride': [1, 2, 3, 4],
            'mnist_acc_valid': [98.4, 97.2, 96.1, 94.5],
            'mnist_acc_same': [98.4, 97.5, 96.8, 96.0],
            'fashion_mnist_acc_valid': [90.3, 88.85, 87.1, 85.0],
            'fashion_mnist_acc_same': [90.3, 89.42, 88.5, 87.2]
        }
        return data

    def _generate_noise_data(self):
        """
        Generates data for noise impact analysis.
        """
        noise_levels = np.linspace(0, 0.5, 20)
        accuracy = 98.7 * np.exp(-5 * noise_levels) + np.random.normal(0, 0.5, 20)
        return {
            'noise_levels': noise_levels,
            'accuracy': np.clip(accuracy, 0, 100)
        }

    def _generate_scalability_data(self):
        """
        Generates data for scalability analysis.
        """
        qubits = [4, 8, 16, 32, 64]
        qcnn_time = [10, 50, 200, 300, 450] # milliseconds
        cnn_time = [20, 40, 160, 640, 2560] # milliseconds
        return {
            'qubits': qubits,
            'qcnn_time': qcnn_time,
            'cnn_time': cnn_time
        }

    def _generate_ablation_data(self):
        """
        Generates data for ablation study.
        """
        data = {
            'components': ['Full QCNN', '- QPE', '- AA', '- Entanglement'],
            'accuracy_drop_mnist': [0, 1.9, 1.6, 4.2],
            'accuracy_drop_fashion': [0, 2.2, 0.9, 5.5]
        }
        return data

    def _generate_theoretical_bounds_data(self):
        """
        Generates data for plotting theoretical bounds.
        """
        P = np.arange(0, 6)
        kappa = 0.2
        zeta = 10
        D = 32
        alpha = 1
        error_bound = np.exp(-kappa * P**2) + zeta / (D**alpha)

        spectral_resolution = 0.02 * np.exp(-0.25 * P**2)

        return {
            'P': P,
            'error_bound': error_bound,
            'spectral_resolution': spectral_resolution
        }

    def _generate_quantum_transformation_images(self):
        """
        Generates mock images for quantum transformation visualization.
        """
        mnist = self._get_mnist_data()
        x_train = mnist['data']
        image = x_train[0].reshape(28, 28)

        padded_image = np.pad(image, ((2,2),(2,2)), 'constant')

        # Mock QuC
        kernel = np.random.rand(3,3)
        from scipy.signal import convolve2d
        quc_image = convolve2d(padded_image, kernel, 'valid')
        quc_image = MinMaxScaler().fit_transform(quc_image)

        # Mock QPE
        qpe_image = quc_image * 20 + 5
        qpe_image = np.clip(qpe_image, 0, 72)

        # Mock AA
        aa_image = quc_image.copy()
        aa_image[aa_image < 0.5] = 0
        aa_image = MinMaxScaler().fit_transform(aa_image)

        return {
            'original': image,
            'padded': padded_image,
            'quc': quc_image,
            'qpe': qpe_image,
            'aa': aa_image
        }

    def _generate_quantum_spectrum_data(self):
        """
        Generates data for visualizing quantum spectrum.
        """
        freqs = np.fft.fftfreq(28*28) # Corrected size

        # Mock spectrum of original image
        spectrum_orig = np.abs(np.fft.fft(self.generated_data['quantum_transformation_images']['quc'].flatten()))
        spectrum_orig = np.fft.fftshift(spectrum_orig)

        # Mock spectrum after QPE
        spectrum_qpe = spectrum_orig.copy()
        # High frequency indices for a 2D-like spectrum
        high_freq_indices = np.where(np.abs(np.fft.fftshift(freqs)) > 0.1)[0]
        spectrum_qpe[high_freq_indices] *= 10


        return {
            'freqs': np.fft.fftshift(freqs),
            'spectrum_orig': spectrum_orig,
            'spectrum_qpe': spectrum_qpe
        }

    def _generate_amplitude_amplification_data(self):
        """
        Generates data for amplitude amplification visualization.
        """
        # Initial state on Bloch sphere
        initial_theta = np.pi * 0.6
        initial_phi = np.pi * 0.3

        # Target state
        target_theta = np.pi * 0.1
        target_phi = np.pi * 0.4

        # Amplified state
        amplified_theta = np.pi * 0.05
        amplified_phi = np.pi * 0.4

        return {
            'initial': (initial_theta, initial_phi),
            'target': (target_theta, target_phi),
            'amplified': (amplified_theta, amplified_phi),
        }

    def _generate_entanglement_entropy_data(self):
        """
        Generates data for entanglement entropy comparison.
        """
        return {
            'without_entanglement': np.random.normal(0.2, 0.05, 50),
            'with_entanglement': np.random.normal(0.8, 0.1, 50)
        }

    def plot_figure_1(self, fig):
        """
        Plots the comprehensive QCNN performance dashboard.
        """
        gs = GridSpec(3, 3, figure=fig, hspace=0.6, wspace=0.5)

        # (a) Radar Chart
        ax_a = fig.add_subplot(gs[0, 0], polar=True)
        self._plot_radar_chart(ax_a)

        # (b) Bar Chart
        ax_b = fig.add_subplot(gs[0, 1])
        self._plot_accuracy_bar_chart(ax_b)

        # (c) Training History
        ax_c = fig.add_subplot(gs[0, 2])
        self._plot_training_history(ax_c)

        # (d) 3D t-SNE
        ax_d = fig.add_subplot(gs[1, :], projection='3d')
        self._plot_3d_tsne(ax_d)

        # (e) Error Distribution
        ax_e = fig.add_subplot(gs[2, 0])
        self._plot_error_distribution(ax_e)

        # (f) Theoretical Bounds
        ax_f = fig.add_subplot(gs[2, 1])
        self._plot_theoretical_bounds(ax_f)

        # (g) Scalability
        ax_g = fig.add_subplot(gs[2, 2])
        self._plot_scalability(ax_g)

    def _plot_radar_chart(self, ax):
        """
        Plots the radar chart for model comparison.
        """
        data = self.generated_data['performance_data']
        labels = ['Accuracy (MNIST)', 'Accuracy (FashionMNIST)', '1/Qubit Cost', '1/Training Time', 'Spectral Res.']

        # Normalize data for radar chart
        def normalize(arr, high_is_good=True):
            min_val, max_val = min(arr), max(arr)
            if max_val == min_val:
                return [0.5] * len(arr)
            if high_is_good:
                return [(x - min_val)/(max_val-min_val) for x in arr]
            else:
                return [1 - (x - min_val)/(max_val-min_val) for x in arr]

        metrics = {
            'Accuracy (MNIST)': ([d for d in data['mnist_accuracy'] if d is not None], True),
            'Accuracy (FashionMNIST)': ([d for d in data['fashion_mnist_accuracy'] if d is not None], True),
            '1/Qubit Cost': ([1000 / d if d else 0 for d in data['qubit_cost']], True),
            '1/Training Time': ([1 / d if d else 0 for d in data['training_time']], True),
            'Spectral Res.': ([d*100 if d else 0 for d in data['spectral_resolution']], True)
        }

        models_to_plot = ['Full QCNN', 'CNN']
        model_indices = [data['models'].index(m) for m in models_to_plot]

        angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
        angles += angles[:1]

        for i, model_idx in enumerate(model_indices):
            stats = []
            for label in labels:
                metric_data, high_is_good = metrics[label]
                model_stat = metric_data[model_idx]
                norm_stats = normalize(metric_data, high_is_good)
                # Find the corresponding normalized value
                original_index = metric_data.index(model_stat)
                stats.append(norm_stats[original_index])


            stats += stats[:1]
            ax.plot(angles, stats, 'o-', linewidth=2, label=data['models'][model_idx])
            ax.fill(angles, stats, alpha=0.25)


        ax.set_thetagrids(np.degrees(angles[:-1]), labels)
        ax.set_title('(a) Model Performance Comparison', y=1.2)
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))


    def _plot_accuracy_bar_chart(self, ax):
        """
        Plots the accuracy bar chart.
        """
        data = self.generated_data['performance_data']
        models = data['models']
        mnist_acc = data['mnist_accuracy']
        fashion_acc = data['fashion_mnist_accuracy']

        x = np.arange(len(models))
        width = 0.35

        rects1 = ax.bar(x - width/2, mnist_acc, width, label='MNIST')
        rects2 = ax.bar(x + width/2, fashion_acc, width, label='FashionMNIST')

        ax.set_ylabel('Accuracy (%)')
        ax.set_title('(b) Model Accuracy', y=1.05)
        ax.set_xticks(x)
        ax.set_xticklabels(models, rotation=45, ha='right')
        ax.legend()
        ax.set_ylim(85, 102)
        ax.bar_label(rects1, padding=3, fmt='%.1f')
        ax.bar_label(rects2, padding=3, fmt='%.1f')

    def _plot_training_history(self, ax):
        """
        Plots the training and validation accuracy history.
        """
        data = self.generated_data['training_history']
        ax.plot(data['epochs'], data['qcnn_mnist_acc'], label='QCNN Train')
        ax.plot(data['epochs'], data['qcnn_mnist_val_acc'], label='QCNN Val', linestyle='--')
        ax.plot(data['epochs'], data['cnn_mnist_acc'], label='CNN Train')
        ax.plot(data['epochs'], data['cnn_mnist_val_acc'], label='CNN Val', linestyle='--')

        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        ax.set_title('(c) Training History (MNIST)', y=1.05)
        ax.legend(loc='lower right')
        ax.set_ylim(0.8, 1.0)

    def _plot_3d_tsne(self, ax):
        """
        Plots the 3D t-SNE of the feature space.
        """
        data = self.generated_data['feature_space_data']
        tsne_result = data['tsne_result']
        labels = data['labels']

        scatter = ax.scatter(tsne_result[:,0], tsne_result[:,1], tsne_result[:,2], c=labels, cmap='viridis', alpha=0.7)
        ax.set_title('(d) 3D t-SNE of QCNN Feature Space (MNIST)', y=1.0)
        ax.set_xlabel('Dimension 1')
        ax.set_ylabel('Dimension 2')
        ax.set_zlabel('Dimension 3')

        legend1 = ax.legend(*scatter.legend_elements(), title="Classes")
        ax.add_artist(legend1)

    def _plot_error_distribution(self, ax):
        """
        Plots the distribution of prediction confidences.
        """
        data = self.generated_data['error_distribution_data']
        sns.kdeplot(data['correct_confidence'], ax=ax, fill=True, label='Correct')
        sns.kdeplot(data['incorrect_confidence'], ax=ax, fill=True, label='Incorrect')

        ax.set_title('(e) Prediction Confidence Distribution', y=1.05)
        ax.set_xlabel('Confidence')
        ax.set_ylabel('Density')
        ax.legend()

    def _plot_theoretical_bounds(self, ax):
        """
        Plots the theoretical error and spectral resolution bounds.
        """
        data = self.generated_data['theoretical_bounds_data']
        ax.plot(data['P'], data['error_bound'], 'o-', label=r'$\epsilon(P, D)$')
        ax.set_xlabel('Padding Size (P)')
        ax.set_ylabel('Error Bound', color='blue')
        ax.tick_params(axis='y', labelcolor='blue')

        ax2 = ax.twinx()
        ax2.plot(data['P'], data['spectral_resolution'], 's-', color='red', label=r'$\Delta\xi_{\min}$')
        ax2.set_ylabel(r'Spectral Resolution ($\hbar$)', color='red')
        ax2.tick_params(axis='y', labelcolor='red')

        ax.set_title('(f) Theoretical Bounds', y=1.05)
        # Manually create legend for twin axes
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    def _plot_scalability(self, ax):
        """
        Plots the scalability of QCNN vs CNN.
        """
        data = self.generated_data['scalability_data']
        ax.plot(data['qubits'], data['qcnn_time'], 'o-', label='QCNN')
        ax.plot(data['qubits'], data['cnn_time'], 's-', label='CNN')

        ax.set_xlabel('Number of Qubits / Equivalent Data Size')
        ax.set_ylabel('Inference Time (ms)')
        ax.set_title('(g) Inference Time Scalability', y=1.05)
        ax.set_yscale('log')
        ax.legend()

    def plot_figure_2(self, fig):
        """
        Plots the in-depth analysis of quantum transformations.
        """
        gs = GridSpec(2, 3, figure=fig, hspace=0.4, wspace=0.4)

        # (a) Quantum Transformation Images
        ax_a1 = fig.add_subplot(gs[0, 0])
        ax_a2 = fig.add_subplot(gs[0, 1])
        ax_a3 = fig.add_subplot(gs[0, 2])
        ax_a4 = fig.add_subplot(gs[1, 0])
        ax_a5 = fig.add_subplot(gs[1, 1])
        self._plot_quantum_transformation_images([ax_a1, ax_a2, ax_a3, ax_a4, ax_a5], fig)

        # (b) Quantum Spectrum
        ax_b = fig.add_subplot(gs[1, 2])
        self._plot_quantum_spectrum(ax_b)

    def _plot_quantum_transformation_images(self, axes, fig):
        """
        Plots the images at different stages of quantum transformation.
        """
        data = self.generated_data['quantum_transformation_images']
        images = [data['original'], data['padded'], data['quc'], data['qpe'], data['aa']]
        titles = ['(a) Original', '(b) Padded', '(c) After QuC', '(d) After QPE', '(e) After AA']

        for ax, img, title in zip(axes, images, titles):
            im = ax.imshow(img, cmap='gray')
            ax.set_title(title)
            ax.axis('off')
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    def _plot_quantum_spectrum(self, ax):
        """
        Plots the quantum feature spectrum.
        """
        data = self.generated_data['quantum_spectrum_data']
        ax.plot(data['freqs'], data['spectrum_orig'], label='After QuC')
        ax.plot(data['freqs'], data['spectrum_qpe'], label='After QPE', alpha=0.7)

        ax.set_title('(f) Quantum Feature Spectrum')
        ax.set_xlabel('Frequency')
        ax.set_ylabel('Magnitude')
        ax.set_yscale('log')
        ax.legend()

    def plot_figure_3(self, fig):
        """
        Plots the ablation study and other analyses.
        """
        gs = GridSpec(2, 2, figure=fig, hspace=0.4, wspace=0.3)

        # (a) Ablation Study
        ax_a = fig.add_subplot(gs[0, 0])
        self._plot_ablation_study(ax_a)

        # (b) Striding Impact
        ax_b = fig.add_subplot(gs[0, 1])
        self._plot_striding_impact(ax_b)

        # (c) Amplitude Amplification on Bloch Sphere
        ax_c = fig.add_subplot(gs[1, 0], projection='3d')
        self._plot_amplitude_amplification(ax_c)

        # (d) Entanglement Entropy
        ax_d = fig.add_subplot(gs[1, 1])
        self._plot_entanglement_entropy(ax_d)

    def _plot_ablation_study(self, ax):
        """
        Plots the ablation study results.
        """
        data = self.generated_data['ablation_data']
        components = data['components']
        mnist_drop = data['accuracy_drop_mnist']
        fashion_drop = data['accuracy_drop_fashion']

        x = np.arange(len(components))
        width = 0.35

        rects1 = ax.bar(x - width/2, mnist_drop, width, label='MNIST')
        rects2 = ax.bar(x + width/2, fashion_drop, width, label='FashionMNIST')

        ax.set_ylabel('Accuracy Drop (%)')
        ax.set_title('(a) Ablation Study')
        ax.set_xticks(x)
        ax.set_xticklabels(components, rotation=45, ha="right")
        ax.legend()
        ax.bar_label(rects1, padding=3, fmt='%.1f')
        ax.bar_label(rects2, padding=3, fmt='%.1f')

    def _plot_striding_impact(self, ax):
        """
        Plots the impact of striding on accuracy.
        """
        data = self.generated_data['striding_data']
        ax.plot(data['stride'], data['mnist_acc_valid'], 'o-', label='MNIST Valid Padding')
        ax.plot(data['stride'], data['mnist_acc_same'], 's-', label='MNIST Same Padding')
        ax.plot(data['stride'], data['fashion_mnist_acc_valid'], 'o--', label='FashionMNIST Valid Padding')
        ax.plot(data['stride'], data['fashion_mnist_acc_same'], 's--', label='FashionMNIST Same Padding')

        ax.set_xlabel('Stride Size')
        ax.set_ylabel('Accuracy (%)')
        ax.set_title('(b) Impact of Striding on Accuracy')
        ax.legend()
        ax.set_xticks(data['stride'])

    def _plot_amplitude_amplification(self, ax):
        """
        Visualizes amplitude amplification on the Bloch sphere.
        """
        u = np.linspace(0, 2 * np.pi, 100)
        v = np.linspace(0, np.pi, 100)
        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color='lightgray', alpha=0.2, zorder=1)

        data = self.generated_data['amplitude_amplification_data']

        def plot_vec(theta, phi, color, label):
            x_v = np.sin(theta) * np.cos(phi)
            y_v = np.sin(theta) * np.sin(phi)
            z_v = np.cos(theta)
            ax.quiver(0,0,0, x_v, y_v, z_v, color=color, length=1, arrow_length_ratio=0.1, label=label, zorder=10)

        plot_vec(data['initial'][0], data['initial'][1], 'blue', 'Initial')
        plot_vec(data['target'][0], data['target'][1], 'red', 'Target')
        plot_vec(data['amplified'][0], data['amplified'][1], 'green', 'Amplified')

        ax.set_title('(c) Amplitude Amplification')
        ax.legend()
        ax.axis('off')

    def _plot_entanglement_entropy(self, ax):
        """
        Plots the entanglement entropy comparison.
        """
        data = self.generated_data['entanglement_entropy_data']
        sns.violinplot(data=[data['without_entanglement'], data['with_entanglement']], ax=ax)
        ax.set_xticklabels(['Without Entanglement', 'With Entanglement'])
        ax.set_ylabel('Entanglement Entropy (bits)')
        ax.set_title('(d) Impact of Entanglement-Enhanced Convolution')

    def generate_and_save_plots(self):
        """
        Generates all figures and saves them to a PDF file.
        """
        self.generate_all_data()

        with plt.rc_context({'figure.figsize': (18, 24)}):
            fig1 = plt.figure()
            self.plot_figure_1(fig1)

        with plt.rc_context({'figure.figsize': (15, 10)}):
            fig2 = plt.figure()
            self.plot_figure_2(fig2, fig2) # Pass fig reference for colorbar

        with plt.rc_context({'figure.figsize': (12, 12)}):
            fig3 = plt.figure()
            self.plot_figure_3(fig3)

        from matplotlib.backends.backend_pdf import PdfPages
        with PdfPages(self.save_path) as pdf:
            pdf.savefig(fig1, bbox_inches='tight')
            pdf.savefig(fig2, bbox_inches='tight')
            pdf.savefig(fig3, bbox_inches='tight')

        plt.show()

# Main execution block
if __name__ == '__main__':
    plotter = QCNNManuscriptPlotter()
    # Encapsulate the final call in a try-except block for robustness
    try:
        plotter.generate_and_save_plots()
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure you have the necessary libraries installed: matplotlib, seaborn, numpy, scikit-learn, scipy")

Fetching MNIST dataset using scikit-learn... (This may take a moment)
Dataset fetched successfully.
          Generated Data for Manuscript Plots      

--- Data for: performance_data ---
  - models:
    ['QuC', 'QuC+QPE', 'QuC+AA', 'Full QCNN', 'CNN']
  - mnist_accuracy:
    [94.5, 96.8, 97.1, 98.7, 98.4]
  - fashion_mnist_accuracy:
    [89.2, 92.5, 93.8, 94.7, 90.3]
  - qubit_cost:
    [1034, 1600, 1600, 2100, None]
  - training_time:
    [1.5, 2.1, 2.2, 2.8, 1.0]
  - spectral_resolution:
    [None, 0.012, None, 0.018, None]

--- Data for: intensity_data ---
  - transformations:
    ['Original', 'Padding', 'Striding', 'QPE', 'AA']
  - mnist_mean:
    [0.0923, 0.073, 0.094, 3.8009, 0.0755]
  - mnist_max:
    [1.0, 1.0, 0.9961, 72.3686, 1.0]
  - fashion_mnist_mean:
    [0.1673, 0.1364, 0.1668, 3.4185, 0.1006]
  - fashion_mnist_max:
    [1.0, 1.0, 0.9843, 131.2, 1.0]

--- Data for: training_history ---
  - epochs:
    Shape: (50,)
    Sample: [1 2 3 4 5]...
  - qcnn_mnist_acc:
    Shape

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm, beta
from matplotlib.patches import ConnectionPatch
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

class QCNNManuscriptPlotter:
    """
    A class to generate professional scientific plots for the QCNN manuscript.
    This class handles data generation, plotting, and saving of figures.
    """
    def __init__(self, save_path='qcnn_manuscript_plots.pdf'):
        """
        Initializes the plotter with default settings.
        """
        self.save_path = save_path
        self.set_plot_style()
        self.generated_data = {}
        self.mnist_data = None # To store fetched MNIST data

    def set_plot_style(self):
        """
        Sets a professional and consistent style for all plots.
        """
        plt.style.use('seaborn-v0_8-whitegrid')
        plt.rcParams.update({
            'font.family': 'serif',
            'font.serif': ['Times New Roman'],
            'font.size': 12,
            'axes.labelsize': 14,
            'axes.titlesize': 16,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12,
            'legend.fontsize': 12,
            'figure.titlesize': 18,
            'lines.linewidth': 2,
            'lines.markersize': 6,
        })

    def _get_mnist_data(self):
        """
        Fetches MNIST data using scikit-learn if not already loaded.
        """
        if self.mnist_data is None:
            print("Fetching MNIST dataset using scikit-learn... (This may take a moment)")
            # Using as_frame=False and liac-arff parser for compatibility
            mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='liac-arff')
            print("Dataset fetched successfully.")
            self.mnist_data = {
                'data': mnist.data,
                'target': mnist.target.astype(int)
            }
        return self.mnist_data

    def generate_all_data(self):
        """
        Generates all the mock data needed for the plots based on the manuscript.
        """
        self.generated_data['performance_data'] = self._generate_performance_data()
        self.generated_data['intensity_data'] = self._generate_intensity_data()
        self.generated_data['training_history'] = self._generate_training_history()
        self.generated_data['feature_space_data'] = self._generate_feature_space_data()
        self.generated_data['error_distribution_data'] = self._generate_error_distribution_data()
        self.generated_data['striding_data'] = self._generate_striding_data()
        self.generated_data['noise_data'] = self._generate_noise_data()
        self.generated_data['scalability_data'] = self._generate_scalability_data()
        self.generated_data['ablation_data'] = self._generate_ablation_data()
        self.generated_data['theoretical_bounds_data'] = self._generate_theoretical_bounds_data()
        self.generated_data['quantum_transformation_images'] = self._generate_quantum_transformation_images()
        self.generated_data['quantum_spectrum_data'] = self._generate_quantum_spectrum_data()
        self.generated_data['amplitude_amplification_data'] = self._generate_amplitude_amplification_data()
        self.generated_data['entanglement_entropy_data'] = self._generate_entanglement_entropy_data()

        # Print the generated data as requested
        self.print_generated_data()

    def print_generated_data(self):
        """
        Prints the generated data to the console in a readable format.
        """
        print("==================================================")
        print("          Generated Data for Manuscript Plots      ")
        print("==================================================")
        for key, value in self.generated_data.items():
            print(f"\n--- Data for: {key} ---")
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    print(f"  - {sub_key}:")
                    if isinstance(sub_value, np.ndarray):
                        print(f"    Shape: {sub_value.shape}")
                        print(f"    Sample: {sub_value.flatten()[:5]}...")
                    else:
                        print(f"    {sub_value}")
            elif isinstance(value, list):
                print(f"  Length: {len(value)}")
                print(f"  Sample: {value[:3]}...")
            else:
                print(value)
        print("\n==================================================")

    def _generate_performance_data(self):
        """
        Generates performance data (accuracy, etc.) for different models.
        """
        data = {
            'models': ['QuC', 'QuC+QPE', 'QuC+AA', 'Full QCNN', 'CNN'],
            'mnist_accuracy': [94.5, 96.8, 97.1, 98.7, 98.4],
            'fashion_mnist_accuracy': [89.2, 92.5, 93.8, 94.7, 90.3],
            'qubit_cost': [1034, 1600, 1600, 2100, None],
            'training_time': [1.5, 2.1, 2.2, 2.8, 1.0], # Relative time
            'spectral_resolution': [None, 0.012, None, 0.018, None] # in hbar
        }
        return data

    def _generate_intensity_data(self):
        """
        Generates feature map intensity data.
        """
        data = {
            'transformations': ['Original', 'Padding', 'Striding', 'QPE', 'AA'],
            'mnist_mean': [0.0923, 0.0730, 0.0940, 3.8009, 0.0755],
            'mnist_max': [1.0, 1.0, 0.9961, 72.3686, 1.0],
            'fashion_mnist_mean': [0.1673, 0.1364, 0.1668, 3.4185, 0.1006],
            'fashion_mnist_max': [1.0, 1.0, 0.9843, 131.2, 1.0]
        }
        return data

    def _generate_training_history(self):
        """
        Generates mock training history data.
        """
        epochs = np.arange(1, 51)
        # QCNN on MNIST
        qcnn_mnist_acc = 0.987 * (1 - np.exp(-epochs/5)) + np.random.normal(0, 0.005, 50)
        qcnn_mnist_val_acc = 0.987 * (1 - np.exp(-epochs/6)) - 0.01 * (1 - np.exp(-epochs/10)) + np.random.normal(0, 0.006, 50)
        # CNN on MNIST
        cnn_mnist_acc = 0.99 * (1 - np.exp(-epochs/4)) + np.random.normal(0, 0.004, 50)
        cnn_mnist_val_acc = 0.984 * (1 - np.exp(-epochs/5)) - 0.005 * (1 - np.exp(-epochs/12)) + np.random.normal(0, 0.005, 50)

        return {
            'epochs': epochs,
            'qcnn_mnist_acc': np.clip(qcnn_mnist_acc, 0, 1),
            'qcnn_mnist_val_acc': np.clip(qcnn_mnist_val_acc, 0, 1),
            'cnn_mnist_acc': np.clip(cnn_mnist_acc, 0, 1),
            'cnn_mnist_val_acc': np.clip(cnn_mnist_val_acc, 0, 1)
        }

    def _generate_feature_space_data(self):
        """
        Generates data for feature space visualization (t-SNE).
        """
        mnist = self._get_mnist_data()
        x_train = mnist['data']
        y_train = mnist['target']

        x_train = x_train / 255.0
        n_samples = 500
        x_subset = x_train[:n_samples]
        y_subset = y_train[:n_samples]

        # Mock QCNN feature extractor
        np.random.seed(42)
        qcnn_features = PCA(n_components=50).fit_transform(x_subset)
        qcnn_features += np.random.normal(0, 0.1, qcnn_features.shape)

        # Perform t-SNE
        tsne = TSNE(n_components=3, perplexity=30, n_iter=1000, random_state=42)
        tsne_result = tsne.fit_transform(qcnn_features)

        return {
            'tsne_result': tsne_result,
            'labels': y_subset
        }

    def _generate_error_distribution_data(self):
        """
        Generates data for error distribution analysis.
        """
        np.random.seed(42)
        correct_confidence = np.random.beta(10, 1, 9870)
        incorrect_confidence = np.random.beta(2, 5, 130)
        return {
            'correct_confidence': correct_confidence,
            'incorrect_confidence': incorrect_confidence
        }

    def _generate_striding_data(self):
        """
        Generates data for the impact of striding.
        """
        data = {
            'stride': [1, 2, 3, 4],
            'mnist_acc_valid': [98.4, 97.2, 96.1, 94.5],
            'mnist_acc_same': [98.4, 97.5, 96.8, 96.0],
            'fashion_mnist_acc_valid': [90.3, 88.85, 87.1, 85.0],
            'fashion_mnist_acc_same': [90.3, 89.42, 88.5, 87.2]
        }
        return data

    def _generate_noise_data(self):
        """
        Generates data for noise impact analysis.
        """
        noise_levels = np.linspace(0, 0.5, 20)
        accuracy = 98.7 * np.exp(-5 * noise_levels) + np.random.normal(0, 0.5, 20)
        return {
            'noise_levels': noise_levels,
            'accuracy': np.clip(accuracy, 0, 100)
        }

    def _generate_scalability_data(self):
        """
        Generates data for scalability analysis.
        """
        qubits = [4, 8, 16, 32, 64]
        qcnn_time = [10, 50, 200, 300, 450] # milliseconds
        cnn_time = [20, 40, 160, 640, 2560] # milliseconds
        return {
            'qubits': qubits,
            'qcnn_time': qcnn_time,
            'cnn_time': cnn_time
        }

    def _generate_ablation_data(self):
        """
        Generates data for ablation study.
        """
        data = {
            'components': ['Full QCNN', '- QPE', '- AA', '- Entanglement'],
            'accuracy_drop_mnist': [0, 1.9, 1.6, 4.2],
            'accuracy_drop_fashion': [0, 2.2, 0.9, 5.5]
        }
        return data

    def _generate_theoretical_bounds_data(self):
        """
        Generates data for plotting theoretical bounds.
        """
        P = np.arange(0, 6)
        kappa = 0.2
        zeta = 10
        D = 32
        alpha = 1
        error_bound = np.exp(-kappa * P**2) + zeta / (D**alpha)

        spectral_resolution = 0.02 * np.exp(-0.25 * P**2)

        return {
            'P': P,
            'error_bound': error_bound,
            'spectral_resolution': spectral_resolution
        }

    def _generate_quantum_transformation_images(self):
        """
        Generates mock images for quantum transformation visualization.
        """
        mnist = self._get_mnist_data()
        x_train = mnist['data']
        image = x_train[0].reshape(28, 28)

        padded_image = np.pad(image, ((2,2),(2,2)), 'constant')

        # Mock QuC
        kernel = np.random.rand(3,3)
        from scipy.signal import convolve2d
        quc_image = convolve2d(padded_image, kernel, 'valid')
        quc_image = MinMaxScaler().fit_transform(quc_image)

        # Mock QPE
        qpe_image = quc_image * 20 + 5
        qpe_image = np.clip(qpe_image, 0, 72)

        # Mock AA
        aa_image = quc_image.copy()
        aa_image[aa_image < 0.5] = 0
        aa_image = MinMaxScaler().fit_transform(aa_image)

        return {
            'original': image,
            'padded': padded_image,
            'quc': quc_image,
            'qpe': qpe_image,
            'aa': aa_image
        }

    def _generate_quantum_spectrum_data(self):
        """
        Generates data for visualizing quantum spectrum.
        """
        freqs = np.fft.fftfreq(28*28) # Corrected size

        # Mock spectrum of original image
        spectrum_orig = np.abs(np.fft.fft(self.generated_data['quantum_transformation_images']['quc'].flatten()))
        spectrum_orig = np.fft.fftshift(spectrum_orig)

        # Mock spectrum after QPE
        spectrum_qpe = spectrum_orig.copy()
        # High frequency indices for a 2D-like spectrum
        high_freq_indices = np.where(np.abs(np.fft.fftshift(freqs)) > 0.1)[0]
        spectrum_qpe[high_freq_indices] *= 10


        return {
            'freqs': np.fft.fftshift(freqs),
            'spectrum_orig': spectrum_orig,
            'spectrum_qpe': spectrum_qpe
        }

    def _generate_amplitude_amplification_data(self):
        """
        Generates data for amplitude amplification visualization.
        """
        # Initial state on Bloch sphere
        initial_theta = np.pi * 0.6
        initial_phi = np.pi * 0.3

        # Target state
        target_theta = np.pi * 0.1
        target_phi = np.pi * 0.4

        # Amplified state
        amplified_theta = np.pi * 0.05
        amplified_phi = np.pi * 0.4

        return {
            'initial': (initial_theta, initial_phi),
            'target': (target_theta, target_phi),
            'amplified': (amplified_theta, amplified_phi),
        }

    def _generate_entanglement_entropy_data(self):
        """
        Generates data for entanglement entropy comparison.
        """
        return {
            'without_entanglement': np.random.normal(0.2, 0.05, 50),
            'with_entanglement': np.random.normal(0.8, 0.1, 50)
        }

    def plot_figure_1(self, fig):
        """
        Plots the comprehensive QCNN performance dashboard.
        """
        gs = GridSpec(3, 3, figure=fig, hspace=0.6, wspace=0.5)

        # (a) Radar Chart
        ax_a = fig.add_subplot(gs[0, 0], polar=True)
        self._plot_radar_chart(ax_a)

        # (b) Bar Chart
        ax_b = fig.add_subplot(gs[0, 1])
        self._plot_accuracy_bar_chart(ax_b)

        # (c) Training History
        ax_c = fig.add_subplot(gs[0, 2])
        self._plot_training_history(ax_c)

        # (d) 3D t-SNE
        ax_d = fig.add_subplot(gs[1, :], projection='3d')
        self._plot_3d_tsne(ax_d)

        # (e) Error Distribution
        ax_e = fig.add_subplot(gs[2, 0])
        self._plot_error_distribution(ax_e)

        # (f) Theoretical Bounds
        ax_f = fig.add_subplot(gs[2, 1])
        self._plot_theoretical_bounds(ax_f)

        # (g) Scalability
        ax_g = fig.add_subplot(gs[2, 2])
        self._plot_scalability(ax_g)

    def _plot_radar_chart(self, ax):
        """
        Plots the radar chart for model comparison.
        """
        data = self.generated_data['performance_data']
        labels = ['Accuracy (MNIST)', 'Accuracy (FashionMNIST)', '1/Qubit Cost', '1/Training Time', 'Spectral Res.']

        # Prepare data for radar chart
        models_to_plot = ['Full QCNN', 'CNN']
        stats_data = {model: [] for model in models_to_plot}
        model_indices = {model: data['models'].index(model) for model in models_to_plot}

        for model in models_to_plot:
            idx = model_indices[model]
            stats_data[model] = [
                data['mnist_accuracy'][idx],
                data['fashion_mnist_accuracy'][idx],
                1000 / data['qubit_cost'][idx] if data['qubit_cost'][idx] else 0,
                1 / data['training_time'][idx] if data['training_time'][idx] else 0,
                data['spectral_resolution'][idx] * 100 if data['spectral_resolution'][idx] else 0
            ]

        # Normalize across models for each metric
        normalized_stats = {model: [] for model in models_to_plot}
        for i in range(len(labels)):
            metric_values = [stats_data[model][i] for model in models_to_plot]
            min_val, max_val = min(metric_values), max(metric_values)
            for model in models_to_plot:
                val = stats_data[model][i]
                norm_val = (val - min_val) / (max_val - min_val) if max_val > min_val else 0.5
                normalized_stats[model].append(norm_val)

        angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
        angles += angles[:1]

        for model in models_to_plot:
            plot_stats = normalized_stats[model] + normalized_stats[model][:1]
            ax.plot(angles, plot_stats, 'o-', linewidth=2, label=model)
            ax.fill(angles, plot_stats, alpha=0.25)

        ax.set_thetagrids(np.degrees(angles[:-1]), labels)
        ax.set_title('(a) Model Performance Comparison', y=1.2)
        ax.legend(loc='upper right', bbox_to_anchor=(1.4, 1.1))


    def _plot_accuracy_bar_chart(self, ax):
        """
        Plots the accuracy bar chart.
        """
        data = self.generated_data['performance_data']
        models = data['models']
        mnist_acc = data['mnist_accuracy']
        fashion_acc = data['fashion_mnist_accuracy']

        x = np.arange(len(models))
        width = 0.35

        rects1 = ax.bar(x - width/2, mnist_acc, width, label='MNIST')
        rects2 = ax.bar(x + width/2, fashion_acc, width, label='FashionMNIST')

        ax.set_ylabel('Accuracy (%)')
        ax.set_title('(b) Model Accuracy', y=1.05)
        ax.set_xticks(x)
        ax.set_xticklabels(models, rotation=45, ha='right')
        ax.legend()
        ax.set_ylim(85, 102)
        ax.bar_label(rects1, padding=3, fmt='%.1f')
        ax.bar_label(rects2, padding=3, fmt='%.1f')

    def _plot_training_history(self, ax):
        """
        Plots the training and validation accuracy history.
        """
        data = self.generated_data['training_history']
        ax.plot(data['epochs'], data['qcnn_mnist_acc'], label='QCNN Train')
        ax.plot(data['epochs'], data['qcnn_mnist_val_acc'], label='QCNN Val', linestyle='--')
        ax.plot(data['epochs'], data['cnn_mnist_acc'], label='CNN Train')
        ax.plot(data['epochs'], data['cnn_mnist_val_acc'], label='CNN Val', linestyle='--')

        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        ax.set_title('(c) Training History (MNIST)', y=1.05)
        ax.legend(loc='lower right')
        ax.set_ylim(0.8, 1.0)

    def _plot_3d_tsne(self, ax):
        """
        Plots the 3D t-SNE of the feature space.
        """
        data = self.generated_data['feature_space_data']
        tsne_result = data['tsne_result']
        labels = data['labels']

        scatter = ax.scatter(tsne_result[:,0], tsne_result[:,1], tsne_result[:,2], c=labels, cmap='viridis', alpha=0.7)
        ax.set_title('(d) 3D t-SNE of QCNN Feature Space (MNIST)', y=1.0)
        ax.set_xlabel('Dimension 1')
        ax.set_ylabel('Dimension 2')
        ax.set_zlabel('Dimension 3')

        legend1 = ax.legend(*scatter.legend_elements(), title="Classes")
        ax.add_artist(legend1)

    def _plot_error_distribution(self, ax):
        """
        Plots the distribution of prediction confidences.
        """
        data = self.generated_data['error_distribution_data']
        sns.kdeplot(data['correct_confidence'], ax=ax, fill=True, label='Correct')
        sns.kdeplot(data['incorrect_confidence'], ax=ax, fill=True, label='Incorrect')

        ax.set_title('(e) Prediction Confidence Distribution', y=1.05)
        ax.set_xlabel('Confidence')
        ax.set_ylabel('Density')
        ax.legend()

    def _plot_theoretical_bounds(self, ax):
        """
        Plots the theoretical error and spectral resolution bounds.
        """
        data = self.generated_data['theoretical_bounds_data']
        ax.plot(data['P'], data['error_bound'], 'o-', label=r'$\epsilon(P, D)$')
        ax.set_xlabel('Padding Size (P)')
        ax.set_ylabel('Error Bound', color='blue')
        ax.tick_params(axis='y', labelcolor='blue')

        ax2 = ax.twinx()
        ax2.plot(data['P'], data['spectral_resolution'], 's-', color='red', label=r'$\Delta\xi_{\min}$')
        ax2.set_ylabel(r'Spectral Resolution ($\hbar$)', color='red')
        ax2.tick_params(axis='y', labelcolor='red')

        ax.set_title('(f) Theoretical Bounds', y=1.05)
        # Manually create legend for twin axes
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    def _plot_scalability(self, ax):
        """
        Plots the scalability of QCNN vs CNN.
        """
        data = self.generated_data['scalability_data']
        ax.plot(data['qubits'], data['qcnn_time'], 'o-', label='QCNN')
        ax.plot(data['qubits'], data['cnn_time'], 's-', label='CNN')

        ax.set_xlabel('Number of Qubits / Equivalent Data Size')
        ax.set_ylabel('Inference Time (ms)')
        ax.set_title('(g) Inference Time Scalability', y=1.05)
        ax.set_yscale('log')
        ax.legend()

    def plot_figure_2(self, fig):
        """
        Plots the in-depth analysis of quantum transformations.
        """
        gs = GridSpec(2, 3, figure=fig, hspace=0.4, wspace=0.4)

        # (a-e) Quantum Transformation Images
        ax_a1 = fig.add_subplot(gs[0, 0])
        ax_a2 = fig.add_subplot(gs[0, 1])
        ax_a3 = fig.add_subplot(gs[0, 2])
        ax_a4 = fig.add_subplot(gs[1, 0])
        ax_a5 = fig.add_subplot(gs[1, 1])
        self._plot_quantum_transformation_images([ax_a1, ax_a2, ax_a3, ax_a4, ax_a5], fig)

        # (f) Quantum Spectrum
        ax_b = fig.add_subplot(gs[1, 2])
        self._plot_quantum_spectrum(ax_b)

    def _plot_quantum_transformation_images(self, axes, fig):
        """
        Plots the images at different stages of quantum transformation.
        """
        data = self.generated_data['quantum_transformation_images']
        images = [data['original'], data['padded'], data['quc'], data['qpe'], data['aa']]
        titles = ['(a) Original', '(b) Padded', '(c) After QuC', '(d) After QPE', '(e) After AA']

        for ax, img, title in zip(axes, images, titles):
            im = ax.imshow(img, cmap='gray')
            ax.set_title(title)
            ax.axis('off')
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    def _plot_quantum_spectrum(self, ax):
        """
        Plots the quantum feature spectrum.
        """
        data = self.generated_data['quantum_spectrum_data']
        ax.plot(data['freqs'], data['spectrum_orig'], label='After QuC')
        ax.plot(data['freqs'], data['spectrum_qpe'], label='After QPE', alpha=0.7)

        ax.set_title('(f) Quantum Feature Spectrum')
        ax.set_xlabel('Frequency')
        ax.set_ylabel('Magnitude')
        ax.set_yscale('log')
        ax.legend()

    def plot_figure_3(self, fig):
        """
        Plots the ablation study and other analyses.
        """
        gs = GridSpec(2, 2, figure=fig, hspace=0.4, wspace=0.3)

        # (a) Ablation Study
        ax_a = fig.add_subplot(gs[0, 0])
        self._plot_ablation_study(ax_a)

        # (b) Striding Impact
        ax_b = fig.add_subplot(gs[0, 1])
        self._plot_striding_impact(ax_b)

        # (c) Amplitude Amplification on Bloch Sphere
        ax_c = fig.add_subplot(gs[1, 0], projection='3d')
        self._plot_amplitude_amplification(ax_c)

        # (d) Entanglement Entropy
        ax_d = fig.add_subplot(gs[1, 1])
        self._plot_entanglement_entropy(ax_d)

    def _plot_ablation_study(self, ax):
        """
        Plots the ablation study results.
        """
        data = self.generated_data['ablation_data']
        components = data['components']
        mnist_drop = data['accuracy_drop_mnist']
        fashion_drop = data['accuracy_drop_fashion']

        x = np.arange(len(components))
        width = 0.35

        rects1 = ax.bar(x - width/2, mnist_drop, width, label='MNIST')
        rects2 = ax.bar(x + width/2, fashion_drop, width, label='FashionMNIST')

        ax.set_ylabel('Accuracy Drop (%)')
        ax.set_title('(a) Ablation Study')
        ax.set_xticks(x)
        ax.set_xticklabels(components, rotation=45, ha="right")
        ax.legend()
        ax.bar_label(rects1, padding=3, fmt='%.1f')
        ax.bar_label(rects2, padding=3, fmt='%.1f')

    def _plot_striding_impact(self, ax):
        """
        Plots the impact of striding on accuracy.
        """
        data = self.generated_data['striding_data']
        ax.plot(data['stride'], data['mnist_acc_valid'], 'o-', label='MNIST Valid Padding')
        ax.plot(data['stride'], data['mnist_acc_same'], 's-', label='MNIST Same Padding')
        ax.plot(data['stride'], data['fashion_mnist_acc_valid'], 'o--', label='FashionMNIST Valid Padding')
        ax.plot(data['stride'], data['fashion_mnist_acc_same'], 's--', label='FashionMNIST Same Padding')

        ax.set_xlabel('Stride Size')
        ax.set_ylabel('Accuracy (%)')
        ax.set_title('(b) Impact of Striding on Accuracy')
        ax.legend()
        ax.set_xticks(data['stride'])

    def _plot_amplitude_amplification(self, ax):
        """
        Visualizes amplitude amplification on the Bloch sphere.
        """
        u = np.linspace(0, 2 * np.pi, 100)
        v = np.linspace(0, np.pi, 100)
        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color='lightgray', alpha=0.2, zorder=1)

        data = self.generated_data['amplitude_amplification_data']

        def plot_vec(theta, phi, color, label):
            x_v = np.sin(theta) * np.cos(phi)
            y_v = np.sin(theta) * np.sin(phi)
            z_v = np.cos(theta)
            ax.quiver(0,0,0, x_v, y_v, z_v, color=color, length=1, arrow_length_ratio=0.1, label=label, zorder=10)

        plot_vec(data['initial'][0], data['initial'][1], 'blue', 'Initial')
        plot_vec(data['target'][0], data['target'][1], 'red', 'Target')
        plot_vec(data['amplified'][0], data['amplified'][1], 'green', 'Amplified')

        ax.set_title('(c) Amplitude Amplification')
        ax.legend()
        ax.axis('off')

    def _plot_entanglement_entropy(self, ax):
        """
        Plots the entanglement entropy comparison.
        """
        data = self.generated_data['entanglement_entropy_data']
        sns.violinplot(data=[data['without_entanglement'], data['with_entanglement']], ax=ax)
        ax.set_xticklabels(['Without Entanglement', 'With Entanglement'])
        ax.set_ylabel('Entanglement Entropy (bits)')
        ax.set_title('(d) Impact of Entanglement-Enhanced Convolution')

    def generate_and_save_plots(self):
        """
        Generates all figures and saves them to a PDF file.
        """
        self.generate_all_data()

        with plt.rc_context({'figure.figsize': (18, 24)}):
            fig1 = plt.figure()
            self.plot_figure_1(fig1)

        with plt.rc_context({'figure.figsize': (15, 10)}):
            fig2 = plt.figure()
            self.plot_figure_2(fig2)

        with plt.rc_context({'figure.figsize': (12, 12)}):
            fig3 = plt.figure()
            self.plot_figure_3(fig3)

        from matplotlib.backends.backend_pdf import PdfPages
        with PdfPages(self.save_path) as pdf:
            pdf.savefig(fig1, bbox_inches='tight')
            pdf.savefig(fig2, bbox_inches='tight')
            pdf.savefig(fig3, bbox_inches='tight')

        plt.show()

# Main execution block
if __name__ == '__main__':
    plotter = QCNNManuscriptPlotter()
    # Encapsulate the final call in a try-except block for robustness
    

In [74]:
# -*- coding: utf-8 -*-
"""
This script generates a multi-panel scientific figure with complex subplots for the
results analysis section of an academic manuscript.

The plots simulate data based on the concepts and figures mentioned in the
uploaded manuscript, "Operator-Algebraic.txt". It includes visualizations of:
(a) Accuracy of QCNN models on MNIST and FashionMNIST datasets.
(b) Training and validation accuracy curves over epochs.
(c) Feature map intensity evolution after quantum operations.
(d) Spectral resolution fidelity as a function of padding size.
(e) The effect of stride size on model accuracy.
(f) Simulated quantum enhancement effects on accuracy.
(g) A normalized confusion matrix for the MNIST dataset.

The script uses standard libraries and is designed for clarity and professional presentation.
"""
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from scipy.stats import norm
from matplotlib.backends.backend_pdf import PdfPages
import os

# Set a professional plotting style using seaborn
sns.set_theme(style="whitegrid", palette="deep")
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['mathtext.fontset'] = 'cm'

# Constants and Parameters for simulation
N_EPOCHS = 20
N_SAMPLES = 1000
N_CLASSES = 10
P_VALUES = np.arange(0, 11, 1)  # Padding values from 0 to 10
STRIDE_VALUES = [1, 2, 3]

# Define a single PDF file to save all plots
output_pdf_file = "scientific_plots.pdf"
pdf_pages = PdfPages(output_pdf_file)

# ----------------------------------------------------------------------------------
# SECTION 1: DATA SIMULATION
# This section generates synthetic data to mimic the results described in the manuscript.
# ----------------------------------------------------------------------------------

def simulate_accuracy_data():
    """Simulates accuracy data for different models and datasets."""
    print("Simulating Accuracy Data...")
    
    # Simulate QCNN and classical CNN accuracy on MNIST and FashionMNIST
    # These values are based on the abstract and body of the manuscript.
    qcnn_mnist_acc = np.random.uniform(0.985, 0.99)
    qcnn_fashion_acc = np.random.uniform(0.945, 0.95)
    
    cnn_mnist_acc = np.random.uniform(0.98, 0.985)
    cnn_fashion_acc = np.random.uniform(0.93, 0.94)

    print(f"  QCNN MNIST Accuracy: {qcnn_mnist_acc:.4f}")
    print(f"  QCNN FashionMNIST Accuracy: {qcnn_fashion_acc:.4f}")
    print(f"  CNN MNIST Accuracy: {cnn_mnist_acc:.4f}")
    print(f"  CNN FashionMNIST Accuracy: {cnn_fashion_acc:.4f}")

    return {
        'model_names': ['QCNN', 'Classical CNN'],
        'mnist_acc': [qcnn_mnist_acc, cnn_mnist_acc],
        'fashion_acc': [qcnn_fashion_acc, cnn_fashion_acc]
    }

def simulate_training_curves():
    """Simulates training and validation accuracy over epochs."""
    print("Simulating Training Curves...")

    epochs = np.arange(1, N_EPOCHS + 1)
    
    # Simulate a smooth, increasing training accuracy curve
    train_acc = 1 - 0.5 * np.exp(-epochs / 5) + np.random.normal(0, 0.01, N_EPOCHS)
    val_acc = 1 - 0.55 * np.exp(-epochs / 5.5) + np.random.normal(0, 0.015, N_EPOCHS)
    
    # Ensure values are within [0, 1]
    train_acc = np.clip(train_acc, 0, 1)
    val_acc = np.clip(val_acc, 0, 1)
    
    # Simulate a late-stage divergence
    val_acc[15:] = val_acc[15:] - np.linspace(0, 0.03, 5)
    val_acc = np.clip(val_acc, 0, 1)

    print("  Epochs:", epochs)
    print("  Training Accuracy:", np.round(train_acc, 4))
    print("  Validation Accuracy:", np.round(val_acc, 4))

    return {'epochs': epochs, 'train_acc': train_acc, 'val_acc': val_acc}

def simulate_feature_map_intensity():
    """Simulates the intensity of a feature map through quantum operations."""
    print("Simulating Feature Map Intensity...")
    
    # Steps in the quantum pipeline
    steps = ['Initial State', 'Padding', 'Convolution', 'QPE', 'AA']
    
    # Simulate mean and max intensity values, showing a boost from QPE and AA
    mean_intensity = np.array([0.5, 0.55, 0.65, 0.85, 0.95])
    max_intensity = np.array([1.0, 1.0, 1.1, 1.3, 1.45])
    
    # Add some random noise
    mean_intensity += np.random.normal(0, 0.02, len(steps))
    max_intensity += np.random.normal(0, 0.05, len(steps))
    
    print("  Steps:", steps)
    print("  Mean Intensity:", np.round(mean_intensity, 4))
    print("  Max Intensity:", np.round(max_intensity, 4))

    return {'steps': steps, 'mean_intensity': mean_intensity, 'max_intensity': max_intensity}

def simulate_spectral_fidelity_data():
    print("Simulating Spectral Fidelity Data...")

    # Simulating values for the spectral resolution as a function of padding size P
    # The resolution improves (gets smaller) as P increases, but saturates.
    min_resolution = 0.018  # Based on manuscript
    zeta = 0.2
    
    # Use a modified exponential decay to model the fidelity
    resolution_fidelity = min_resolution * (1 + 5 * np.exp(-zeta * P_VALUES**2))
    
    print("  Padding Values (P):", P_VALUES)
    print("  Spectral Resolution:", np.round(resolution_fidelity, 4))

    return {'padding_values': P_VALUES, 'resolution': resolution_fidelity}

def simulate_stride_accuracy_data():
    """Simulates the effect of stride size on accuracy."""
    print("Simulating Stride Accuracy Data...")
    
    # Stride 1 should be highest accuracy, higher strides lead to information loss
    accuracies = np.array([0.987, 0.955, 0.880])
    
    print("  Stride Values:", STRIDE_VALUES)
    print("  Accuracies:", np.round(accuracies, 4))

    return {'stride_values': STRIDE_VALUES, 'accuracies': accuracies}

def simulate_quantum_enhancement():
    """Simulates the effect of QPE, AA, and Entanglement on accuracy."""
    print("Simulating Quantum Enhancement Effects...")

    baseline = np.random.uniform(0.9, 0.95, N_EPOCHS)
    
    # Simulate a progressive enhancement over the baseline
    qpe = baseline + np.random.uniform(0.01, 0.03, N_EPOCHS)
    aa = baseline + np.random.uniform(0.03, 0.05, N_EPOCHS)
    entanglement = baseline + np.random.uniform(0.05, 0.07, N_EPOCHS)
    
    print("  Baseline Acc:", np.round(baseline, 4))
    print("  QPE Enhanced Acc:", np.round(qpe, 4))
    print("  AA Enhanced Acc:", np.round(aa, 4))
    print("  Entanglement Enhanced Acc:", np.round(entanglement, 4))

    return {'baseline': baseline, 'qpe': qpe, 'aa': aa, 'entanglement': entanglement}

def simulate_confusion_matrix_data():
    """Simulates data for a confusion matrix for MNIST."""
    print("Simulating Confusion Matrix Data...")

    # True labels and predicted labels
    y_true = np.random.randint(0, N_CLASSES, N_SAMPLES)
    
    # Simulate predictions with a high accuracy
    y_pred = y_true.copy()
    num_errors = int(N_SAMPLES * 0.015)  # Simulate a 1.5% error rate
    error_indices = np.random.choice(N_SAMPLES, num_errors, replace=False)
    
    for i in error_indices:
        # Change the prediction for these samples
        y_pred[i] = np.random.choice([j for j in range(N_CLASSES) if j != y_true[i]])
    
    conf_matrix = confusion_matrix(y_true, y_pred, normalize='true')
    
    print("  Simulated Confusion Matrix (normalized):\n", np.round(conf_matrix, 4))

    return conf_matrix

def simulate_pca_data():
    """Simulates data for a PCA feature space distribution plot."""
    print("Simulating PCA Feature Space...")

    # Create synthetic feature data
    X = np.random.rand(100, 50)  # 100 samples, 50 features
    y = np.random.randint(0, 2, 100) # Two classes for simplicity

    # Apply PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    print("  Simulated PCA data shape:", X_pca.shape)

    return {'data': X_pca, 'labels': y}

# ----------------------------------------------------------------------------------
# SECTION 2: PLOTTING
# This section defines the functions for generating each subplot.
# ----------------------------------------------------------------------------------

def plot_accuracy_comparison(ax, data):
    """Plot QCNN vs Classical CNN accuracy on two datasets."""
    x = np.arange(len(data['model_names']))
    width = 0.35
    
    bar1 = ax.bar(x - width/2, data['mnist_acc'], width, label='MNIST', color=sns.color_palette("deep")[0])
    bar2 = ax.bar(x + width/2, data['fashion_acc'], width, label='FashionMNIST', color=sns.color_palette("deep")[1])
    
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title('QCNN vs. Classical CNN Accuracy', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(data['model_names'])
    ax.set_ylim(0.9, 1.0)
    ax.legend(loc='lower right', fontsize=10)
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(a)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_training_curves(ax, data):
    """Plot training and validation accuracy over epochs."""
    ax.plot(data['epochs'], data['train_acc'], label='Training Accuracy', marker='o', linestyle='-', color=sns.color_palette("deep")[0])
    ax.plot(data['epochs'], data['val_acc'], label='Validation Accuracy', marker='s', linestyle='--', color=sns.color_palette("deep")[1])
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    ax.set_ylim(0.8, 1.0)
    ax.legend(fontsize=10)
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(b)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_feature_intensity(ax, data):
    """Plot mean and max feature map intensity over quantum pipeline steps."""
    x_pos = np.arange(len(data['steps']))
    
    ax.plot(x_pos, data['mean_intensity'], marker='o', label='Mean Intensity', color=sns.color_palette("deep")[0])
    ax.plot(x_pos, data['max_intensity'], marker='s', label='Max Intensity', color=sns.color_palette("deep")[1])
    
    ax.set_ylabel('Intensity (Normalized)', fontsize=12)
    ax.set_title('Feature Map Intensity Progression', fontsize=14, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(data['steps'], rotation=45, ha='right')
    ax.set_ylim(0.0, 1.5)
    ax.legend(loc='upper left', fontsize=10)
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(c)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_spectral_fidelity(ax, data):

    ax.plot(data['padding_values'], data['resolution'], marker='o', linestyle='-', color=sns.color_palette("deep")[2])
    
    ax.set_xlabel('Padding Size $P$', fontsize=12)
    ax.set_ylabel(r'Spectral Resolution $\Delta\xi_{\min}$ (a.u.)', fontsize=12)
    ax.set_title('Spectral Resolution vs. Padding Size', fontsize=14, fontweight='bold')
    ax.set_ylim(0, 0.1)
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(d)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_stride_effect(ax, data):
  
    ax.bar(data['stride_values'], data['accuracies'], color=sns.color_palette("deep")[3])
    
    ax.set_xlabel('Stride Size', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title('Effect of Stride Size on Accuracy', fontsize=14, fontweight='bold')
    ax.set_xticks(data['stride_values'])
    ax.set_ylim(0.8, 1.0)
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(e)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_quantum_enhancement(ax, data):
  
    epochs = np.arange(1, N_EPOCHS + 1)
    ax.plot(epochs, data['baseline'], label='Baseline QCNN', marker='.', linestyle='--', color='gray')
    ax.plot(epochs, data['qpe'], label='QPE Enhancement', marker='o', linestyle='-', color=sns.color_palette("deep")[0])
    ax.plot(epochs, data['aa'], label='AA Enhancement', marker='s', linestyle='-', color=sns.color_palette("deep")[1])
    ax.plot(epochs, data['entanglement'], label='Entanglement Enhancement', marker='^', linestyle='-', color=sns.color_palette("deep")[2])
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title('Simulated Quantum Enhancements', fontsize=14, fontweight='bold')
    ax.set_ylim(0.9, 1.05)
    ax.legend(fontsize=8, loc='lower right')
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(f)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_confusion_matrix(ax, data):
  
    sns.heatmap(data, annot=True, fmt=".2f", cmap="Blues", ax=ax, cbar=False)
    
    ax.set_title('Normalized Confusion Matrix (MNIST)', fontsize=14, fontweight='bold')
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=12)
    ax.set_xticklabels(np.arange(N_CLASSES))
    ax.set_yticklabels(np.arange(N_CLASSES), rotation=0)
    ax.text(0.02, 0.9, '(g)', transform=ax.transAxes, fontsize=16, fontweight='bold')

def plot_pca_distribution(ax, data):
  
    ax.scatter(data['data'][:, 0], data['data'][:, 1], c=data['labels'], cmap='viridis', s=20)
    
    ax.set_xlabel('Principal Component 1', fontsize=12)
    ax.set_ylabel('Principal Component 2', fontsize=12)
    ax.set_title('Feature Space Distribution (PCA)', fontsize=14, fontweight='bold')
    ax.spines[['right', 'top']].set_visible(False)
    ax.text(0.02, 0.9, '(h)', transform=ax.transAxes, fontsize=16, fontweight='bold')

# ----------------------------------------------------------------------------------
# SECTION 3: MAIN EXECUTION FLOW
# Orchestrates data simulation, plotting, and saving to a PDF.
# ----------------------------------------------------------------------------------

def create_complex_figure():
  
    
    # Simulate all the necessary data
    accuracy_data = simulate_accuracy_data()
    training_data = simulate_training_curves()
    feature_intensity_data = simulate_feature_map_intensity()
    spectral_fidelity_data = simulate_spectral_fidelity_data()
    stride_accuracy_data = simulate_stride_accuracy_data()
    enhancement_data = simulate_quantum_enhancement()
    confusion_matrix_data = simulate_confusion_matrix_data()
    pca_data = simulate_pca_data()

    # Create the main figure and subplots using a complex grid
    fig = plt.figure(figsize=(15, 12), constrained_layout=True)
    gs = gridspec.GridSpec(4, 4, figure=fig)

    # Subplot (a): Accuracy Comparison
    ax1 = fig.add_subplot(gs[0, :2])
    plot_accuracy_comparison(ax1, accuracy_data)

    # Subplot (b): Training Curves
    ax2 = fig.add_subplot(gs[0, 2:])
    plot_training_curves(ax2, training_data)

    # Subplot (c): Feature Map Intensity
    ax3 = fig.add_subplot(gs[1, 0])
    plot_feature_intensity(ax3, feature_intensity_data)

    # Subplot (d): Spectral Fidelity
    ax4 = fig.add_subplot(gs[1, 1])
    plot_spectral_fidelity(ax4, spectral_fidelity_data)
    
    # Subplot (e): Stride Effect
    ax5 = fig.add_subplot(gs[1, 2])
    plot_stride_effect(ax5, stride_accuracy_data)

    # Subplot (f): Quantum Enhancements
    ax6 = fig.add_subplot(gs[1, 3])
    plot_quantum_enhancement(ax6, enhancement_data)
    
    # Subplot (g): Confusion Matrix
    ax7 = fig.add_subplot(gs[2:, 2:])
    plot_confusion_matrix(ax7, confusion_matrix_data)

    # Subplot (h): PCA Feature Distribution
    ax8 = fig.add_subplot(gs[2:, :2])
    plot_pca_distribution(ax8, pca_data)
    
    fig.suptitle('Figure 1: Comprehensive Results Analysis of QCNN Model', fontsize=16, fontweight='bold')
    
    # Adjust layout and save the figure to the PDF file
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.close(fig)

if __name__ == '__main__':
    print("Starting complex scientific plotting script...")
    create_complex_figure()
    pdf_pages.close()
    print(f"\nAll plots have been successfully saved to '{output_pdf_file}'.")


Starting complex scientific plotting script...
Simulating Accuracy Data...
  QCNN MNIST Accuracy: 0.9900
  QCNN FashionMNIST Accuracy: 0.9465
  CNN MNIST Accuracy: 0.9813
  CNN FashionMNIST Accuracy: 0.9312
Simulating Training Curves...
  Epochs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
  Training Accuracy: [0.5826 0.6667 0.727  0.7762 0.8071 0.8507 0.8896 0.902  0.9281 0.9334
 0.9382 0.9632 0.9488 0.964  0.9737 0.9861 0.9961 0.9948 0.9963 0.9995]
  Validation Accuracy: [0.5565 0.6006 0.7143 0.7509 0.7757 0.8208 0.8263 0.8677 0.8878 0.9044
 0.9378 0.9395 0.9303 0.9392 0.9346 0.9832 0.9641 0.9755 0.9775 0.942 ]
Simulating Feature Map Intensity...
  Steps: ['Initial State', 'Padding', 'Convolution', 'QPE', 'AA']
  Mean Intensity: [0.5065 0.5789 0.6402 0.8432 0.9606]
  Max Intensity: [1.1271 1.0318 1.0579 1.2639 1.4256]
Simulating Spectral Fidelity Data...
  Padding Values (P): [ 0  1  2  3  4  5  6  7  8  9 10]
  Spectral Resolution: [0.108  0.0917 0.0584 0.0329 0.02

In [1]:
# qcnn_results_subplots.py
# Save this file and run with python3
!pip install --upgrade pip
!pip install --upgrade tensorflow keras

import os
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import MaxNLocator
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from scipy.fft import fft2, ifft2, fftshift
import tensorflow as tf
from tensorflow.keras.datasets import mnist, fashion_mnist
from PIL import Image
import itertools

# -----------------------------
# Utility functions
# -----------------------------
def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)

def norm01(x):
    mn = np.min(x)
    mx = np.max(x)
    if mx - mn == 0:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def pad_image(img, P=3, mode='cyclic'):
    # img: 2D array
    if mode == 'cyclic':
        return np.pad(img, ((P,P),(P,P)), mode='wrap')
    elif mode == 'symmetric':
        return np.pad(img, ((P,P),(P,P)), mode='symmetric')
    else:
        return np.pad(img, ((P,P),(P,P)), mode='constant')

# AA amplification formula (manuscript): sin((2m+1) * theta)
def aa_amplitude(m, theta):
    return math.sin((2*m + 1) * theta)

# QPE amplitude spike synthetic model: a few exponential peaks across frequency bins
def synthetic_qpe_spectrum(bins=128, peak_locations=None, heights=None, widths=None):
    x = np.linspace(0, 1, bins)
    y = np.zeros_like(x)
    if peak_locations is None:
        peak_locations = [0.12, 0.44, 0.78]
    if heights is None:
        heights = [1.0, 2.2, 1.6]
    if widths is None:
        widths = [0.02, 0.01, 0.015]
    for loc,h,w in zip(peak_locations, heights, widths):
        y += h * np.exp(-0.5 * ((x - loc)/w)**2)
    # add a baseline noise floor
    y += 0.05 * np.random.rand(len(x))
    return x, y

# Frequency-domain "Hadamard-like" elementwise product demonstration
def freq_hadamard_demo(img, kern):
    Fimg = fftshift(fft2(img))
    Fkern = fftshift(fft2(kern, s=img.shape))
    # elementwise multiply in frequency domain (Hadamard-like)
    Frecon = Fimg * Fkern
    recon = np.real(ifft2(fftshift(Frecon)))
    return Fimg, Fkern, Frecon, recon

# Pretty label helper for subplots
def label_ax(ax, label, xpos=-0.05, ypos=1.08, weight='bold'):
    ax.text(xpos, ypos, label, transform=ax.transAxes,
            fontsize=12, fontweight=weight, va='top')

# -----------------------------
# Load datasets (MNIST & FashionMNIST)
# -----------------------------
print("Loading MNIST and FashionMNIST...")
(x1_train, y1_train), (x1_test, y1_test) = mnist.load_data()
(x2_train, y2_train), (x2_test, y2_test) = fashion_mnist.load_data()

# Convert to float [0,1]
x1_train = x1_train.astype(np.float32) / 255.0
x1_test  = x1_test.astype(np.float32) / 255.0
x2_train = x2_train.astype(np.float32) / 255.0
x2_test  = x2_test.astype(np.float32) / 255.0

# select small representative subsets for plotting to speed computation
n_samples_display = 5
mnist_samples = x1_train[:n_samples_display]
fashion_samples = x2_train[:n_samples_display]

# -----------------------------
# Generate values based on manuscript equations and settings
# -----------------------------
# 1) Spectral resolution bound vs P (Eq similar to manuscript: Delta_xi_min >= const * ||[...]||^-1 * e^{-zeta P^2})
zeta = 0.25
kernel_norm = 2.5  # approximate from manuscript notes
hbar = 1.0
P_vals = np.arange(0, 9)  # padding sizes 0..8
Delta_xi_min = (hbar / 2.0) * (1.0 / kernel_norm) * np.exp(-zeta * (P_vals**2))

# 2) Fidelity vs padding P (epsilon(P) = exp(-kappa P^2) + zeta/(product D^alpha))
kappa = 0.5
zeta_fid = 1e-2
Dprod = 28**2  # very rough proxy spatial dims product
alpha = 1.0
epsilon_P = np.exp(-kappa * P_vals**2) + zeta_fid / (Dprod**alpha)
fidelity = 1.0 - epsilon_P

# 3) QPE synthetic spectrum (simulate QPE amplitude spikes)
bins = 256
qpe_x, qpe_y = synthetic_qpe_spectrum(bins=bins, peak_locations=[0.15, 0.5, 0.82],
                                     heights=[3.2, 72.37, 18.0], widths=[0.012, 0.004, 0.01])

# 4) AA amplification: choose theta from manuscript (theta = arcsin(1/sqrt(8)) for example)
theta = math.asin(1.0 / math.sqrt(8.0))
m_iters = np.arange(0, 7)  # 0..6 iterations
aa_vals = [aa_amplitude(m, theta) for m in m_iters]

# 5) Intensity transforms (manuscript suggests QPE amplifies intensity)
# We'll compute mean intensity across sample images for original, padded, qpe-augmented (synthetic)
def compute_mean_max(images):
    ms = [np.mean(img) for img in images]
    Mx = [np.max(img) for img in images]
    return np.mean(ms), np.mean(Mx)

mnist_mean, mnist_max = compute_mean_max(mnist_samples)
fashion_mean, fashion_max = compute_mean_max(fashion_samples)
# synthetic QPE intensity scale factors (from file, e.g., 72.4 for MNIST QPE max)
qpe_gain_mnist_mean = 3.71  # from manuscript
qpe_gain_mnist_max = 72.37
qpe_gain_fashion_mean = 3.42
qpe_gain_fashion_max = 131.2

# 6) Feature-space t-SNE: produce a small set of vectors (we'll use flattened images as 'features')
X_tsne = np.vstack([x1_train[:500].reshape(500, -1), x2_train[:500].reshape(500, -1)])
labels_tsne = np.hstack([y1_train[:500], y2_train[:500] + 10])  # keep classes separate for coloring
tsne = TSNE(n_components=2, perplexity=40, init='random', random_state=0, learning_rate=200)
X_embedded = tsne.fit_transform(X_tsne)

# 7) Synthetic "training" curves and uncertainty for panel of learning dynamics
epochs = np.arange(1, 51)
train_acc = 0.5 + 0.48 * (1 - np.exp(-0.12 * (epochs - 1)))  # asymptotic to ~0.98
val_acc = train_acc - 0.02 - 0.06 * np.exp(-0.08 * (epochs - 1))
quantum_entropy = 4.9 - 0.02 * epochs + 0.1 * np.sin(0.25 * epochs)  # synthetic tracking metric
ensemble_loss_mean = 1.2 * np.exp(-0.06 * epochs) + 0.05
ensemble_loss_std = 0.12 * np.exp(-0.03 * epochs) + 0.02

# 8) Confusion matrix for synthetic predictions on small test sample (we simulate a classifier)
np.random.seed(0)
y_true = y2_test[:200]  # fashion test labels
# simulate predicted labels with some confusion
y_pred = y_true.copy()
# add errors: flip 12% randomly
flip_idx = np.random.choice(len(y_true), size=int(0.12 * len(y_true)), replace=False)
for idx in flip_idx:
    y_pred[idx] = (y_pred[idx] + np.random.randint(1,10)) % 10
cm = confusion_matrix(y_true, y_pred)

# 9) Frequency domain Hadamard demo for an example MNIST image and a synthetic kernel
sample_img = x1_train[3]
kernel = np.zeros((7,7))
kernel[3,3] = 1.0
# create a smoothed kernel
for i in range(7):
    for j in range(7):
        kernel[i,j] = math.exp(-((i-3)**2 + (j-3)**2) / (2*1.5**2))
kernel = kernel / np.sum(kernel)
Fimg, Fkern, Frecon, recon_img = freq_hadamard_demo(sample_img, kernel)

# -----------------------------
# Print numeric values used (so authors can paste them into manuscript)
# -----------------------------
print("\n--- Numeric values used for plotting (printed arrays saved to qcnn_plot_values.npz) ---")
print("P_vals:", P_vals)
print("Delta_xi_min:", np.round(Delta_xi_min, 6))
print("fidelity (1 - epsilon):", np.round(fidelity, 6))
print("qpe spectrum sample (first 10 y values):", np.round(qpe_y[:10], 6))
print("aa_vals for m=0..6:", np.round(aa_vals, 6))
print("mnist_mean, mnist_max:", mnist_mean, mnist_max)
print("fashion_mean, fashion_max:", fashion_mean, fashion_max)
print("synthetic training final train_acc[-1]:", train_acc[-1], "val_acc[-1]:", val_acc[-1])
print("ensemble loss mean/ std (epoch 50):", ensemble_loss_mean[-1], ensemble_loss_std[-1])
print("confusion matrix shape:", cm.shape)

# Save arrays used
np.savez('qcnn_plot_values.npz',
         P_vals=P_vals,
         Delta_xi_min=Delta_xi_min,
         fidelity=fidelity,
         qpe_x=qpe_x, qpe_y=qpe_y,
         aa_m_iters=m_iters, aa_vals=aa_vals,
         mnist_mean=mnist_mean, mnist_max=mnist_max,
         fashion_mean=fashion_mean, fashion_max=fashion_max,
         X_embedded=X_embedded, labels_tsne=labels_tsne,
         epochs=epochs, train_acc=train_acc, val_acc=val_acc, quantum_entropy=quantum_entropy,
         ensemble_loss_mean=ensemble_loss_mean, ensemble_loss_std=ensemble_loss_std,
         confusion_matrix=cm,
         Fimg=np.abs(Fimg), Fkern=np.abs(Fkern), Frecon=np.abs(Frecon), recon_img=recon_img)

# -----------------------------
# Create multipanel figure
# -----------------------------
matplotlib.rcParams.update({'font.size': 10, 'figure.dpi': 150})
fig = plt.figure(figsize=(11.7, 16.5))  # A4 portrait like layout
gs = gridspec.GridSpec(6, 2, height_ratios=[1, 1, 1, 1, 1, 1], hspace=0.45, wspace=0.32)

# Panel (a): representative samples MNIST & Fashion side-by-side
ax = fig.add_subplot(gs[0, 0])
images_to_show = list(mnist_samples) + list(fashion_samples)
titles = ['MNIST {}'.format(i) for i in range(len(mnist_samples))] + ['Fash {}'.format(i) for i in range(len(fashion_samples))]
ncols = 5
for i, img in enumerate(images_to_show):
    sub = ax.inset_axes([i*0.18, 0.02, 0.16, 0.96]) if i < ncols else None
# We'll plot a grid manually
ax.clear()
rows = 2
cols = ncols
for i, img in enumerate(images_to_show):
    r = i // cols
    c = i % cols
    ax_img = fig.add_axes([0.06 + c*0.18, 0.90 - r*0.115, 0.14, 0.1])
    ax_img.imshow(img, cmap='gray', interpolation='nearest')
    ax_img.set_xticks([])
    ax_img.set_yticks([])
    if i < n_samples_display:
        ax_img.set_title("MN {}".format(i), fontsize=8)
    else:
        ax_img.set_title("F {}".format(i - n_samples_display), fontsize=8)
# frame placeholder
ax.axis('off')
label_ax(ax, '(a) Representative samples (MNIST, FashionMNIST)')

# Panel (b): intensity means / maxs across transformations
ax = fig.add_subplot(gs[0, 1])
bars = ['Original MN mean', 'QPE MN mean (est)', 'Original FM mean', 'QPE FM mean (est)']
vals = [mnist_mean, mnist_mean * qpe_gain_mnist_mean, fashion_mean, fashion_mean * qpe_gain_fashion_mean]
ax.bar(range(len(vals)), vals)
ax.set_xticks(range(len(vals)))
ax.set_xticklabels(bars, rotation=20, ha='right')
ax.set_ylabel('Mean intensity')
ax.set_title('Intensity transforms under QPE (synthetic amplification)')
label_ax(ax, '(b)')

# Panel (c): QPE spectrum (large spike demonstration)
ax = fig.add_subplot(gs[1, 0])
ax.plot(qpe_x, qpe_y, lw=1.2)
ax.set_xlabel('Normalized frequency')
ax.set_ylabel('QPE amplitude (arb.)')
ax.set_title('Synthetic QPE spectrum (illustrative of phase amplification)')
ax.grid(True, alpha=0.25)
label_ax(ax, '(c)')

# Panel (d): AA amplitude vs iterations
ax = fig.add_subplot(gs[1, 1])
ax.plot(m_iters, aa_vals, marker='o', lw=1.5)
ax.set_xlabel('Grover iterations m')
ax.set_ylabel('Amplitude |sin((2m+1)θ)|')
ax.set_title('Amplitude Amplification (AA) behavior using θ from manuscript')
ax.set_xticks(m_iters)
ax.grid(True)
label_ax(ax, '(d)')

# Panel (e): Fidelity vs padding (tradeoff plot)
ax = fig.add_subplot(gs[2, 0])
ax.plot(P_vals, fidelity, marker='s', label='Fidelity')
ax.plot(P_vals, Delta_xi_min / np.max(Delta_xi_min), '--', label='Normed spectral resolution (Δξ_min)')
ax.set_xlabel('Padding P')
ax.set_ylabel('Fidelity / Normed Δξ_min')
ax.set_title('Fidelity and spectral resolution vs padding (manuscript model)')
ax.legend(frameon=False)
ax.grid(True)
label_ax(ax, '(e)')

# Panel (f): t-SNE of flattened 500+500 features (entangled vs classical style)
ax = fig.add_subplot(gs[2, 1])
cmap = plt.get_cmap('tab20')
ax.scatter(X_embedded[:500,0], X_embedded[:500,1], s=8, c='tab:blue', alpha=0.7, label='MNIST subset')
ax.scatter(X_embedded[500:,0], X_embedded[500:,1], s=8, c='tab:orange', alpha=0.7, label='Fashion subset')
ax.set_title('t-SNE visualization of combined feature sets (500 each)')
ax.legend(markerscale=2)
label_ax(ax, '(f)')

# Panel (g): Training curves + quantum entropy overlay
ax = fig.add_subplot(gs[3, 0])
ax.plot(epochs, train_acc, label='Train accuracy', lw=1.5)
ax.plot(epochs, val_acc, label='Val accuracy', lw=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax_twin = ax.twinx()
ax_twin.plot(epochs, quantum_entropy, label='Quantum entropy (scaled)', color='green', lw=1.0, alpha=0.7)
ax_twin.set_ylabel('Quantum entropy (bits)')
ax.set_title('Training dynamics and quantum entropy')
ax.legend(loc='upper left')
ax_twin.legend(loc='upper right')
label_ax(ax, '(g)')

# Panel (h): Loss convergence with ensemble uncertainty (shaded region)
ax = fig.add_subplot(gs[3, 1])
ax.plot(epochs, ensemble_loss_mean, label='Ensemble val loss (mean)')
ax.fill_between(epochs, ensemble_loss_mean - ensemble_loss_std, ensemble_loss_mean + ensemble_loss_std, alpha=0.25)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Loss convergence with ensemble uncertainty')
ax.grid(True)
label_ax(ax, '(h)')

# Panel (i): Feature variance vs accuracy synthetic coevolution
ax = fig.add_subplot(gs[4, 0])
feature_variance = 0.4 + 0.6 * (1 - np.exp(-0.1 * (epochs - 1)))
ax.plot(epochs, feature_variance, label='Feature variance (σ²_f)')
ax.plot(epochs, train_acc, label='Train acc (for reference)', linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Metric')
ax.set_title('Feature variance — accuracy coevolution (synthetic)')
ax.legend()
label_ax(ax, '(i)')

# Panel (j): Confusion matrix (fashion sample)
ax = fig.add_subplot(gs[4, 1])
im = ax.imshow(cm, interpolation='nearest', cmap='viridis')
ax.set_title('Simulated confusion matrix (FashionMNIST subset)')
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(j)')

# Panel (k): Frequency-domain Hadamard demo images (Fimg, Fkern magnitude)
ax = fig.add_subplot(gs[5, 0])
ax.imshow(np.log1p(np.abs(Fimg)), cmap='inferno')
ax.set_title('log|FFT(image)|')
ax.axis('off')
label_ax(ax, '(k)')

# Panel (l): Reconstructed image after freq hadamard product (recon)
ax = fig.add_subplot(gs[5, 1])
ax.imshow(norm01(recon_img), cmap='gray')
ax.set_title('Reconstruction after frequency elementwise product (Hadamard-like)')
ax.axis('off')
label_ax(ax, '(l)')

# Save and finalize
plt.suptitle('Complex scientific subplots for QCNN Results Analysis — multipanel', fontsize=14, y=0.995)
plt.savefig(out_pdf, bbox_inches='tight', dpi=300)
np.savez('qcnn_plot_values.npz')  # already saved above, call again (idempotent)
print("Saved plotting arrays to qcnn_plot_values.npz")
plt.close(fig)


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 648.1 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.8 MB 655.4 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.8 MB 774.0 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.8 MB 784.3 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.8 MB 808.4 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 809.2 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.8 MB 819.2 kB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.8 MB 849.3 kB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.8 MB 873.3 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.8 MB 893.0 kB/s eta 0:00:02
   ------------ --------------------------- 0.6/1.8 MB 913.7 kB/s eta 0:00:02
   -

ERROR: To modify pip, please run the following command:
C:\Users\HP\anaconda3\python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.5 which is incompatible.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.19.0 which is incompatible.


   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/376.0 MB 1.4 MB/s eta 0:04:36
   ---------------------------------------- 0.1/376.0 MB 656.4 kB/s eta 0:09:33
   ---------------------------------------- 0.1/376.0 MB 901.1 kB/s eta 0:06:58
   ---------------------------------------- 0.2/376.0 MB 893.0 kB/s eta 0:07:01
   ---------------------------------------- 0.2/376.0 MB 831.5 kB/s eta 0:07:32
   ---------------------------------------- 0.3/376.0 MB 930.9 kB/s eta 0:06:44
   ---------------------------------------- 0.4/376.0 MB 1.0 MB/s eta 0:06:01
   ---------------------------------------- 0.5/376.0 MB 1.1 MB/s eta 0:05:33
   ---------------------------------------- 0.5/376.0 MB 1.1 MB/s eta 0:05:30
   ---------------------------------------- 0.5/376.0 MB 1.1 MB/s eta 0:05:38
   ---------------------------------------- 0.6/376.0 MB 1.1 MB/s eta 0:05:30
   ---------------------------------------- 0.7/376.0 MB 1.2 

In [1]:
# qcnn_results_subplots_pytorch_expanded.py
# Large multipanel figure (20 panels) for manuscript Results analysis.
# Uses PyTorch / torchvision for MNIST & FashionMNIST loading.
# Saves PDF and numeric arrays (.npz).
# Run with Python 3.8+ (works on 3.11 too).

import os
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import MaxNLocator
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy.fft import fft2, ifft2, fftshift
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import random
import warnings
warnings.filterwarnings("ignore")

# -------------------------------------
# Utilities
# -------------------------------------
def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)

def norm01(x):
    mn = np.min(x)
    mx = np.max(x)
    if mx - mn == 0:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def pad_image(img, P=3, mode='cyclic'):
    if mode == 'cyclic':
        return np.pad(img, ((P,P),(P,P)), mode='wrap')
    elif mode == 'symmetric':
        return np.pad(img, ((P,P),(P,P)), mode='symmetric')
    else:
        return np.pad(img, ((P,P),(P,P)), mode='constant')

def aa_amplitude(m, theta):
    return math.sin((2*m + 1) * theta)

def synthetic_qpe_spectrum(bins=256, peak_locations=None, heights=None, widths=None):
    x = np.linspace(0, 1, bins)
    y = np.zeros_like(x)
    if peak_locations is None:
        peak_locations = [0.12, 0.44, 0.78]
    if heights is None:
        heights = [1.0, 2.2, 1.6]
    if widths is None:
        widths = [0.02, 0.01, 0.015]
    for loc,h,w in zip(peak_locations, heights, widths):
        y += h * np.exp(-0.5 * ((x - loc)/w)**2)
    y += 0.02 * np.random.rand(len(x))
    return x, y

def freq_hadamard_demo(img, kern):
    Fimg = fftshift(fft2(img))
    Fkern = fftshift(fft2(kern, s=img.shape))
    Frecon = Fimg * Fkern
    recon = np.real(ifft2(fftshift(Frecon)))
    return Fimg, Fkern, Frecon, recon

def label_ax(ax, label, xpos=-0.05, ypos=1.08, weight='bold'):
    ax.text(xpos, ypos, label, transform=ax.transAxes,
            fontsize=11, fontweight=weight, va='top')

# -------------------------------------
# Reproducibility
# -------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

# -------------------------------------
# Load datasets with torchvision
# -------------------------------------
print("Downloading/loading MNIST and FashionMNIST via torchvision (if not present)...")
transform = transforms.Compose([transforms.ToTensor()])  # yields [0,1] tensors

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
fashion_train = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
fashion_test  = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

def dataset_to_numpy(ds):
    xs = []
    ys = []
    for i in range(len(ds)):
        img, lab = ds[i]
        # img shape [1,H,W] -> numpy HxW
        xs.append(img.squeeze(0).numpy())
        ys.append(int(lab))
    return np.stack(xs, axis=0), np.array(ys, dtype=np.int64)

x1_train, y1_train = dataset_to_numpy(mnist_train)
x1_test,  y1_test  = dataset_to_numpy(mnist_test)
x2_train, y2_train = dataset_to_numpy(fashion_train)
x2_test,  y2_test  = dataset_to_numpy(fashion_test)

# small display subsets
n_samples_display = 5
mnist_samples = x1_train[:n_samples_display]
fashion_samples = x2_train[:n_samples_display]
# Correctly define images_to_show
images_to_show = np.vstack([mnist_samples, fashion_samples])

# -------------------------------------
# Manuscript-related derived metrics and synthetic models
# -------------------------------------
# Spectral resolution bound vs padding P
zeta = 0.25
kernel_norm = 2.5
hbar = 1.0
P_vals = np.arange(0, 11)  # padding 0..10
Delta_xi_min = (hbar / 2.0) * (1.0 / kernel_norm) * np.exp(-zeta * (P_vals**2))

# Fidelity model vs padding
kappa = 0.5
zeta_fid = 1e-2
Dprod = 28**2
alpha = 1.0
epsilon_P = np.exp(-kappa * P_vals**2) + zeta_fid / (Dprod**alpha)
fidelity = 1.0 - epsilon_P

# QPE synthetic spectrum
bins = 512
qpe_x, qpe_y = synthetic_qpe_spectrum(bins=bins, peak_locations=[0.15, 0.5, 0.82],
                                     heights=[3.2, 72.37, 18.0], widths=[0.012, 0.004, 0.01])

# AA amplitude
theta = math.asin(1.0 / math.sqrt(8.0))
m_iters = np.arange(0, 11)
aa_vals = np.array([aa_amplitude(m, theta) for m in m_iters])

# Intensity measures
def compute_mean_max(images):
    ms = [np.mean(img) for img in images]
    Mx = [np.max(img) for img in images]
    return np.mean(ms), np.mean(Mx)

mnist_mean, mnist_max = compute_mean_max(mnist_samples)
fashion_mean, fashion_max = compute_mean_max(fashion_samples)
qpe_gain_mnist_mean = 3.71
qpe_gain_mnist_max = 72.37
qpe_gain_fashion_mean = 3.42
qpe_gain_fashion_max = 131.2

# t-SNE for 800 points (400 each) to keep runtime reasonable
n_embed_per = 400
X_tsne_raw = np.vstack([x1_train[:n_embed_per].reshape(n_embed_per, -1), x2_train[:n_embed_per].reshape(n_embed_per, -1)])
labels_tsne = np.hstack([y1_train[:n_embed_per], y2_train[:n_embed_per] + 10])
tsne = TSNE(n_components=2, perplexity=30, init='pca', random_state=SEED, learning_rate=200)
X_embedded = tsne.fit_transform(X_tsne_raw)

# synthetic training-like dynamics
epochs = np.arange(1, 101)
train_acc = 0.48 + 0.5 * (1 - np.exp(-0.08 * (epochs - 1)))
val_acc = train_acc - 0.02 - 0.05 * np.exp(-0.06 * (epochs - 1))
quantum_entropy = 5.0 - 0.02 * epochs + 0.15 * np.sin(0.18 * epochs)
ensemble_loss_mean = 1.6 * np.exp(-0.04 * epochs) + 0.02
ensemble_loss_std  = 0.18 * np.exp(-0.03 * epochs) + 0.02

# Confusion matrix on small subset (simulate classifier)
np.random.seed(SEED)
y_true = y2_test[:300]
y_pred = y_true.copy()
flip_idx = np.random.choice(len(y_true), size=int(0.14 * len(y_true)), replace=False)
for idx in flip_idx:
    y_pred[idx] = (y_pred[idx] + np.random.randint(1,10)) % 10
cm = confusion_matrix(y_true, y_pred)

# Frequency-domain hadamard demo
sample_img = x1_train[7]
kernel = np.zeros((9,9))
for i in range(9):
    for j in range(9):
        kernel[i,j] = math.exp(-((i-4)**2 + (j-4)**2) / (2*2.0**2))
kernel = kernel / np.sum(kernel)
Fimg, Fkern, Frecon, recon_img = freq_hadamard_demo(sample_img, kernel)

# Per-class mean images for MNIST and FashionMNIST (first 10 classes each)
mnist_class_means = np.array([np.mean(x1_train[y1_train == cls], axis=0) for cls in range(10)])
fashion_class_means = np.array([np.mean(x2_train[y2_train == cls], axis=0) for cls in range(10)])

# Inter-class cosine similarity matrix (on flattened class means combined)
combined_means = np.vstack([mnist_class_means.reshape(10, -1), fashion_class_means.reshape(10, -1)])
norms = np.linalg.norm(combined_means, axis=1, keepdims=True)
combined_normed = combined_means / (norms + 1e-12)
cosine_sim = combined_normed @ combined_normed.T  # 20x20

# Kernel spectral profiles: compare three kernels (gaussian small, gaussian wide, laplacian-like)
def make_gaussian_kernel(size, sigma):
    k = np.fromfunction(lambda i, j: np.exp(-(((i-(size-1)/2)**2 + (j-(size-1)/2)**2) / (2*sigma**2))), (size, size))
    return k / np.sum(k)

kern1 = make_gaussian_kernel(7, sigma=0.8)
kern2 = make_gaussian_kernel(11, sigma=2.2)
# laplacian-like
kern3 = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]], dtype=float)
kern3 = kern3 / np.sum(np.abs(kern3))
kernels = [kern1, kern2, kern3]

# Correlation between metrics
metric_matrix = np.vstack([fidelity[:len(P_vals)], Delta_xi_min, np.interp(P_vals, [P_vals.min(), P_vals.max()], [aa_vals.min(), aa_vals.max()]), np.interp(P_vals, [P_vals.min(), P_vals.max()], [quantum_entropy.min(), quantum_entropy.max()])])
metric_corr = np.corrcoef(metric_matrix)

# PCA explained variance (flatten small sample)
X_pca_sample = X_tsne_raw[:1000]  # 1000 samples
pca = PCA(n_components=50, random_state=SEED)
pca.fit(X_pca_sample)
explained_var = pca.explained_variance_ratio_

# Class-wise sample variance: compute pixel variance for 10 fashion classes on small subset
class_vars = []
for cls in range(10):
    samples = x2_train[y2_train == cls][:200]
    pixel_vars = np.var(samples.reshape(samples.shape[0], -1), axis=1)
    class_vars.append(pixel_vars)
# For boxplot need list of arrays per class

# ROC-like synthetic curves: pick 3 classes from Fashion, do one-vs-rest using a synthetic score
classes_for_roc = [0, 1, 2]
y_true_subset = y2_test[:800]
# Create synthetic "scores" by adding small noise to a ground-truth signal
scores = np.zeros((len(y_true_subset), len(classes_for_roc)))
for i, cls in enumerate(classes_for_roc):
    base = (y_true_subset == cls).astype(float)
    scores[:, i] = base * (0.7 + 0.25 * np.random.rand(len(base))) + (1 - base) * (0.3 * np.random.rand(len(base)))

# -------------------------------------
# Print numeric arrays summary (for manuscript appendices)
# -------------------------------------
print("\n--- Numeric values summary (saved to qcnn_plot_values_expanded.npz) ---")
print("P_vals:", P_vals)
print("Delta_xi_min (rounded):", np.round(Delta_xi_min, 6))
print("fidelity (rounded):", np.round(fidelity, 6))
print("qpe_y sample (first 12):", np.round(qpe_y[:12], 6))
print("aa_vals (m=0..10):", np.round(aa_vals, 6))
print("mnist_mean, fashion_mean:", np.round(mnist_mean,6), np.round(fashion_mean,6))
print("combined cosine_sim shape:", cosine_sim.shape)
print("explained_var first 10:", np.round(explained_var[:10], 5))
print("confusion matrix shape:", cm.shape)

np.savez('qcnn_plot_values_expanded.npz',
          P_vals=P_vals,
          Delta_xi_min=Delta_xi_min,
          fidelity=fidelity,
          qpe_x=qpe_x, qpe_y=qpe_y,
          aa_m_iters=m_iters, aa_vals=aa_vals,
          mnist_mean=mnist_mean, mnist_max=mnist_max,
          fashion_mean=fashion_mean, fashion_max=fashion_max,
          X_embedded=X_embedded, labels_tsne=labels_tsne,
          epochs=epochs, train_acc=train_acc, val_acc=val_acc, quantum_entropy=quantum_entropy,
          ensemble_loss_mean=ensemble_loss_mean, ensemble_loss_std=ensemble_loss_std,
          confusion_matrix=cm,
          Fimg=np.abs(Fimg), Fkern=np.abs(Fkern), Frecon=np.abs(Frecon), recon_img=recon_img,
          mnist_class_means=mnist_class_means, fashion_class_means=fashion_class_means,
          cosine_sim=cosine_sim, kernels=np.array(kernels, dtype=object),
          metric_corr=metric_corr, explained_var=explained_var,
          class_vars=np.array(class_vars, dtype=object),
          roc_scores=scores, y_true_subset=y_true_subset)


print("Saved numeric arrays to qcnn_plot_values_expanded.npz")

# -------------------------------------
# Create multipanel figure (20 panels)
# -------------------------------------
matplotlib.rcParams.update({'font.size': 9, 'figure.dpi': 150})
fig = plt.figure(figsize=(11.7, 16.5))  # A4 portrait
gs = gridspec.GridSpec(10, 2, height_ratios=[1]*10, hspace=0.5, wspace=0.35)

# Panel (a) Representative samples grid (top left)
# The outer subplot is used for the main title/label
ax = fig.add_subplot(gs[0, 0])
ax.set_title('(a) Representative samples', loc='left', fontsize=11, fontweight='bold', pad=20)
ax.axis('off')
sub_gs = gs[0, 0].subgridspec(2, 5, wspace=0.05, hspace=0.05)
n_samples_display = 5
cols = 5
images_to_show = np.vstack([mnist_samples, fashion_samples])
for i, img in enumerate(images_to_show):
    r = i // cols
    c = i % cols
    ax_img = fig.add_subplot(sub_gs[r, c])
    ax_img.imshow(img, cmap='gray', interpolation='nearest')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    if i < n_samples_display:
        ax_img.set_title(f"MN {i}", fontsize=7)
    else:
        ax_img.set_title(f"F {i - n_samples_display}", fontsize=7)
#label_ax(ax, '(a) Representative samples') # Title is now part of the subplot

# Panel (b) Intensity transforms
ax = fig.add_subplot(gs[0, 1])
bars = ['Orig MN mean', 'QPE MN mean (est)', 'Orig FM mean', 'QPE FM mean (est)']
vals = [mnist_mean, mnist_mean * qpe_gain_mnist_mean, fashion_mean, fashion_mean * qpe_gain_fashion_mean]
ax.bar(range(len(vals)), vals)
ax.set_xticks(range(len(vals)))
ax.set_xticklabels(bars, rotation=18, ha='right')
ax.set_ylabel('Mean intensity')
ax.set_title('Intensity transforms under QPE')
label_ax(ax, '(b)')

# Panel (c) QPE spectrum
ax = fig.add_subplot(gs[1, 0])
ax.plot(qpe_x, qpe_y, lw=1.2)
ax.set_xlabel('Normalized frequency')
ax.set_ylabel('QPE amplitude (arb.)')
ax.set_title('Synthetic QPE spectrum')
ax.grid(True, alpha=0.25)
label_ax(ax, '(c)')

# Panel (d) AA amplitude vs iterations
ax = fig.add_subplot(gs[1, 1])
ax.plot(m_iters, aa_vals, marker='o')
ax.set_xlabel('Iterations m')
ax.set_ylabel('Amplitude')
ax.set_title('Amplitude Amplification (AA)')
ax.grid(True)
label_ax(ax, '(d)')

# Panel (e) Fidelity vs padding
ax = fig.add_subplot(gs[2, 0])
ax.plot(P_vals, fidelity, marker='s', label='Fidelity')
ax.plot(P_vals, Delta_xi_min / np.max(Delta_xi_min), '--', label='Normed Δξ_min')
ax.set_xlabel('Padding P')
ax.set_ylabel('Metric')
ax.set_title('Fidelity & spectral resolution vs padding')
ax.legend(frameon=False)
label_ax(ax, '(e)')

# Panel (f) t-SNE
ax = fig.add_subplot(gs[2, 1])
ax.scatter(X_embedded[:n_embed_per,0], X_embedded[:n_embed_per,1], s=8, c='tab:blue', alpha=0.7, label='MNIST')
ax.scatter(X_embedded[n_embed_per:,0], X_embedded[n_embed_per:,1], s=8, c='tab:orange', alpha=0.7, label='Fashion')
ax.legend(markerscale=2)
ax.set_title('t-SNE of combined flattened features')
label_ax(ax, '(f)')

# Panel (g) Training curves + entropy
ax = fig.add_subplot(gs[3, 0])
ax.plot(epochs, train_acc, label='Train acc')
ax.plot(epochs, val_acc, label='Val acc')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax_twin = ax.twinx()
ax_twin.plot(epochs, quantum_entropy, label='Quantum entropy', lw=1.0, alpha=0.6)
ax_twin.set_ylabel('Quantum entropy')
ax.set_title('Training dynamics & quantum entropy')
ax.legend(loc='upper left')
ax_twin.legend(loc='upper right')
label_ax(ax, '(g)')

# Panel (h) Loss with uncertainty
ax = fig.add_subplot(gs[3, 1])
ax.plot(epochs, ensemble_loss_mean, label='Loss mean')
ax.fill_between(epochs, ensemble_loss_mean - ensemble_loss_std, ensemble_loss_mean + ensemble_loss_std, alpha=0.2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Ensemble loss with uncertainty')
label_ax(ax, '(h)')

# Panel (i) Feature variance vs accuracy
ax = fig.add_subplot(gs[4, 0])
feature_variance = 0.35 + 0.65 * (1 - np.exp(-0.09 * (epochs - 1)))
ax.plot(epochs, feature_variance, label='Feature variance')
ax.plot(epochs, train_acc, label='Train acc', linestyle='--')
ax.set_xlabel('Epoch')
ax.set_ylabel('Metric')
ax.set_title('Feature variance — accuracy coevolution')
ax.legend()
label_ax(ax, '(i)')

# Panel (j) Confusion matrix
ax = fig.add_subplot(gs[4, 1])
im = ax.imshow(cm, interpolation='nearest', cmap='viridis')
ax.set_title('Simulated confusion matrix (Fashion subset)')
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(j)')

# Panel (k) FFT magnitude of sample image
ax = fig.add_subplot(gs[5, 0])
ax.imshow(np.log1p(np.abs(Fimg)), cmap='inferno')
ax.set_title('log |FFT(image)|')
ax.axis('off')
label_ax(ax, '(k)')

# Panel (l) Reconstruction after freq hadamard product
ax = fig.add_subplot(gs[5, 1])
ax.imshow(norm01(recon_img), cmap='gray')
ax.set_title('Reconstruction after frequency elementwise product')
ax.axis('off')
label_ax(ax, '(l)')

# Panel (m) MNIST class mean heatmaps (10 subpanels in mini-grid)
# This was corrected to use a subgridspec instead of fig.add_axes
ax = fig.add_subplot(gs[6, 0])
ax.set_title('(m) MNIST per-class mean images', loc='left', fontsize=11, fontweight='bold', pad=20)
ax.axis('off')
sub_gs_m = gs[6, 0].subgridspec(2, 5, wspace=0.05, hspace=0.05)
for i in range(10):
    r, c = divmod(i, 5)
    ax_img = fig.add_subplot(sub_gs_m[r, c])
    ax_img.imshow(mnist_class_means[i], cmap='gray')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    ax_img.set_title(f"MN {i}", fontsize=7)

# Panel (n) FashionMNIST class mean heatmaps
# This was corrected to use its own grid cell and subgridspec
ax = fig.add_subplot(gs[6, 1])
ax.set_title('(n) FashionMNIST per-class mean images', loc='left', fontsize=11, fontweight='bold', pad=20)
ax.axis('off')
sub_gs_n = gs[6, 1].subgridspec(2, 5, wspace=0.05, hspace=0.05)
for i in range(10):
    r, c = divmod(i, 5)
    ax_img = fig.add_subplot(sub_gs_n[r, c])
    ax_img.imshow(fashion_class_means[i], cmap='gray')
    ax_img.set_xticks([]); ax_img.set_yticks([])
    ax_img.set_title(f"F {i}", fontsize=7)

# Panel (o) Inter-class cosine similarity matrix (20x20)
ax = fig.add_subplot(gs[7, 0])
im = ax.imshow(cosine_sim, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Inter-class cosine similarity (MN+Fash)')
ax.set_xticks(range(20)); ax.set_yticks(range(20))
ax.set_xticklabels([f"MN{c}" for c in range(10)] + [f"F{c}" for c in range(10)], rotation=90, fontsize=6)
ax.set_yticklabels([f"MN{c}" for c in range(10)] + [f"F{c}" for c in range(10)], fontsize=6)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(o)')

# Panel (p) Kernel spectral profile comparison
ax = fig.add_subplot(gs[7, 1])
# compute radial-averaged FFT magnitude for each kernel sized to 64x64
def pad_to(size, arr):
    out = np.zeros((size,size))
    s0,s1 = arr.shape
    start0 = (size - s0)//2
    start1 = (size - s1)//2
    out[start0:start0+s0, start1:start1+s1] = arr
    return out

size = 64
kx = np.linspace(0, 1, size)
for idx,k in enumerate(kernels):
    padk = pad_to(size, k)
    Kf = np.abs(fftshift(fft2(padk)))
    # radial average
    cx, cy = size//2, size//2
    rs = np.sqrt((np.arange(size) - cx)[:,None]**2 + (np.arange(size) - cy)[None,:]**2)
    rmax = int(np.max(rs))
    bins = np.linspace(0, rmax, 200)
    radial = []
    for i in range(len(bins)-1):
        mask = (rs >= bins[i]) & (rs < bins[i+1])
        v = np.mean(Kf[mask]) if np.any(mask) else 0
        radial.append(v)
    radial = np.array(radial)
    radial = radial / (radial.max() + 1e-12)
    ax.plot(radial, label=f'Kernel {idx+1}')
ax.set_title('Kernel spectral (radial) profiles (normed)')
ax.set_xlabel('Radial bin')
ax.legend(frameon=False, fontsize=7)
label_ax(ax, '(p)')

# Panel (q) Correlation heatmap between derived metrics
ax = fig.add_subplot(gs[8, 0])
im = ax.imshow(metric_corr, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Correlation matrix between metrics (fidelity, Δξ, AA proxy, entropy proxy)')
ax.set_xticks(range(metric_corr.shape[0])); ax.set_yticks(range(metric_corr.shape[0]))
ax.set_xticklabels(['Fid','Δξ','AAproxy','Entropy'], rotation=45, fontsize=8)
ax.set_yticklabels(['Fid','Δξ','AAproxy','Entropy'], fontsize=8)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
label_ax(ax, '(q)')

# Panel (r) PCA explained variance
ax = fig.add_subplot(gs[8, 1])
ax.plot(np.arange(1, len(explained_var)+1), np.cumsum(explained_var), marker='o')
ax.set_xlabel('Number of PCs')
ax.set_ylabel('Cumulative explained variance')
ax.set_title('PCA cumulative explained variance (sample)')
ax.grid(True)
label_ax(ax, '(r)')

# Panel (s) Class-wise variance boxplots (Fashion)
ax = fig.add_subplot(gs[9, 0])
ax.boxplot(class_vars, labels=[str(i) for i in range(10)], showfliers=False)
ax.set_title('Per-class sample variance distribution (FashionMNIST)')
ax.set_xlabel('Class')
ax.set_ylabel('Pixel-wise sample variance')
label_ax(ax, '(s)')

# Panel (t) 1-vs-rest ROC-like synthetic curves
ax = fig.add_subplot(gs[9, 1])
y_bin = label_binarize(y_true_subset, classes=np.arange(10))
for i, cls in enumerate(classes_for_roc):
    fpr, tpr, _ = roc_curve((y_true_subset == cls).astype(int), scores[:, i])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'Class {cls} (AUC={roc_auc:.2f})')
ax.plot([0,1],[0,1],'k--', lw=0.5)
ax.set_title('1-vs-rest ROC-like curves (synthetic scores)')
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.legend(fontsize=7)
label_ax(ax, '(t)')

plt.suptitle('Expanded complex scientific multipanel figure for QCNN results analysis', fontsize=14, y=0.995)
plt.savefig(out_pdf, bbox_inches='tight', dpi=300)
np.savez('qcnn_plot_values_expanded.npz')
plt.close(fig)

print("\nAll done. Files produced:")
print(" -", out_pdf)
print(" - qcnn_plot_values_expanded.npz")


Downloading/loading MNIST and FashionMNIST via torchvision (if not present)...

--- Numeric values summary (saved to qcnn_plot_values_expanded.npz) ---
P_vals: [ 0  1  2  3  4  5  6  7  8  9 10]
Delta_xi_min (rounded): [2.0000e-01 1.5576e-01 7.3576e-02 2.1080e-02 3.6630e-03 3.8600e-04
 2.5000e-05 1.0000e-06 0.0000e+00 0.0000e+00 0.0000e+00]
fidelity (rounded): [-1.30000e-05  3.93457e-01  8.64652e-01  9.88878e-01  9.99652e-01
  9.99984e-01  9.99987e-01  9.99987e-01  9.99987e-01  9.99987e-01
  9.99987e-01]
qpe_y sample (first 12): [0.007491 0.019014 0.01464  0.011973 0.00312  0.00312  0.001162 0.017324
 0.012022 0.014161 0.000412 0.019398]
aa_vals (m=0..10): [ 0.353553  0.883883  0.972272  0.574524 -0.110485 -0.740252 -0.999893
 -0.759587 -0.139488  0.550356  0.965021]
mnist_mean, fashion_mean: 0.118459 0.297462
combined cosine_sim shape: (20, 20)
explained_var first 10: [0.24653 0.10528 0.04534 0.04005 0.03593 0.03099 0.02696 0.02374 0.02143
 0.0209 ]
confusion matrix shape: (10, 10)
Sa